# □ 機械学習パイプライン：２値分類問題 with k-fold Cross Validation

Version 15.0  made on 2025.5.3

https://chatgpt.com/share/68148702-d520-8005-aeb5-c3bb3d267bc3

# ◇ TabNet使用時は、最初にRuntime ⇒ Change Runtime Type から GPUを選択する

# ◆ 環境設定 (最初に実行するだけで良い)

## 1) ランタイムに接続 (Google Colaboratoryで使用する場合)

In [ ]:
from google.colab import drive
import sys
from subprocess import check_output, CalledProcessError

# 1) Mount Google Drive
drive.mount('/content/drive')

# 2) Print Python version
print(f"Python version: {sys.version.split()[0]}")

# 3) Detect whether GPU is available
gpu_available = False
gpu_name = None

# Try with PyTorch (available by default in Colab)
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
except ImportError:
    pass

# Fallback to nvidia-smi if torch isn’t available or didn’t detect
if not gpu_available:
    try:
        output = check_output(
            ["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"],
            stderr=open('/dev/null', 'w')
        ).decode().strip().split('\n')
        if output and output[0]:
            gpu_available = True
            gpu_name = output[0]
    except (FileNotFoundError, CalledProcessError):
        gpu_available = False

# 4) Print engine and GPU name
if gpu_available:
    print("Engine: GPU")
    print(f"GPU: {gpu_name}")
    !nvidia-smi
else:
    print("Engine: CPU")


## 2) Parameterの設定と必要なLibrary のインストール (四つのセルを順に評価する)

In [ ]:
# Optuna trial counts
n_trialsA = 80  # Lasso, Ridge
n_trialsB = 20  # RandomForest
n_trialsC = 60  # XGBoost
n_trialsD = 40  # LightGBM

# Optuna trial counts
# model                 n_trials default
# Lasso & Ridge         60
# Random Forest         20
# XGBoost               50
# LightGBM              40
# YDF                   30
# LightGBMTuner         500 (num_boost_round)

# Yggldrasi Decision Forest default parameters
# n1 32  n2 32  step_size

# SHAP value sample size
shap_sample_size = 100

# SHAPを行うか否か
shap_for_random_forest = True
shap_for_xgboost = True

# LIMEを行うか否か
perform_lime = True

# Feature importance figure size
size_x = 8
size_y = 4.8

# Default values for the project name and folder
default_project_name = 'MLclassification'
default_project_folder = 'AI'
default_random_state = 42
default_test_size = 0.2

# Function to safely convert input to float
def get_float_input(prompt, default):
    while True:
        user_input = input(prompt) or default
        try:
            value = float(user_input)
            if 0.0 < value < 1.0:
                return value
            else:
                print("Please enter a float between 0.0 and 1.0.")
        except ValueError:
            print("Invalid input. Please enter a valid float.")


# Function to safely convert input to int
def get_int_input(prompt, default):
    while True:
        user_input = input(prompt) or default
        try:
            value = int(user_input)
            if 0 <= value <= 4294967295:
                return value
            else:
                print('Please enter an integer between 0 and 4294967295(=2**32-1)')
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

print('■ Set a random_state [Press Return to use the default value: 42]')
random_state = get_int_input(f'Enter the random state (自然数) (default {default_random_state}): ', default_random_state)
print('')
print(f'◇ Random State: {random_state}')
print('')

print('■ Set a test size [Press Return to use the default value: 0.2]')
test_size = get_float_input(f'Enter the test size (0〜1) (default {default_test_size}): ', default_test_size)
print('')
print(f'◇ Test Size: {test_size}')
print('')


# Optunaのインストール
!pip install optuna -q

# XGBoostのインストール
!pip3 install xgboost -q
!pip3 install -q pydot
!pip3 install graphviz -q

# LightGBMTunerCVのためのインストール
!pip install optuna-integration[lightgbm] -q

# CatBoostのインストール
!pip install catboost --quiet

# 拡張子 xls を読み込むためのライブラリーのインストール
!pip install xlrd -q

# rickのインストール
# !pip install rich

# SHAPとLIMEのインストール
!pip install shap -q
!pip install lime -q
!pip install imgkit -q

# Noto Sans CJK JPフォントのダウンロードとインストール
!apt-get update -qq
!apt-get install -y -qq fonts-noto-cjk

# lifelinesのインストール (2024年7月の時点でErrorが出るがこのProgramには支障は出ない)
!pip install lifelines --quiet
print('')

# Yggdrasil Decision Forestのインストール
!pip install ydf -U -q

# TabNetのインストール
!pip install pytorch-tabnet -q
# !pip install dill

# 警告文の抑制
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 3) 分析対象の(xlsx, xls, csv)の読み込み: スクロールして出力の「ファイル選択」をクリック

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd

def load_excel_file():
    # ファイルアップロードのためのダイアログを表示
    uploaded = files.upload()

    # アップロードされたファイル名を取得
    if uploaded:
        for filename in uploaded.keys():
            # ファイルの拡張子がExcelファイルか確認
            if filename.endswith('.xlsx') or filename.endswith('.xls'):
                # Excelファイルをpandasで読み込む
                df = pd.read_excel(filename)
                print('')
                print(f"読み込んだデータ:  {filename}")
                print('')
            elif filename.endswith('.csv'):
                df = pd.read_csv(filename)
                print('')
                print(f'読み込んだファイル:  {filename}')
                print('')
            else:
                print(f"アップロードされたファイル '{filename}' はExcelまたはCSVのファイルではありません。")
    else:
        print("ファイルがアップロードされませんでした。")

    return df

# 関数を呼び出してExcelファイルを読み込む
df = load_excel_file()
print('')
row0, col0 = df.shape
print(f'■ データの次元: {row0}行 {col0}列')
print('')
print(df.head(10))
print('')


# データフレームに空のセルが一つでもあるかを確認
if df.isnull().values.any():
    # 空のセル（NaN）の数をカウント
    num_missing = df.isna().sum().sum()
    # 全てのセルの数をカウント
    total_cells = df.size
    # 空のセルの割合を計算
    missing_percentage = (num_missing / total_cells) * 100
    print(f"◇ データフレームには{num_missing}個の空のセル(全セルの{round(missing_percentage,3)}%)があります。")
    print('')

    # ユーザーに空のセルを削除するか中央値で置換するかを尋ねる
    choice = input("◆ 空欄を削除しますか？ 中央値で補間しますか？ (削除/補間): \n")

    if choice == '削除':
        print('')
        print('■ 空のセルを含むサンプルを削除します。')
        print('')
        initial_count = len(df)
        df.dropna(inplace=True)
        final_count = len(df)
        deleted_count = initial_count - final_count
        row, col = df.shape
        print(f'□ {deleted_count} サンプル ({round(100*deleted_count/initial_count,2)}%) が削除されました。')
        print(f'□ {final_count} サンプルが残っています。')
        print('')
        print(f'■ 削除後のデータの次元: {row}行 {col}列')
        print('')
        print('空のセルを削除した後のデータ')
        print('')
        print(df.head(10))
    elif choice == '補間' or choice == '置換' or choice == '補完':
        print('')
        print('■ 空欄は中央値で置換します。')
        print('')
        samples_replaced = 0
        # 各列で空のセルをチェックし、中央値で置き換える
        for column in df.columns:
            if df[column].isnull().any():  # 空のセルがあるかどうかを確認
                median = df[column].median()  # その列の中央値を計算
                samples_replaced += df[column].isnull().sum()  # 置換するサンプルの数をカウント
                df[column].fillna(median, inplace=True)  # 空のセルを中央値で置き換え
        row, col = df.shape
        print(f'□ {samples_replaced} サンプル ({round(100*samples_replaced/row,2)}%) の空欄が中央値で置換されました。')
        print('')
        print('')
        print(f'■ 置換後のデータの次元: {row}行 {col}列')
        print('')
        print('空欄を中央値で置換した後のデータ')
        print('')
        print(df.head(10))
    else:
        print('◇ 無効な選択です。何も変更しませんでした。')
else:
    print("◇ データフレームに空のセルはありません。")


# Numpyのデータの作成
data = df.values
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# k setting for cross-validation
n_samples = len(df)
min_test_samples = 50
k = max(2, np.floor(n_samples / min_test_samples).astype(int))
max_k = 10
k = min(k, max_k)
if k < 3:
    k = 3


# ◆ プログラム：プロジェクト名と保存フォルダーの作成 (再計算のたびに実行する)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
from google.colab import files
import pickle
import os, sys
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import warnings
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=UserWarning)
from lime import lime_tabular


# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)
# Matplotlibのデフォルトフォント設定を変更
# plt.rcParams['font.family'] = font_prop.get_name()



# Detect whether GPU is available
gpu_available = False
gpu_name = None
# Try with PyTorch (available by default in Colab)
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
except ImportError:
    pass
# Fallback to nvidia-smi if torch isn’t available or didn’t detect
if not gpu_available:
    try:
        output = check_output(
            ["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"],
            stderr=open('/dev/null', 'w')
        ).decode().strip().split('\n')
        if output and output[0]:
            gpu_available = True
            gpu_name = output[0]
    except (FileNotFoundError, CalledProcessError):
        gpu_available = False



# 計算結果の保存先の指定
print('')
print('■ Set a Project Name [Press Return to use the default name: MLclassification]')

# Prompt the user for input, with defaults if nothing is entered
project_name = input(f"Enter the project name (default: {default_project_name}): ") or default_project_name
print('')
print(f'◇ プロジェクト名: {project_name}')

print('')
print('■ Set a Project Folder [Press Return to use the default folder: AI]')

project_folder = input(f"Enter the project folder (default: {default_project_folder}): ") or default_project_folder

# Directory settings based on user input or defaults
project_directory = '/content/drive/MyDrive/' + str(project_folder) + '/' + str(project_name)
print('')
print(f'◇ プロジェクトフォルダ: {project_directory}')

# Run number logic
try:
    runnumber
    runnumber += 1
except NameError:
    runnumber = 1

# Create directories for results, models, and figures
import os
def create_directory(project_directory, folder_name, runnumber):
    path_name = 'path_' + str(folder_name)
    while True:
        # ディレクトリパスを作成
        path_new = os.path.join(project_directory, str(folder_name), f'run{runnumber}')

        # ディレクトリが存在しない場合は作成
        if not os.path.exists(path_new):
            os.makedirs(path_new)
            # print(f"Directory created: {path_new}")
            return path_new

        # ディレクトリが存在し、かつ、空である場合はそのディレクトリを使用
        elif os.path.isdir(path_new) and not os.listdir(path_new):
            # print(f"Directory already exists and is empty: {path_new}")
            return path_new

        # ディレクトリが存在し、かつ、空でない場合はrunnumberを増やして次のループで再試行
        else:
            # print(f"Directory already exists and is not empty: {path_new}. Incrementing runnumber.")
            runnumber += 1

path_table = create_directory(project_directory, 'table', runnumber)
path_model = create_directory(project_directory, 'model', runnumber)
path_figure = create_directory(project_directory, 'figure', runnumber)


# Print the paths
print('')
print(f'◇ 作成した表の保存先: {path_table}')
print('')
print(f'◇ 予測モデルの保存先: {path_model}')
print('')
print(f'◇ 作成した図の保存先: {path_figure}')
print('')


from sklearn.model_selection import train_test_split
import pandas as pd

# assuming feature_names and label_column are already defined, e.g.:
feature_names = df.columns[:-1]
label_column  = df.columns[-1]

try:
    # perform numpy split
    x_train, x_test, t_train, t_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state
    )

    # build pandas versions
    x_train_pand = pd.DataFrame(x_train, columns=feature_names)
    x_test_pand  = pd.DataFrame(x_test,  columns=feature_names)
    t_train_pand = pd.Series(t_train, name=label_column)
    t_test_pand  = pd.Series(t_test,  name=label_column)

    print("◇ Train-test split successful (numpy & pandas).")
except Exception as e:
    print(f"◇ An error occurred during train-test split: {e}")





# 以下補助関数の定義

# Sigmoid関数 (to convert a numeric output into a probability in LIME)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 出力をProbability scoreに変換する関数
def predict_proba(model, Z):
    predictions = model.predict(Z)
    return np.vstack([1-sigmoid(predictions), sigmoid(predictions)]).T




# SHAPの共通関数。回帰問題にも分類問題にも対応
def ShapValueFunction(model_made,
                      df_data,
                      x_train_data,
                      x_test_data,
                      size_x_value,
                      size_y_value,
                      path_figure_info,
                      path_table_info,
                      figure_name,
                      excel_name,
                      method_name,
                      task_type='classification',
                      model_type='general'):
    """
    SHAP値およびBeeswarm図を作成し、結果を保存する関数。

    Parameters:
    - model_made: 学習済みモデル
    - df_data: データフレーム（特徴量とターゲットを含む）
    - x_train_data: 学習データの特徴量
    - x_test_data: テストデータの特徴量
    - size_x_value: プロットの横サイズ
    - size_y_value: プロットの縦サイズ
    - path_figure_info: 図の保存先パス
    - path_table_info: テーブル（Excel）の保存先パス
    - figure_name: 図のファイル名（拡張子なし）
    - excel_name: Excelファイル名（拡張子なし）
    - method_name: モデル名や手法名
    - task_type: 'regression' または 'classification'
    - model_type: 'tree', 'naive_bayes', 'general' など
    """

    print('')
    print(f'■ {method_name} モデルにおける 各変数の SHAP値（{task_type} 問題）')
    print('')

    import shap

    try:
        feature_names = df_data.columns[:-1]
        # SHAPエクスプレイナーの初期化
        if model_type == 'tree':
            # 決定木ベースのモデル用
            explainer = shap.TreeExplainer(model_made)
            shap_values = explainer(x_test_data)

        elif model_type == "ydf":
            X_train_df = pd.DataFrame(x_train_data, columns=feature_names)
            X_test_df = pd.DataFrame(x_test_data, columns=feature_names)
            explainer = shap.TreeExplainer(model_made, X_train_df)

            # create prediction function that provides column-wide dict
            def ydf_predict(X):
                X_df = pd.DataFrame(X, columns=feature_names)
                X_dict = {col: X_df[col].values for col in X_df.columns}
                return model_made.predict(X_dict)

            explainer = shap.Explainer(ydf_predict, X_train_df)
            shap_values = explainer(X_test_df)

        else:
            # その他のモデル用
            explainer = shap.Explainer(model_made, x_train_data)
            shap_values = explainer(x_test_data)

        print('□ BarPlot of SHAP values')
        # SHAP値のバーグラフをプロット
        plt.figure(figsize=(size_x_value, size_y_value))
        shap.summary_plot(shap_values,
                          x_test_data,
                          feature_names=df_data.columns[:-1],
                          plot_type="bar",
                          show=False)

        # Y軸のラベルフォントを変更
        ax = plt.gca()
        for label in ax.get_yticklabels():
            try:
                label.set_fontproperties(font_prop)
            except:
                pass

        # SHAP値のバーグラフを保存
        barplot_path = os.path.join(path_figure_info, f"{figure_name}_barplot.png")
        plt.savefig(barplot_path, bbox_inches='tight')
        plt.show()
        plt.close()  # メモリ解放のためプロットを閉じる
        print(f"BarPlot saved to: {barplot_path}")
        print('')

        print('□ DotPlot of SHAP values (Beeswarm Plot)')
        # SHAP値のドットプロット（ビースウォームプロット）をプロット
        plt.figure(figsize=(size_x_value, size_y_value))
        shap.summary_plot(shap_values,
                          x_test_data,
                          feature_names=df_data.columns[:-1],
                          plot_type="dot",
                          show=False)

        # Y軸のラベルフォントを変更
        ax = plt.gca()
        for label in ax.get_yticklabels():
            try:
                label.set_fontproperties(font_prop)
            except:
                pass

        # SHAP値のドットプロットを保存
        dotplot_path = os.path.join(path_figure_info, f"{figure_name}_dotplot.png")
        plt.savefig(dotplot_path, bbox_inches='tight')
        plt.show()
        plt.close()  # メモリ解放のためプロットを閉じる
        print(f"DotPlot saved to: {dotplot_path}")
        print('')

        # 各特徴量の平均絶対SHAP値を計算
        # feature_names = df_data.columns[:-1]
        if isinstance(shap_values, shap.Explanation):
            shap_sum = np.abs(shap_values.values).mean(axis=0)
        else:
            shap_sum = np.abs(shap_values).mean(axis=0)

        # SHAP値をデータフレームに変換し、降順にソート
        shap_df = pd.DataFrame(shap_sum, index=feature_names, columns=["SHAP Value"]).sort_values(by="SHAP Value", ascending=False)

        # SHAP値をExcelファイルとしてエクスポート
        shap_excel_filename = os.path.join(path_table_info, f"{excel_name}.xlsx")
        shap_df.to_excel(shap_excel_filename, index=True)

        print(f"SHAP値が {shap_excel_filename} に保存されました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        print("SHAP値の計算およびプロットをスキップします。")



# LIME解析（TabNet含む汎用対応）、インライン表示＋3種類の図を保存
# 表示コードはそのまま、保存処理のみ追加しています
def LimeContributionValueFunction(
    model_made,
    df_data,
    x_train_data,
    x_test_data,
    t_test_data,
    size_x_value,
    size_y_value,
    path_figure_info,
    path_table_info,
    figure_name,
    excel_name,
    method_name,
    task_type='classification',
    random_state=42,
    model_type='general'
):

    from lime import lime_tabular
    import numpy as np
    import pandas as pd
    import os
    import matplotlib.pyplot as plt
    from IPython.display import display

    print('')
    print(f'■ 各症例における {method_name} モデルのLIME分析（{task_type} 問題）')
    print('')

    feature_names = df_data.columns[:-1]
    label_column  = df_data.columns[-1]
    num_features = min(len(feature_names), 10)

    # --- データ整形 ---
    if model_type == 'YDF':
        X_train = pd.DataFrame(x_train_data, columns=feature_names) if isinstance(x_train_data, np.ndarray) else x_train_data.copy()
        X_test  = pd.DataFrame(x_test_data,  columns=feature_names) if isinstance(x_test_data,  np.ndarray) else x_test_data.copy()
        y_test  = pd.Series(t_test_data, name=label_column)       if isinstance(t_test_data, np.ndarray) else t_test_data.copy()
    else:
        X_train = x_train_data.values if hasattr(x_train_data, 'values') else np.array(x_train_data)
        X_test  = x_test_data.values  if hasattr(x_test_data,  'values') else np.array(x_test_data)
        y_test  = t_test_data.values  if hasattr(t_test_data,  'values') else np.array(t_test_data)

    # --- LIME Explainer ---
    if task_type == 'classification':
        mode = 'classification'; class_names = ['NegativeClass','PositiveClass']
    elif task_type == 'regression':
        mode = 'regression';     class_names = None
    else:
        raise ValueError("task_type must be 'classification' or 'regression'")

    explainer = lime_tabular.LimeTabularExplainer(
        training_data=np.array(X_train),
        feature_names=feature_names.tolist(),
        class_names=class_names,
        mode=mode,
        discretize_continuous=True
    )

    # --- 推論関数生成 ---
    def get_predict_fn(model, feature_names, model_type='general'):
        if model_type in ['general','YDF']:
            def pred_fn(x):
                x = np.array(x); x = np.atleast_2d(x) if x.ndim==1 else x
                df_in = pd.DataFrame(x, columns=feature_names)
                preds = np.array(model.predict(df_in)).flatten()
                return np.column_stack([1-preds, preds])
            return pred_fn
        elif model_type=='TabNet':
            def pred_fn(x):
                x = np.array(x); x = np.atleast_2d(x) if x.ndim==1 else x
                df_in = pd.DataFrame(x, columns=feature_names)
                probs = []
                for idx in range(len(df_in)):
                    out = np.array(model.predict(df_in.iloc[[idx]]))
                    if out.size==1: p=float(out)
                    elif out.ndim==2 and out.shape[0]==1: p=out[0,1]
                    else: raise ValueError(f"Unexpected: {out.shape}")
                    probs.append(p)
                return np.column_stack([1-np.array(probs), np.array(probs)])
            return pred_fn
        else:
            raise ValueError(f"Unsupported model_type: {model_type}")

    base_predict_fn = get_predict_fn(model_made, feature_names, model_type)

    # --- インスタンスサンプリング ---
    rng = np.random.RandomState(random_state)
    total = X_test.shape[0]
    n_inst = min(max(1, total//10), 100)
    idxs = rng.choice(np.arange(total), size=n_inst, replace=False)[:2]

    os.makedirs(path_figure_info, exist_ok=True)
    os.makedirs(path_table_info, exist_ok=True)

    for i in idxs:
        target = y_test.iloc[i] if model_type=='YDF' else y_test[i]
        print(f"□ インスタンス {i} (target={target}) の LIME解析")
        print('')

        # --- predict_fn 切替 ---
        if task_type=='classification':
            predict_fn = base_predict_fn if model_type=='TabNet' else (
                model_made.predict_proba if hasattr(model_made,'predict_proba') else base_predict_fn
            )
        else:
            predict_fn = model_made.predict

        inst = X_test.iloc[i].values if model_type=='YDF' else X_test[i]
        exp  = explainer.explain_instance(inst, predict_fn, num_features=num_features)

        # --- 1) デフォルトLIMEプロット（表示＆保存） ---
        exp.show_in_notebook(show_table=True)
        fig1 = exp.as_pyplot_figure(); plt.show()
        fig1_path = os.path.join(path_figure_info, f"{figure_name}_instance_{i}_default.png")
        fig1.savefig(fig1_path, bbox_inches='tight')
        plt.close(fig1)
        print('')
        print('')

        # --- 2) 横棒グラフ（表示＆保存） ---
        df_cont = pd.DataFrame({f:[c] for f,c in exp.as_list()}).T
        df_cont.columns = [f'Instance_{i}']


        # --- 3) テーブルプロット（表示＆保存） ---
        display(df_cont)
        fig3, ax3 = plt.subplots(figsize=(size_x_value, size_y_value))
        ax3.axis('off')
        tbl = ax3.table(
            cellText=df_cont.values,
            rowLabels=df_cont.index,
            colLabels=df_cont.columns,
            loc='center'
        )
        tbl.auto_set_font_size(False); tbl.set_fontsize(8)
        plt.show()
        fig3_path = os.path.join(path_table_info, f"{excel_name}_instance_{i}_table.png")
        fig3.savefig(fig3_path, bbox_inches='tight')
        plt.close(fig3)
        print('')
        print('')

        # --- Excel保存 ---
        excel_path = os.path.join(path_table_info, f"{excel_name}_instance_{i}.xlsx")
        df_cont.to_excel(excel_path)

        print(
            f"Saved figures for instance {i}:\n"
            f"  1) Default: {os.path.basename(fig1_path)}\n"
            # f"  2) Barh:    {os.path.basename(fig2_path)}\n"
            f"  2) Table:   {os.path.basename(fig3_path)}\n"
            f"And Excel:   {os.path.basename(excel_path)}\n"
        )
        print('')
        print('')




def display_fold_averages(
        k,
        df,
        fold_cm_percent,
        fold_tp_rate,
        fold_fp_rate,
        fold_fn_rate,
        fold_tn_rate,
        mean_acc,
        mean_prec,
        mean_rec,
        mean_f1,
        mean_spec,
        mean_auc,
        mean_cind,
        mean_prerec,
        model_name
        ):

    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "LightGBMTunerOptuna kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "lightgbmtuneroptuna_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "LightGBMOptuna kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "lightgbmoptuna_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "LightGBM kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "lightgbm_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "MLP kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "mlp_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "AdaBoost kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "adaboost_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "GradientBoosting kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "gradientboosting_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "GaussianProcess kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "gaussianprocess_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "LinearDiscriminantAnalysis kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    hmname = "lineardiscriminantanalysis_confusion_heatmap_kfCV.png"
    hmpath = os.path.join(path_figure, hmname)
    plt.savefig(hmpath)
    plt.show()
    plt.close()

    print('')
    print(f'■ {k}-fold Cross Validation を用いた {model_name} による2値分類の評価結果')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')
    print('□　データのサンプル数: ' + str(len(df)))
    print('')
    print('□　Fold-Averageによる混同行列(%): ')
    print(pd.DataFrame(fold_cm_percent,  index=['Actual Negative', 'Actual Positive'], columns=['Predicted Negative', 'Predicted Positive'] ))
    print('')
    print(f'真陽性 {100*fold_tp_rate:.2f} %, 偽陽性 {100*fold_fp_rate:.2f} %, 偽陰性 {100*fold_fn_rate:.2f} %, 真陰性 {100*fold_tn_rate:.2f} %')
    print('')
    print(f'   Accuracy(正解率):  {mean_acc:.5f}   precision(精度):  {mean_prec:.5f}    recall(再現率=感度):  {mean_rec:.5f}')
    print(f'   f1-score(f1値):  {mean_f1:.5f}   specificity(特異度):  {mean_spec:.5f}')
    print('')



    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                        [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent,
        fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec,
        mean_auc, mean_cind, mean_prerec,
        "LinearDiscriminantAnalysis kfCV"
    )

    print("\n□ fold-averaged confusion matrix heat-map:\n")
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        fold_cm_percent, annot=True, fmt=".2f", cbar=False,
        xticklabels=["Pred 0", "Pred 1"],
        yticklabels=["True 0", "True 1"]
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("fold-averaged Confusion Matrix (%)")
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")

    print(f'   AUC of ROC curve:  {mean_auc:.5f}    C-index (c-Statistics):  {mean_cind:.5f}')
    print(f'   AUC of Precision-Recall plot: {mean_prerec:.5f}')
    print('')






#　　以下、各種機械学習の定義

# simple Linear Regression
def LinearKFold(k=k, show_shap=True, perform_lime=True, threshold=0.5):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import StratifiedKFold, cross_val_predict
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        roc_curve, precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    import numpy as np
    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    print('')
    print('■■■ Linear Regression kfCV　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold instead of KFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    lr = LinearRegression()

    # Lists for fold metrics
    aucs = []
    accs = []
    precs = []
    recs = []
    f1s = []
    specs = []
    aps = []

    # For ROC and Precision-Recall curve plotting per fold
    fprs = []
    tprs = []
    precisions = []
    recalls = []

    # For true fold-averaged confusion matrix
    tn_rates = []
    fp_rates = []
    fn_rates = []
    tp_rates = []


    # ----------------- Stratified K-fold Cross Validation -----------------
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        lr.fit(X_train, y_train)
        predictions = lr.predict(X_test)
        # Binarize predictions for classification metrics
        predictions_round = [1 if p >= threshold else 0 for p in predictions]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, predictions_round).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Compute metrics per fold
        auc_fold = roc_auc_score(y_test, predictions)
        acc_fold = accuracy_score(y_test, predictions_round)
        prec_fold = precision_score(y_test, predictions_round)
        rec_fold = recall_score(y_test, predictions_round)
        f1_fold = f1_score(y_test, predictions_round)
        ap_fold = average_precision_score(y_test, predictions)
        spec_fold = tn / (tn + fp)

        aucs.append(auc_fold)
        accs.append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s.append(f1_fold)
        specs.append(spec_fold)
        aps.append(ap_fold)

        # Compute ROC curve for this fold
        fpr, tpr, _ = roc_curve(y_test, predictions)
        fprs.append(fpr)
        tprs.append(tpr)

        # Compute Precision-Recall curve for this fold
        precision_vals, recall_vals, _ = precision_recall_curve(y_test, predictions)
        precisions.append(precision_vals)
        recalls.append(recall_vals)


    # -----------------Calculate fold-average scores-----------------
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_cind = np.mean(aucs)
    mean_prerec = np.mean(aps)

    hmname = 'linear_confusion_heatmap_kfCV.png'
    rocname = 'linear_roc_curve_kfCV.png'
    prerecname = 'linear_prerec_curve_kfCV.png'
    hmpath=os.path.join(path_figure, hmname)
    rocpath=os.path.join(path_figure, rocname)
    prerecpath=os.path.join(path_figure, prerecname)

    # -----------------True fold-averaged confusion matrix-----------------
    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                         [fold_fp_rate, fold_tp_rate]])
    fold_cm_percent = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_percent, fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec, mean_auc, mean_cind, mean_prerec,
        "Linear Regression kfCV"
    )

    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_percent, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('fold-averaged Confusion Matrix (%)')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmpath)
    plt.show()
    plt.close()


    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr = np.linspace(0, 1, 100)
    # Interpolate each fold's TPR at mean_fpr
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_linear = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'Linear Regression'], dtype=object)

    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocpath)
    plt.show()
    plt.close()


    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        # Sort by recall for interpolation
        idx_sort = np.argsort(recalls[i])
        recall_sorted = recalls[i][idx_sort]
        precision_sorted = precisions[i][idx_sort]
        precisions_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))
    mean_precision_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_precision_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecpath)
    plt.show()
    plt.close()


    # -----------------Retrain the model on the full dataset-----------------
    model_full = LinearRegression()
    model_full.fit(X, y)

    # Saving the full model
    os.makedirs(path_model, exist_ok=True)
    model_filename = os.path.join(path_model, 'linear_model_full.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # -----------------Feature importance (coefficients) extraction-----------------
    feature_importances = model_full.coef_
    column_names = df.columns[:-1]
    feature_importances_df = pd.DataFrame({'Feature': column_names, 'Importance': feature_importances})
    feature_importances_df['Abs_Importance'] = feature_importances_df['Importance'].abs()
    feature_importances_df = feature_importances_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    excel_filename = os.path.join(path_table, 'linear_feature_importances_kfCV.xlsx')
    feature_importances_df.to_excel(excel_filename, index=False)

    # Print top 10 features
    importance_sorted_idx = np.argsort(np.abs(feature_importances))[::-1]
    top_idxs = importance_sorted_idx[:10]
    print('')
    print('')
    print('')
    print('■ Linear Regression model の 係数による Feature Importance')
    print('')

    fig, ax = plt.subplots(figsize=(10, 6))
    # plt.figure(figsize=(size_x, size_y))
    plt.barh(range(len(top_idxs)), feature_importances[top_idxs], align='center')
    plt.yticks(range(len(top_idxs)), [column_names[i] for i in top_idxs])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Linear Regression')
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.gca().invert_yaxis()
    ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'linear_feature_importances_kfCV.png'))
    plt.show()
    plt.close()


    # -----------------Hold-out method for SHAP and LIME analyses-----------------
    model_holdout = LinearRegression()
    model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'linear_shap_values_holdout', 'linear_shap_values_holdout',
            'Linear Regression', task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'linear_lime_explanation_holdout', 'linear_lime_feature_contributions_holdout',
            'Linear Regression', task_type='classification', random_state=random_state
        )
        print('')

    # -----------------Prepare and return results-----------------
    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame([res], columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'], index=['Linear Regression'])

    return [res2, roc_linear]






# Lasso Regression with normalization and fold-Averaged metrics
def LassoKFoldN(alpha=0.01, k=k, show_shap=True, perform_lime=True, threshold=0.5):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Lasso
    from sklearn.metrics import (
        mean_squared_error, roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold
    from sklearn.preprocessing import MinMaxScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    print('')
    print('■■■ Lasso Regression normalized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')


    # ----------------- Stratified K-fold Cross Validation -----------------
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    lasso = Lasso(alpha=alpha, random_state=random_state)

    # Initialize accumulators
    feature_importances = np.zeros((X.shape[1],))
    aucs, accs, precs, recs, f1s, aps = [], [], [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    specs, aps = [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = MinMaxScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        lasso.fit(x_train_scaled, y_train)
        predictions = lasso.predict(x_test_scaled)

        # Feature importance accumulation
        feature_importances += np.abs(lasso.coef_)

        # Binarize for classification metrics
        preds_round = [1 if p >= threshold else 0 for p in predictions]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, preds_round).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # Compute metrics
        aucs.append(roc_auc_score(y_test, predictions))
        accs.append(accuracy_score(y_test, preds_round))
        precs.append(precision_score(y_test, preds_round, zero_division=0))
        recs.append(recall_score(y_test, preds_round))
        f1s.append(f1_score(y_test, preds_round))
        aps.append(average_precision_score(y_test, predictions))
        specs.append(tn/(tn+fp))

        # ROC and PR curves per fold
        fpr, tpr, _ = roc_curve(y_test, predictions)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, predictions)
        precisions.append(prec_vals); recalls.append(rec_vals)


    # -----------------Calculate fold-average scores-----------------
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_ap = np.mean(aps)
    mean_spec = np.mean(specs)
    mean_cind = np.mean(aucs)
    mean_prerec = np.mean(aps)


    # File paths
    hmname = os.path.join(path_figure, 'lasso_normalized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'lasso_normalized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'lasso_normalized_prerec_curve_kfCV.png')

    # fold-averaged confusion matrix
    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                         [fold_fp_rate, fold_tp_rate]])
    fold_cm_pct = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_pct, fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec, mean_auc, mean_cind, mean_prerec,
        "Lasso Regression Normalized kfCV"
    )

    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()



    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_lassoN = np.array([mean_fpr, mean_tpr, mean_auc, 'Lasso Regression normalized'], dtype=object)

    print('')
    print('□ fold-Averaged ROC Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-ROC (AUC={mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()



    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_rec_all = np.linspace(0,1,100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_rec_all, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ fold-Averaged Precision-Recall Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_all, mean_prec_curve, label=f'fold-PR (AP={mean_ap:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()



    # -----------------Feature importance (coefficients) extraction-----------------
    feature_importances /= k
    feature_names = df.columns[:-1]
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    exname = os.path.join(path_table, 'lasso_feature_importances_normalized_kfCV.xlsx')
    feat_df.to_excel(exname, index=False)

    print('')
    print('◇ Top 10 Feature Importances (fold-Averaged):')
    print('')
    top_idx = np.argsort(feature_importances)[-10:]
    top_feats = feature_names[top_idx]
    top_imps = feature_importances[top_idx]
    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Lasso Regression normalized')
    plt.barh(top_feats, top_imps, color='skyblue')
    plt.xlabel('Importance')
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    # ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'lasso_feature_importances_normalized_kfCV.png'))
    plt.show(); plt.close()
    print('')

    # Retrain on full dataset
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = Lasso(alpha=alpha, random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'lasso_model_normalized_full.pkl'), 'wb') as f:
        pickle.dump(model_full, f)


    # -----------------Hold-out method for SHAP and LIME analyses-----------------
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = Lasso(alpha=alpha, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'lasso_normalized_shap_values_holdout', 'lasso_normalized_shap_values_holdout',
            'Lasso Regression normalized', task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'lasso_normalized_lime_explanation_holdout', 'lasso_normalized_lime_feature_contributions_holdout',
            'Lasso Regression normalized', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=[f'Lasso Regression normalized (α={alpha:.4f})']
    )
    return [res2, roc_lassoN]





# Lasso Regression with standardization and fold-Averaged metrics
def LassoKFoldS(alpha=0.01, k=k, show_shap=True, perform_lime=True, threshold=0.5):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Lasso
    from sklearn.metrics import (
        mean_squared_error, roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold
    from sklearn.preprocessing import StandardScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    print('')
    print('■■■ Lasso Regression standardized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')


    # ----------------- Stratified K-fold Cross Validation -----------------
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    lasso = Lasso(alpha=alpha, random_state=random_state)

    # Initialize accumulators
    feature_importances = np.zeros((X.shape[1],))
    aucs, accs, precs, recs, f1s, aps = [], [], [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    specs, aps = [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = StandardScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        lasso.fit(x_train_scaled, y_train)
        predictions = lasso.predict(x_test_scaled)

        # Feature importance accumulation
        feature_importances += np.abs(lasso.coef_)

        # Binarize for classification metrics
        preds_round = [1 if p >= threshold else 0 for p in predictions]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, preds_round).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # Compute metrics
        aucs.append(roc_auc_score(y_test, predictions))
        accs.append(accuracy_score(y_test, preds_round))
        precs.append(precision_score(y_test, preds_round, zero_division=0))
        recs.append(recall_score(y_test, preds_round))
        f1s.append(f1_score(y_test, preds_round))
        aps.append(average_precision_score(y_test, predictions))
        specs.append(tn / (tn + fp))

        # ROC and PR curves per fold
        fpr, tpr, _ = roc_curve(y_test, predictions)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, predictions)
        precisions.append(prec_vals); recalls.append(rec_vals)


    # -----------------Calculate fold-average scores-----------------
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_ap = np.mean(aps)
    mean_spec = np.mean(specs)
    mean_cind = np.mean(aucs)
    mean_prerec = np.mean(aps)

    # File paths
    hmname = os.path.join(path_figure, 'lasso_standardized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'lasso_standardized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'lasso_standardized_prerec_curve_kfCV.png')

    # fold-averaged confusion matrix
    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                         [fold_fp_rate, fold_tp_rate]])
    fold_cm_pct = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_pct, fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec, mean_auc, mean_cind, mean_prerec,
        "Lasso Regression standardized kfCV"
    )

    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()



    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_lassoS = np.array([mean_fpr, mean_tpr, mean_auc, 'Lasso Regression standardized'], dtype=object)

    print('')
    print('□ fold-Averaged ROC Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-ROC (AUC={mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()



    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_rec_all = np.linspace(0,1,100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_rec_all, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ fold-Averaged Precision-Recall Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_all, mean_prec_curve, label=f'fold-PR (AP={mean_ap:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()



    # -----------------Feature importance (coefficients) extraction-----------------
    feature_importances /= k
    feature_names = df.columns[:-1]
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    exname = os.path.join(path_table, 'lasso_feature_importances_standardized_kfCV.xlsx')
    feat_df.to_excel(exname, index=False)

    print('')
    print('◇ Top 10 Feature Importances (fold-Averaged):')
    print('')
    top_idx = np.argsort(feature_importances)[-10:]
    top_feats = feature_names[top_idx]
    top_imps = feature_importances[top_idx]
    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Lasso Regression standardized')
    plt.barh(top_feats, top_imps, color='skyblue')
    plt.xlabel('Importance')
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    # ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'lasso_feature_importances_standardized_kfCV.png'))
    plt.show(); plt.close()
    print('')

    # Retrain on full dataset
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = Lasso(alpha=alpha, random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'lasso_model_standardized_full.pkl'), 'wb') as f:
        pickle.dump(model_full, f)


    # -----------------Hold-out method for SHAP and LIME analyses-----------------
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = Lasso(alpha=alpha, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'lasso_standardized_shap_values_holdout', 'lasso_standardized_shap_values_holdout',
            'Lasso Regression standardized', task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'lasso_standardized_lime_explanation_holdout', 'lasso_standardized_lime_feature_contributions_holdout',
            'Lasso Regression standardized', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=[f'Lasso Regression standardized (α={alpha:.4f})']
    )
    return [res2, roc_lassoS]




#-------------------------------------------------------------------------------



def LassoKFoldOptunaN(k=k, show_shap=True, perform_lime=True, n_trials=50, threshold=0.5, target='logloss'):
    import os
    import pickle
    import numpy as np
    import pandas as pd
    import optuna
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    import seaborn as sns
    from sklearn.linear_model import Lasso, Ridge
    from sklearn.model_selection import StratifiedKFold, KFold
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve,
        average_precision_score, mean_squared_error, log_loss
    )

    # Header for Optuna-based optimization
    print('')
    print(f'■■■ Lasso Regression normalized kfCV with Optuna (Optimizing {target}, Trial count {n_trials})■■■')
    print('')

    # Objective for Optuna: minimize the selected target metric (MSE or log-loss)
    def objective(trial):
        # Suggest alpha
        alpha = trial.suggest_float('alpha', 0.0001, 10.0, log=True)
        # Stratified K-Fold for balanced splits
        kf_inner = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        scores = []
        for train_idx, test_idx in kf_inner.split(X, y):
            xx_train, xx_test = X[train_idx], X[test_idx]
            tt_train, tt_test = y[train_idx], y[test_idx]
            scaler_inner = MinMaxScaler()
            xx_tr_scaled = scaler_inner.fit_transform(xx_train)
            xx_te_scaled = scaler_inner.transform(xx_test)
            model = Ridge(alpha=alpha, random_state=random_state)
            model.fit(xx_tr_scaled, tt_train)
            preds = model.predict(xx_te_scaled)
            if target == 'mse':
                scores.append(mean_squared_error(tt_test, preds))
            elif target == 'logloss':
                preds_clipped = np.clip(preds, 1e-15, 1-1e-15)
                scores.append(log_loss(tt_test, preds_clipped))
            else:
                raise ValueError(f"Unknown target: {target}  The target must be either logloss or mse.")
        return np.mean(scores)

    # Create and run study
    study = optuna.create_study(direction='minimize')
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # Best hyperparameters
    best_alpha = study.best_trial.params['alpha']
    best_trial_no = study.best_trial.number

    # Log best trial
    print('')
    print('')
    print('◇ Hyperparameter Optimization with Optuna for Lasso Regression normalized (detailed logs below)')
    print(f'Best trial #{best_trial_no} for minimizing {target.upper()}:')
    print(f'  Value: {study.best_trial.value:.4f}')
    print('  Params:')
    for key, val in study.best_trial.params.items():
        print(f'    {key}: {val}')
    print('')

    # Retrain on full dataset
    scaler_full = MinMaxScaler()
    X_scaled_full = scaler_full.fit_transform(X)
    best_model_full = Lasso(alpha=best_alpha, random_state=random_state)
    best_model_full.fit(X_scaled_full, y)

    # Save full model
    os.makedirs(path_model, exist_ok=True)
    model_file = os.path.join(path_model, 'lasso_model_full_normalized_kfCV_optuna.pkl')
    with open(model_file, 'wb') as f:
        pickle.dump(best_model_full, f)

    # Header for fold-averaged CV
    print('')
    print('■■■ Lasso Regression normalized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # Stratified K-Fold for fold metrics
    kf_outer = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    # Initialize accumulators
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, pr_prec, pr_rec = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    # Cross-validation loop
    for tr_idx, te_idx in kf_outer.split(X, y):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler_cv = MinMaxScaler()
        X_tr_s = scaler_cv.fit_transform(X_tr)
        X_te_s = scaler_cv.transform(X_te)

        model_cv = Lasso(alpha=best_alpha, random_state=random_state)
        model_cv.fit(X_tr_s, y_tr)
        preds_cont = model_cv.predict(X_te_s)
        preds_bin = [1 if p >= threshold else 0 for p in preds_cont]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_te, preds_bin).ravel()
        tot = tn + fp + fn + tp
        tn_rates.append(tn / tot)
        fp_rates.append(fp / tot)
        fn_rates.append(fn / tot)
        tp_rates.append(tp / tot)
        specs.append(tn / (tn + fp))  # specificity

        # Metrics
        aucs.append(roc_auc_score(y_te, preds_cont))
        accs.append(accuracy_score(y_te, preds_bin))
        precs.append(precision_score(y_te, preds_bin, zero_division=0))
        recs.append(recall_score(y_te, preds_bin))
        f1s.append(f1_score(y_te, preds_bin))
        aps.append(average_precision_score(y_te, preds_cont))

        # Curves
        fpr, tpr, _ = roc_curve(y_te, preds_cont)
        fprs.append(fpr); tprs.append(tpr)
        pr_p, pr_r, _ = precision_recall_curve(y_te, preds_cont)
        pr_prec.append(pr_p); pr_rec.append(pr_r)

    # fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_cind = mean_auc
    mean_prerec = np.mean(aps)

    # fold confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn], [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Lasso Regression normalized with Optuna"
    )

    # Plot heatmap
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'lasso_normalized_confusion_heatmap_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot ROC
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_lassoNoptuna = np.array([mean_fpr, mean_tpr, mean_auc, 'Lasso Regression normalized with Optuna'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'lasso_normalized_roc_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot Precision-Recall
    mean_recall = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(pr_rec)):
        idx = np.argsort(pr_rec[i])
        pr_interp.append(np.interp(mean_recall, pr_rec[i][idx], pr_prec[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(pr_rec)): plt.plot(pr_rec[i], pr_prec[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'lasso_normalized_prerec_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Prepare results
    res_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Lasso Regression normalized with Optuna']
    )
    res = [res_df, roc_lassoNoptuna]

    # Feature importance for best full model
    print('')
    print(f'◇ {k}-fold Cross Validation と Optuna で得られた Lasso Regression normalized の Best Model における Feature Importance')
    print('')
    feature_names = df.columns[:-1]
    feat_imp = np.abs(best_model_full.coef_)
    sorted_idx = np.argsort(feat_imp)[-10:]
    sorted_imp = feat_imp[sorted_idx]
    sorted_feat = feature_names[sorted_idx]

    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': feat_imp})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    os.makedirs(path_table, exist_ok=True)
    fi_df.to_excel(os.path.join(path_table, 'lasso_normalized_feature_importances_kfCV_optuna.xlsx'), index=False)

    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Lasso Regression normalized kfCV with Optuna')
    plt.barh(sorted_feat, sorted_imp)
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'lasso_normalized_feature_importances_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Hold-out analysis for SHAP and LIME
    scaler_ho = MinMaxScaler()
    x_train_s = scaler_ho.fit_transform(x_train)
    x_test_s = scaler_ho.transform(x_test)
    hold_model = Lasso(alpha=best_alpha, random_state=random_state)
    hold_model.fit(x_train_s, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            hold_model, df, x_train_s, x_test_s,
            size_x, size_y, path_figure, path_table,
            'lasso_normalized_shap_values_holdout_optuna',
            'lasso_normalized_shap_values_holdout_optuna',
            'Lasso Regression normalized with Optuna',
            task_type='classification', model_type='general'
        )
        print('')
    if perform_lime:
        print('')
        LimeContributionValueFunction(
            hold_model, df, x_train_s, x_test_s, t_test,
            size_x, size_y, path_figure, path_table,
            'lasso_normalized_lime_explanation_holdout_optuna',
            'lasso_normalized_lime_feature_contributions_holdout_optuna',
            'Lasso Regression normalized with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    optuna.logging.disable_default_handler()
    return res






def LassoKFoldOptunaS(k=k, show_shap=True, perform_lime=True, n_trials=50, threshold=0.5, target='logloss'):
    import os
    import pickle
    import numpy as np
    import pandas as pd
    import optuna
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    import seaborn as sns
    from sklearn.linear_model import Lasso, Ridge
    from sklearn.model_selection import StratifiedKFold, KFold
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve,
        average_precision_score, mean_squared_error, log_loss
    )

    # Header for Optuna-based optimization
    print('')
    print(f'■■■ Lasso Regression standardized kfCV with Optuna (Optimizing {target}, Trial count {n_trials}) ■■■')
    print('')

    # Objective for Optuna: minimize the selected target metric (MSE or log-loss)
    def objective(trial):
        alpha = trial.suggest_float('alpha', 0.0001, 10.0, log=True)
        kf_inner = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        scores = []
        for train_idx, test_idx in kf_inner.split(X, y):
            xx_train, xx_test = X[train_idx], X[test_idx]
            tt_train, tt_test = y[train_idx], y[test_idx]
            scaler_inner = StandardScaler()
            xx_tr_scaled = scaler_inner.fit_transform(xx_train)
            xx_te_scaled = scaler_inner.transform(xx_test)
            model = Ridge(alpha=alpha, random_state=random_state)
            model.fit(xx_tr_scaled, tt_train)
            preds = model.predict(xx_te_scaled)
            if target == 'mse':
                scores.append(mean_squared_error(tt_test, preds))
            elif target == 'logloss':
                preds_clipped = np.clip(preds, 1e-15, 1-1e-15)
                scores.append(log_loss(tt_test, preds_clipped))
            else:
                raise ValueError(f"Unknown target: {target}. Must be 'logloss' or 'mse'.")
        return np.mean(scores)

    study = optuna.create_study(direction='minimize')
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_alpha = study.best_trial.params['alpha']
    best_trial_no = study.best_trial.number

    print('')
    print('◇ Hyperparameter Optimization with Optuna for Lasso Regression standardized (detailed logs below)')
    print(f'Best trial #{best_trial_no} for minimizing {target.upper()}:')
    print(f'  Value: {study.best_trial.value:.4f}')
    print('  Params:')
    for key, val in study.best_trial.params.items():
        print(f'    {key}: {val}')
    print('')

    # Retrain on full dataset
    scaler_full = StandardScaler()
    X_scaled_full = scaler_full.fit_transform(X)
    best_model_full = Lasso(alpha=best_alpha, random_state=random_state)
    best_model_full.fit(X_scaled_full, y)

    # Save full model
    os.makedirs(path_model, exist_ok=True)
    model_file = os.path.join(path_model, 'lasso_model_full_standardized_kfCV_optuna.pkl')
    with open(model_file, 'wb') as f:
        pickle.dump(best_model_full, f)

    # Header for fold-averaged CV
    print('')
    print('■■■ Lasso Regression standardized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    kf_outer = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, pr_prec, pr_rec = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    for tr_idx, te_idx in kf_outer.split(X, y):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler_cv = StandardScaler()
        X_tr_s = scaler_cv.fit_transform(X_tr)
        X_te_s = scaler_cv.transform(X_te)

        model_cv = Lasso(alpha=best_alpha, random_state=random_state)
        model_cv.fit(X_tr_s, y_tr)
        preds_cont = model_cv.predict(X_te_s)
        preds_bin = [1 if p >= threshold else 0 for p in preds_cont]

        tn, fp, fn, tp = confusion_matrix(y_te, preds_bin).ravel()
        tot = tn + fp + fn + tp
        tn_rates.append(tn / tot)
        fp_rates.append(fp / tot)
        fn_rates.append(fn / tot)
        tp_rates.append(tp / tot)
        specs.append(tn / (tn + fp))

        aucs.append(roc_auc_score(y_te, preds_cont))
        accs.append(accuracy_score(y_te, preds_bin))
        precs.append(precision_score(y_te, preds_bin, zero_division=0))
        recs.append(recall_score(y_te, preds_bin))
        f1s.append(f1_score(y_te, preds_bin))
        aps.append(average_precision_score(y_te, preds_cont))

        fpr, tpr, _ = roc_curve(y_te, preds_cont)
        fprs.append(fpr); tprs.append(tpr)
        pr_p, pr_r, _ = precision_recall_curve(y_te, preds_cont)
        pr_prec.append(pr_p); pr_rec.append(pr_r)

    # Compute fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_cind = mean_auc
    mean_prerec = np.mean(aps)

    # fold confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn], [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-average table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Lasso Regression standardized with Optuna"
    )

    # Plot fold confusion heatmap
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'lasso_standardized_confusion_heatmap_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_lassoSoptuna = np.array([mean_fpr, mean_tpr, mean_auc, 'Lasso Regression standardized with Optuna'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'lasso_standardized_roc_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold PR curve
    mean_recall = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(pr_rec)):
        idx = np.argsort(pr_rec[i])
        pr_interp.append(np.interp(mean_recall, pr_rec[i][idx], pr_prec[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(pr_rec)): plt.plot(pr_rec[i], pr_prec[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'lasso_standardized_prerec_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Prepare results
    res_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Lasso Regression standardized with Optuna']
    )
    res = [res_df, roc_lassoSoptuna]

    # Feature importance for best full model
    print('')
    print(f'◇ {k}-fold Cross Validation と Optuna で得られた Lasso Regression standardized の Best Model における Feature Importance')
    print('')
    feature_names = df.columns[:-1]
    feat_imp = np.abs(best_model_full.coef_)
    sorted_idx = np.argsort(feat_imp)[-10:]
    sorted_imp = feat_imp[sorted_idx]
    sorted_feat = feature_names[sorted_idx]

    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': feat_imp})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    os.makedirs(path_table, exist_ok=True)
    excel_file = os.path.join(path_table, 'lasso_standardized_feature_importances_kfCV_optuna.xlsx')
    fi_df.to_excel(excel_file, index=False)

    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Lasso Regression standardized kfCV with Optuna')
    plt.barh(sorted_feat, sorted_imp)
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'lasso_standardized_feature_importances_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Hold-out analysis for SHAP and LIME
    scaler_ho = StandardScaler()
    x_train_s = scaler_ho.fit_transform(x_train)
    x_test_s = scaler_ho.transform(x_test)
    hold_model = Lasso(alpha=best_alpha, random_state=random_state)
    hold_model.fit(x_train_s, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            hold_model, df, x_train_s, x_test_s,
            size_x, size_y, path_figure, path_table,
            'lasso_standardized_shap_values_holdout_optuna',
            'lasso_standardized_shap_values_holdout_optuna',
            'Lasso Regression standardized with Optuna',
            task_type='classification', model_type='general'
        )
        print('')
    if perform_lime:
        print('')
        LimeContributionValueFunction(
            hold_model, df, x_train_s, x_test_s, t_test,
            size_x, size_y, path_figure, path_table,
            'lasso_standardized_lime_explanation_holdout_optuna',
            'lasso_standardized_lime_feature_contributions_holdout_optuna',
            'Lasso Regression standardized with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    optuna.logging.disable_default_handler()
    return res







# Ridge Regression with normalization and fold-Averaged metrics
def RidgeKFoldN(alpha=0.01, k=k, show_shap=True, perform_lime=True, threshold=0.5):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Ridge
    from sklearn.metrics import (
        mean_squared_error, roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold
    from sklearn.preprocessing import MinMaxScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    print('')
    print('■■■ Ridge Regression normalized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')


    # ----------------- Stratified K-fold Cross Validation -----------------
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    ridge = Ridge(alpha=alpha, random_state=random_state)

    # Initialize accumulators
    feature_importances = np.zeros((X.shape[1],))
    aucs, accs, precs, recs, f1s, aps = [], [], [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    specs, aps = [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = MinMaxScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        ridge.fit(x_train_scaled, y_train)
        predictions = ridge.predict(x_test_scaled)

        # Feature importance accumulation
        feature_importances += np.abs(ridge.coef_)

        # Binarize for classification metrics
        preds_round = [1 if p >= threshold else 0 for p in predictions]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, preds_round).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # Compute metrics
        aucs.append(roc_auc_score(y_test, predictions))
        accs.append(accuracy_score(y_test, preds_round))
        precs.append(precision_score(y_test, preds_round, zero_division=0))
        recs.append(recall_score(y_test, preds_round))
        f1s.append(f1_score(y_test, preds_round))
        aps.append(average_precision_score(y_test, predictions))
        specs.append(tn/(tn+fp))

        # ROC and PR curves per fold
        fpr, tpr, _ = roc_curve(y_test, predictions)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, predictions)
        precisions.append(prec_vals); recalls.append(rec_vals)


    # -----------------Calculate fold-average scores-----------------
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_ap = np.mean(aps)
    mean_spec = np.mean(specs)
    mean_cind = np.mean(aucs)
    mean_prerec = np.mean(aps)


    # File paths
    hmname = os.path.join(path_figure, 'ridge_normalized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'ridge_normalized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'ridge_normalized_prerec_curve_kfCV.png')

    # fold-averaged confusion matrix
    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                         [fold_fp_rate, fold_tp_rate]])
    fold_cm_pct = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_pct, fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec, mean_auc, mean_cind, mean_prerec,
        "Ridge Regression normalized"
    )

    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()



    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_ridgeN = np.array([mean_fpr, mean_tpr, mean_auc, 'Ridge Regression normalized'], dtype=object)

    print('')
    print('□ fold-Averaged ROC Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-ROC (AUC={mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()



    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_rec_all = np.linspace(0,1,100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_rec_all, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ fold-Averaged Precision-Recall Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_all, mean_prec_curve, label=f'fold-PR (AP={mean_ap:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()



    # -----------------Feature importance (coefficients) extraction-----------------
    feature_importances /= k
    feature_names = df.columns[:-1]
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    exname = os.path.join(path_table, 'ridge_feature_importances_normalized_kfCV.xlsx')
    feat_df.to_excel(exname, index=False)

    print('')
    print('◇ Top 10 Feature Importances (fold-Averaged):')
    print('')
    top_idx = np.argsort(feature_importances)[-10:]
    top_feats = feature_names[top_idx]
    top_imps = feature_importances[top_idx]
    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Ridge Regression normalized')
    plt.barh(top_feats, top_imps, color='skyblue')
    plt.xlabel('Importance')
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    # ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'ridge_feature_importances_normalized_kfCV.png'))
    plt.show(); plt.close()
    print('')

    # Retrain on full dataset
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = Ridge(alpha=alpha, random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'ridge_model_normalized_full.pkl'), 'wb') as f:
        pickle.dump(model_full, f)


    # -----------------Hold-out method for SHAP and LIME analyses-----------------
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = Ridge(alpha=alpha, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'ridge_normalized_shap_values_holdout', 'ridge_normalized_shap_values_holdout',
            'Ridge Regression normalized', task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'ridge_normalized_lime_explanation_holdout', 'ridge_normalized_lime_feature_contributions_holdholdout',
            'Ridge Regression normalized', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=[f'Ridge Regression normalized (α={alpha:.4f})']
    )
    return [res2, roc_ridgeN]








# Ridge Regression with normalization and fold-Averaged metrics
def RidgeKFoldS(alpha=0.01, k=k, show_shap=True, perform_lime=True, threshold=0.5):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import Ridge
    from sklearn.metrics import (
        mean_squared_error, roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold
    from sklearn.preprocessing import StandardScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    print('')
    print('■■■ Ridge Regression standardized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')


    # ----------------- Stratified K-fold Cross Validation -----------------
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    ridge = Ridge(alpha=alpha, random_state=random_state)

    # Initialize accumulators
    feature_importances = np.zeros((X.shape[1],))
    aucs, accs, precs, recs, f1s, aps = [], [], [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    specs, aps = [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = StandardScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        ridge.fit(x_train_scaled, y_train)
        predictions = ridge.predict(x_test_scaled)

        # Feature importance accumulation
        feature_importances += np.abs(ridge.coef_)

        # Binarize for classification metrics
        preds_round = [1 if p >= threshold else 0 for p in predictions]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, preds_round).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # Compute metrics
        aucs.append(roc_auc_score(y_test, predictions))
        accs.append(accuracy_score(y_test, preds_round))
        precs.append(precision_score(y_test, preds_round, zero_division=0))
        recs.append(recall_score(y_test, preds_round))
        f1s.append(f1_score(y_test, preds_round))
        aps.append(average_precision_score(y_test, predictions))
        specs.append(tn/(tn+fp))

        # ROC and PR curves per fold
        fpr, tpr, _ = roc_curve(y_test, predictions)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, predictions)
        precisions.append(prec_vals); recalls.append(rec_vals)


    # -----------------Calculate fold-average scores-----------------
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_ap = np.mean(aps)
    mean_spec = np.mean(specs)
    mean_cind = np.mean(aucs)
    mean_prerec = np.mean(aps)


    # File paths
    hmname = os.path.join(path_figure, 'ridge_standardized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'ridge_standardized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'ridge_standardized_prerec_curve_kfCV.png')

    # fold-averaged confusion matrix
    fold_tn_rate = np.mean(tn_rates)
    fold_fp_rate = np.mean(fp_rates)
    fold_fn_rate = np.mean(fn_rates)
    fold_tp_rate = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn_rate, fold_fn_rate],
                         [fold_fp_rate, fold_tp_rate]])
    fold_cm_pct = fold_cm * 100

    display_fold_averages(
        k, df, fold_cm_pct, fold_tp_rate, fold_fp_rate, fold_fn_rate, fold_tn_rate,
        mean_acc, mean_prec, mean_rec, mean_f1, mean_spec, mean_auc, mean_cind, mean_prerec,
        "Ridge Regression standardized"
    )

    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()



    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_ridgeS = np.array([mean_fpr, mean_tpr, mean_auc, 'Ridge Regression standardized'], dtype=object)

    print('')
    print('□ fold-Averaged ROC Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-ROC (AUC={mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()



    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_rec_all = np.linspace(0,1,100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_rec_all, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ fold-Averaged Precision-Recall Curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_all, mean_prec_curve, label=f'fold-PR (AP={mean_ap:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()



    # -----------------Feature importance (coefficients) extraction-----------------
    feature_importances /= k
    feature_names = df.columns[:-1]
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    exname = os.path.join(path_table, 'ridge_feature_importances_standardized_kfCV.xlsx')
    feat_df.to_excel(exname, index=False)

    print('')
    print('◇ Top 10 Feature Importances (fold-Averaged):')
    print('')
    top_idx = np.argsort(feature_importances)[-10:]
    top_feats = feature_names[top_idx]
    top_imps = feature_importances[top_idx]
    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Ridge Regression standardized')
    plt.barh(top_feats, top_imps, color='skyblue')
    plt.xlabel('Importance')
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    # ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'ridge_feature_importances_standardized_kfCV.png'))
    plt.show(); plt.close()
    print('')

    # Retrain on full dataset
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = Ridge(alpha=alpha, random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'ridge_model_standardized_full.pkl'), 'wb') as f:
        pickle.dump(model_full, f)


    # -----------------Hold-out method for SHAP and LIME analyses-----------------
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = Ridge(alpha=alpha, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'ridge_shap_values_holdout_standardized', 'ridge_shap_values_holdout_standardized',
            'Ridge Regression standardized', task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'ridge_lime_explanation_holdout_standardized', 'ridge_lime_feature_contributions_holdholdout_standardized',
            'Ridge Regression standardized', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=[f'Ridge Regression standardized (α={alpha:.4f})']
    )
    return [res2, roc_ridgeS]






def RidgeKFoldOptunaN(k=k, show_shap=True, perform_lime=True, n_trials=50, threshold=0.5, target='logloss'):
    import os
    import pickle
    import numpy as np
    import pandas as pd
    import optuna
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    import seaborn as sns
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import StratifiedKFold, KFold
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve,
        average_precision_score, mean_squared_error, log_loss
    )

    # Header for Optuna-based optimization
    print('')
    print(f'■■■ Ridge Regression normalized kfCV with Optuna (Optimizing {target}, Trial count {n_trials}) ■■■')
    print('')

    # Objective for Optuna: minimize the selected target metric (MSE or log-loss)
    def objective(trial):
        # Suggest alpha
        alpha = trial.suggest_float('alpha', 0.0001, 10.0, log=True)
        # Stratified K-Fold for balanced splits
        kf_inner = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        scores = []
        for train_idx, test_idx in kf_inner.split(X, y):
            xx_train, xx_test = X[train_idx], X[test_idx]
            tt_train, tt_test = y[train_idx], y[test_idx]
            scaler_inner = MinMaxScaler()
            xx_tr_scaled = scaler_inner.fit_transform(xx_train)
            xx_te_scaled = scaler_inner.transform(xx_test)
            model = Ridge(alpha=alpha, random_state=random_state)
            model.fit(xx_tr_scaled, tt_train)
            preds = model.predict(xx_te_scaled)
            if target == 'mse':
                scores.append(mean_squared_error(tt_test, preds))
            elif target == 'logloss':
                preds_clipped = np.clip(preds, 1e-15, 1-1e-15)
                scores.append(log_loss(tt_test, preds_clipped))
            else:
                raise ValueError(f"Unknown target: {target}. Must be either 'logloss' or 'mse'.")
        return np.mean(scores)

    # Create and run study
    study = optuna.create_study(direction='minimize')
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # Best hyperparameters
    best_alpha = study.best_trial.params['alpha']
    best_trial_no = study.best_trial.number

    # Log best trial
    print('')
    print('')
    print('◇ Hyperparameter Optimization with Optuna for Ridge Regression normalized (detailed logs below)')
    print(f'Best trial #{best_trial_no} for minimizing {target.upper()}:')
    print(f'  Value: {study.best_trial.value:.4f}')
    print('  Params:')
    for key, val in study.best_trial.params.items():
        print(f'    {key}: {val}')
    print('')

    # Retrain on full dataset
    scaler_full = MinMaxScaler()
    X_scaled_full = scaler_full.fit_transform(X)
    best_model_full = Ridge(alpha=best_alpha, random_state=random_state)
    best_model_full.fit(X_scaled_full, y)

    # Save full model
    os.makedirs(path_model, exist_ok=True)
    model_file = os.path.join(path_model, 'ridge_normalize_model_fulld_kfCV_optuna.pkl')
    with open(model_file, 'wb') as f:
        pickle.dump(best_model_full, f)

    # Header for fold-averaged CV
    print('')
    print('■■■ Ridge Regression normalized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # Stratified K-Fold for fold metrics
    kf_outer = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, pr_prec, pr_rec = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    # Cross-validation loop
    for tr_idx, te_idx in kf_outer.split(X, y):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler_cv = MinMaxScaler()
        X_tr_s = scaler_cv.fit_transform(X_tr)
        X_te_s = scaler_cv.transform(X_te)

        model_cv = Ridge(alpha=best_alpha, random_state=random_state)
        model_cv.fit(X_tr_s, y_tr)
        preds_cont = model_cv.predict(X_te_s)
        preds_bin = [1 if p >= threshold else 0 for p in preds_cont]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_te, preds_bin).ravel()
        tot = tn + fp + fn + tp
        tn_rates.append(tn / tot)
        fp_rates.append(fp / tot)
        fn_rates.append(fn / tot)
        tp_rates.append(tp / tot)
        specs.append(tn / (tn + fp))  # specificity

        # Metrics
        aucs.append(roc_auc_score(y_te, preds_cont))
        accs.append(accuracy_score(y_te, preds_bin))
        precs.append(precision_score(y_te, preds_bin, zero_division=0))
        recs.append(recall_score(y_te, preds_bin))
        f1s.append(f1_score(y_te, preds_bin))
        aps.append(average_precision_score(y_te, preds_cont))

        # Curves
        fpr, tpr, _ = roc_curve(y_te, preds_cont)
        fprs.append(fpr); tprs.append(tpr)
        pr_p, pr_r, _ = precision_recall_curve(y_te, preds_cont)
        pr_prec.append(pr_p); pr_rec.append(pr_r)

    # fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_cind = mean_auc
    mean_prerec = np.mean(aps)

    # fold confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn], [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-average table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Ridge Regression normalized with Optuna"
    )

    # Plot fold confusion heatmap
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'ridge_normalized_confusion_heatmap_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_ridgeNoptuna = np.array([mean_fpr, mean_tpr, mean_auc, 'Ridge Regression normalized with Optuna'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'ridge_normalized_roc_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold PR curve
    mean_recall = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(pr_rec)):
        idx = np.argsort(pr_rec[i])
        pr_interp.append(np.interp(mean_recall, pr_rec[i][idx], pr_prec[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(pr_rec)): plt.plot(pr_rec[i], pr_prec[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'ridge_normalized_prerec_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Prepare results
    res_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Ridge Regression normalized with Optuna']
    )
    res = [res_df, roc_ridgeNoptuna]

    # Feature importance for best full model
    print('')
    print(f'◇ {k}-fold Cross Validation と Optuna で得られた Ridge Regression normalized の Best Model における Feature Importance')
    print('')
    feature_names = df.columns[:-1]
    feat_imp = np.abs(best_model_full.coef_)
    sorted_idx = np.argsort(feat_imp)[-10:]
    sorted_imp = feat_imp[sorted_idx]
    sorted_feat = feature_names[sorted_idx]

    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': feat_imp})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    os.makedirs(path_table, exist_ok=True)
    excel_file = os.path.join(path_table, 'ridge_normalized_feature_importances_kfCV_optuna.xlsx')
    fi_df.to_excel(excel_file, index=False)

    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Ridge Regression normalized kfCV with Optuna')
    plt.barh(sorted_feat, sorted_imp)
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'ridge_normalized_feature_importances_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Hold-out analysis for SHAP and LIME
    scaler_ho = MinMaxScaler()
    x_train_s = scaler_ho.fit_transform(x_train)
    x_test_s = scaler_ho.transform(x_test)
    hold_model = Ridge(alpha=best_alpha, random_state=random_state)
    hold_model.fit(x_train_s, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            hold_model, df, x_train_s, x_test_s,
            size_x, size_y, path_figure, path_table,
            'ridge_normalized_shap_values_holdout',
            'ridge_normalized_shap_values_holdout',
            'Ridge Regression normalized with Optuna',
            task_type='classification', model_type='general'
        )
        print('')
    if perform_lime:
        print('')
        LimeContributionValueFunction(
            hold_model, df, x_train_s, x_test_s, t_test,
            size_x, size_y, path_figure, path_table,
            'ridge_normalized_lime_explanation_holdout_optuna',
            'ridge_normalized_lime_feature_contributions_holdout_optuna',
            'Ridge Regression normalized with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    optuna.logging.disable_default_handler()
    return res






def RidgeKFoldOptunaS(k=k, show_shap=True, perform_lime=True, n_trials=50, threshold=0.5, target='logloss'):
    import os
    import pickle
    import numpy as np
    import pandas as pd
    import optuna
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    import seaborn as sns
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import StratifiedKFold, KFold
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve,
        average_precision_score, mean_squared_error, log_loss
    )

    # Header for Optuna-based optimization
    print('')
    print(f'■■■ Ridge Regression standardized kfCV with Optuna (Optimized {target}, Trial count {n_trials}) ■■■')
    print('')

    # Objective for Optuna: minimize the selected target metric (MSE or log-loss)
    def objective(trial):
        alpha = trial.suggest_float('alpha', 0.0001, 10.0, log=True)
        kf_inner = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        scores = []
        for train_idx, test_idx in kf_inner.split(X, y):
            xx_train, xx_test = X[train_idx], X[test_idx]
            tt_train, tt_test = y[train_idx], y[test_idx]
            scaler_inner = StandardScaler()
            xx_tr_scaled = scaler_inner.fit_transform(xx_train)
            xx_te_scaled = scaler_inner.transform(xx_test)
            model = Ridge(alpha=alpha, random_state=random_state)
            model.fit(xx_tr_scaled, tt_train)
            preds = model.predict(xx_te_scaled)
            if target == 'mse':
                scores.append(mean_squared_error(tt_test, preds))
            elif target == 'logloss':
                preds_clipped = np.clip(preds, 1e-15, 1-1e-15)
                scores.append(log_loss(tt_test, preds_clipped))
            else:
                raise ValueError(f"Unknown target: {target}. Must be 'logloss' or 'mse'.")
        return np.mean(scores)

    study = optuna.create_study(direction='minimize')
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_alpha = study.best_trial.params['alpha']
    best_trial_no = study.best_trial.number

    print('')
    print('◇ Hyperparameter Optimization with Optuna for Ridge Regression standardized (detailed logs below)')
    print(f'Best trial #{best_trial_no} for minimizing {target.upper()}:')
    print(f'  Value: {study.best_trial.value:.4f}')
    print('  Params:')
    for key, val in study.best_trial.params.items():
        print(f'    {key}: {val}')
    print('')

    # Retrain on full dataset
    scaler_full = StandardScaler()
    X_scaled_full = scaler_full.fit_transform(X)
    best_model_full = Ridge(alpha=best_alpha, random_state=random_state)
    best_model_full.fit(X_scaled_full, y)

    # Save full model
    os.makedirs(path_model, exist_ok=True)
    model_file = os.path.join(path_model, 'ridge_standardized_model_full_kfCV_optuna.pkl')
    with open(model_file, 'wb') as f:
        pickle.dump(best_model_full, f)

    # Header for fold-averaged CV
    print('')
    print('■■■ Ridge Regression standardized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    kf_outer = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, pr_prec, pr_rec = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    for tr_idx, te_idx in kf_outer.split(X, y):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler_cv = StandardScaler()
        X_tr_s = scaler_cv.fit_transform(X_tr)
        X_te_s = scaler_cv.transform(X_te)

        model_cv = Ridge(alpha=best_alpha, random_state=random_state)
        model_cv.fit(X_tr_s, y_tr)
        preds_cont = model_cv.predict(X_te_s)
        preds_bin = [1 if p >= threshold else 0 for p in preds_cont]

        tn, fp, fn, tp = confusion_matrix(y_te, preds_bin).ravel()
        tot = tn + fp + fn + tp
        tn_rates.append(tn / tot)
        fp_rates.append(fp / tot)
        fn_rates.append(fn / tot)
        tp_rates.append(tp / tot)
        specs.append(tn / (tn + fp))

        aucs.append(roc_auc_score(y_te, preds_cont))
        accs.append(accuracy_score(y_te, preds_bin))
        precs.append(precision_score(y_te, preds_bin, zero_division=0))
        recs.append(recall_score(y_te, preds_bin))
        f1s.append(f1_score(y_te, preds_bin))
        aps.append(average_precision_score(y_te, preds_cont))

        fpr, tpr, _ = roc_curve(y_te, preds_cont)
        fprs.append(fpr); tprs.append(tpr)
        pr_p, pr_r, _ = precision_recall_curve(y_te, preds_cont)
        pr_prec.append(pr_p); pr_rec.append(pr_r)

    # Compute fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_cind = mean_auc
    mean_prerec = np.mean(aps)

    # fold confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn], [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-average table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Ridge Regression standardized with Optuna"
    )

    # Plot fold confusion heatmap
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'ridge_standardized_confusion_heatmap_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(tprs_interp, axis=0)
    roc_ridgeSoptuna = np.array([mean_fpr, mean_tpr, mean_auc, 'Ridge Regression standardized_with Optuna'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'ridge_standardized_roc_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Plot fold PR curve
    mean_recall = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(pr_rec)):
        idx = np.argsort(pr_rec[i])
        pr_interp.append(np.interp(mean_recall, pr_rec[i][idx], pr_prec[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(pr_rec)): plt.plot(pr_rec[i], pr_prec[i], alpha=0.3)
    plt.plot([0,1], [0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'ridge_standardized_prerec_curve_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Prepare results
    res_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Ridge Regression standardized with Optuna']
    )
    res = [res_df, roc_ridgeSoptuna]

    # Feature importance for best full model
    print('')
    print(f'◇ {k}-fold Cross Validation と Optuna で得られた Ridge Regression standardized の Best Model における Feature Importance')
    print('')
    feature_names = df.columns[:-1]
    feat_imp = np.abs(best_model_full.coef_)
    sorted_idx = np.argsort(feat_imp)[-10:]
    sorted_imp = feat_imp[sorted_idx]
    sorted_feat = feature_names[sorted_idx]

    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': feat_imp})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    os.makedirs(path_table, exist_ok=True)
    excel_file = os.path.join(path_table, 'ridge_standardized_feature_importances_kfCV_optuna.xlsx')
    fi_df.to_excel(excel_file, index=False)

    plt.figure(figsize=(size_x, size_y))
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Ridge Regression standardized kfCV with Optuna')
    plt.barh(sorted_feat, sorted_imp)
    ax = plt.gca()
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'ridge_standardized_feature_importances_kfCV_optuna.png'))
    plt.show(); plt.close()

    # Hold-out analysis for SHAP and LIME
    scaler_ho = StandardScaler()
    x_train_s = scaler_ho.fit_transform(x_train)
    x_test_s = scaler_ho.transform(x_test)
    hold_model = Ridge(alpha=best_alpha, random_state=random_state)
    hold_model.fit(x_train_s, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            hold_model, df, x_train_s, x_test_s,
            size_x, size_y, path_figure, path_table,
            'ridge_standardized_shap_values_holdout',
            'ridge_standardized_shap_values_holdout',
            'Ridge Regression standardized with Optuna',
            task_type='classification', model_type='general'
        )
        print('')
    if perform_lime:
        print('')
        LimeContributionValueFunction(
            hold_model, df, x_train_s, x_test_s, t_test,
            size_x, size_y, path_figure, path_table,
            'ridge_standardized_lime_explanation_holdout_optuna',
            'ridge_standardized_lime_feature_contributions_holdout_optuna',
            'Ridge Regression standardized with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    optuna.logging.disable_default_handler()
    return res






# Logistic Regression normalized kfCV with fold-Averaged metrics
def LogisticKFoldN(k=k, show_shap=True, perform_lime=True):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import MinMaxScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    # Hold-out 用のデータの分割
    x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    print('')
    print('■■■ Logistic Regression normalized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold instead of KFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for fold metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For true fold-averaged confusion matrix
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    # For ROC and Precision-Recall curves per fold
    fprs, tprs, precisions, recalls = [], [], [], []
    # Initialize accumulator for feature importances
    feature_importances = np.zeros((df.shape[1] - 1,))
    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_pred_probs = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = MinMaxScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        # Initialize and fit the Logistic Regression model
        model = LogisticRegression(random_state=random_state)
        model.fit(x_train_scaled, y_train)

        # Accumulate feature importances (absolute coefficients)
        feature_importances += np.abs(model.coef_[0])

        # Predict classes and probabilities
        y_pred = model.predict(x_test_scaled)
        y_pred_prob = model.predict_proba(x_test_scaled)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        # specificity
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        aucs.append(roc_auc_score(y_test, y_pred_prob))
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred, zero_division=0))
        recs.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        aps.append(average_precision_score(y_test, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(prec_vals); recalls.append(rec_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test)
        all_pred_probs.extend(y_pred_prob)

    # Compute fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_prerec = np.mean(aps)
    mean_cind = mean_auc

    # Compute true fold-averaged confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Logistic Regression normalized"
    )

    # Print summary
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')

    # Paths for figures
    hmname = os.path.join(path_figure, 'logistic_normalized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'logistic_normalized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'logistic_normalized_prerec_curve_kfCV.png')

    # Plot fold-averaged confusion matrix heatmap
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()

    # Plot fold-averaged ROC curve
    mean_fpr = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_logisticN = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'Logistic Regression normalized'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # Plot fold-averaged Precision-Recall curve
    mean_rec_curve = np.linspace(0,1,100)
    pr_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # Average the feature importances over all folds
    feature_importances /= k
    feature_names = df.columns[:-1]
    sorted_idx = np.argsort(feature_importances)[-10:]
    sorted_importance = feature_importances[sorted_idx]
    sorted_features = feature_names[sorted_idx]

    # Export feature importances
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    excel_filename = os.path.join(path_table, 'logistic_normalized_feature_importances_kfCV.xlsx')
    feat_df.to_excel(excel_filename, index=False)

    print('')
    print(f'◇ {k}-fold Cross Validation で得られた Logistic Regression normalized における Feature Importance')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_features, sorted_importance, color='skyblue')
    plt.xlabel('Average Feature Importance')
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Logistic Regression normalized')
    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    font_prop = fm.FontProperties(fname=font_path)
    ax = plt.gca()
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    plt.savefig(os.path.join(path_figure, 'logistic_normalized_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # Optionally retrain the model on the full dataset
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = LogisticRegression(random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'logistic_normalized_model_full.pkl'), 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out method for SHAP and LIME analyses
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = LogisticRegression(random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap == True:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'logistic_normalized_shap_values_holdout',
            'logistic_normalized_shap_values_holdout',
            'Logistic Regression normalized',
            task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            t_test, size_x, size_y, path_figure, path_table,
            'logistic_normalized_lime_explanation_holdout',
            'logistic_normalized_lime_feature_contributions_holdout',
            'Logistic Regression normalized',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['Logistic Regression normalized']
    )
    return [res2, roc_logisticN]







# Logistic Regression standardized kfCV with fold-Averaged metrics
def LogisticKFoldS(k=k, show_shap=True, perform_lime=True):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import StandardScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    # Hold-out 用のデータの分割
    x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    print('')
    print('■■■ Logistic Regression standardized kfCV with fold-Averaged metrics　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold instead of KFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for fold metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For true fold-averaged confusion matrix
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    # For ROC and Precision-Recall curves per fold
    fprs, tprs, precisions, recalls = [], [], [], []
    # Initialize accumulator for feature importances
    feature_importances = np.zeros((df.shape[1] - 1,))
    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_pred_probs = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler = StandardScaler()
        x_train_scaled = scaler.fit_transform(X_train)
        x_test_scaled = scaler.transform(X_test)

        # Initialize and fit the Logistic Regression model
        model = LogisticRegression(random_state=random_state)
        model.fit(x_train_scaled, y_train)

        # Accumulate feature importances (absolute coefficients)
        feature_importances += np.abs(model.coef_[0])

        # Predict classes and probabilities
        y_pred = model.predict(x_test_scaled)
        y_pred_prob = model.predict_proba(x_test_scaled)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        # specificity
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        aucs.append(roc_auc_score(y_test, y_pred_prob))
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred, zero_division=0))
        recs.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        aps.append(average_precision_score(y_test, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(prec_vals); recalls.append(rec_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test)
        all_pred_probs.extend(y_pred_prob)

    # Compute fold-averaged scores
    mean_auc = np.mean(aucs)
    mean_acc = np.mean(accs)
    mean_prec = np.mean(precs)
    mean_rec = np.mean(recs)
    mean_f1 = np.mean(f1s)
    mean_spec = np.mean(specs)
    mean_prerec = np.mean(aps)
    mean_cind = mean_auc

    # Compute true fold-averaged confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Logistic Regression standardized"
    )

    # Print summary
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')

    # Paths for figures
    hmname = os.path.join(path_figure, 'logistic_standardized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'logistic_standardized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'logistic_standardized_prerec_curve_kfCV.png')

    # Plot fold-averaged confusion matrix heatmap
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()

    # Plot fold-averaged ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_logisticS = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'Logistic Regression standardized'], dtype=object)

    print('')
    print('□　ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # Plot fold-averaged Precision-Recall curve
    mean_rec_curve = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # Average the feature importances over all folds
    feature_importances /= k
    feature_names = df.columns[:-1]
    sorted_idx = np.argsort(feature_importances)[-10:]
    sorted_importance = feature_importances[sorted_idx]
    sorted_features = feature_names[sorted_idx]

    # Export feature importances
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    feat_df['Abs_Importance'] = feat_df['Importance'].abs()
    feat_df = feat_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    excel_filename = os.path.join(path_table, 'logistic_standardized_feature_importances_kfCV.xlsx')
    feat_df.to_excel(excel_filename, index=False)

    print('')
    print(f'◇ {k}-fold Cross Validation で得られた Logistic Regression standardized における Feature Importance')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_features, sorted_importance, color='skyblue')
    plt.xlabel('Average Feature Importance')
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Logistic Regression standardized')
    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    font_prop = fm.FontProperties(fname=font_path)
    ax = plt.gca()
    for label in ax.get_yticklabels(): label.set_fontproperties(font_prop)
    plt.savefig(os.path.join(path_figure, 'logistic_standardized_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # Optionally retrain the model on the full dataset
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    model_full = LogisticRegression(random_state=random_state)
    model_full.fit(X_scaled, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'logistic_standardized_model_full.pkl'), 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out method for SHAP and LIME analyses
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = LogisticRegression(random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'logistic_standardized_shap_values_holdout',
            'logistic_standardized_shap_values_holdout',
            'Logistic Regression standardized',
            task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            t_test, size_x, size_y, path_figure, path_table,
            'logistic_standardized_lime_explanation_holdout',
            'logistic_standardized_lime_feature_contributions_holdout',
            'Logistic Regression standardized',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Logistic Regression standardized']
    )
    return [res2, roc_logisticS]








# SVM Normalized kfCV with fold-Averaged metrics
def SVMKFoldN(k=k, show_shap=False, perform_lime=True):
    import numpy as np
    import pandas as pd
    from sklearn.svm import SVC
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import MinMaxScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    # Hold-out 用のデータの分割
    x_train, x_test, t_train, t_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    print('')
    print('■■■ Support Vector Machine normalized kfCV with fold-Averaged metrics ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # データの線形性の確認
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X, y)
    y_pred_lr = lr.predict(X)
    r2 = r2_score(y, y_pred_lr)
    if r2 >= 0.8:
        linear = 1
    elif r2 < 0.2:
        linear = 0
    else:
        linear = 0

    kernel_option = 'linear' if linear == 1 else 'rbf'

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # KFold cross-validation
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Initialize accumulator for feature importances
    feature_importances = np.zeros((df.shape[1] - 1,))

    # Lists for fold metrics
    auc_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    specs = []
    aps = []

    # For true fold-averaged confusion matrix
    tn_rates = []
    fp_rates = []
    fn_rates = []
    tp_rates = []

    # For ROC and Precision-Recall curves per fold
    fprs = []
    tprs = []
    precisions = []
    recalls = []

    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_pred_probs = []

    for train_index, test_index in kf.split(X_scaled, y):
        X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Initialize the SVM model with probability estimation enabled
        model = SVC(kernel=kernel_option, probability=True, random_state=random_state)
        model.fit(X_train_fold, y_train_fold)

        # Assuming a linear SVM, extract the coefficients
        # For non-linear SVM, this part is not applicable
        if kernel_option == 'linear':
            feature_importances += np.abs(model.coef_[0])

        # Predict classes and probabilities for evaluation
        y_pred = model.predict(X_test_fold)
        y_pred_prob = model.predict_proba(X_test_fold)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test_fold, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        # specificity
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        aps.append(average_precision_score(y_test_fold, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test_fold, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test_fold, y_pred_prob)
        precisions.append(prec_vals)
        recalls.append(rec_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test_fold)
        all_pred_probs.extend(y_pred_prob)

    # Compute fold-averaged scores
    mean_auc = np.mean(auc_scores)
    mean_acc = np.mean(accuracy_scores)
    mean_prec = np.mean(precision_scores)
    mean_rec = np.mean(recall_scores)
    mean_f1 = np.mean(f1_scores)
    mean_spec = np.mean(specs)
    mean_prerec = np.mean(aps)
    mean_cind = mean_auc

    # Compute true fold-averaged confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Support Vector Machine normalized"
    )

    # Print summary
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')

    # Paths for figures
    hmname = os.path.join(path_figure, 'svm_normalized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'svm_normalized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'svm_normalized_prerec_curve_kfCV.png')

    # Plot fold-averaged confusion matrix heatmap
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()

    # Plot fold-averaged ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_SVMN = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'SVM normalized'], dtype=object)

    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # Plot fold-averaged Precision-Recall curve
    mean_rec_curve = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # Initialize the SVM model with probability estimation enabled for full data
    model_full = SVC(kernel=kernel_option, probability=True, random_state=random_state)
    model_full.fit(X_scaled, y)

    # Ensure the path exists
    os.makedirs(path_model, exist_ok=True)
    model_filename = os.path.join(path_model, 'svm_normalized_full_kfCV.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Feature importance for SVM can be calculated only when the kernel is linear.
    if kernel_option == 'linear':
        # Average the feature importances over all folds
        feature_importances /= k
        # Get the feature names
        feature_names = df.columns[:-1]
        # Sort the features by importance
        sorted_idx = np.argsort(feature_importances)[-10:]
        sorted_importance = feature_importances[sorted_idx]
        sorted_features = feature_names[sorted_idx]

        # Export feature importances
        feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
        feature_importances_df['Abs_Importance'] = feature_importances_df['Importance'].abs()
        feature_importances_df = feature_importances_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
        excel_filename = os.path.join(path_table, 'svm_normalized_feature_importances_kfold_fold.xlsx')
        feature_importances_df.to_excel(excel_filename, index=False)

        print('')
        print(f'◇ {k}-fold Cross Validation で得られた SVM normalized kfCV における Feature Importance (fold-Averaged)')
        print('')
        plt.figure(figsize=(size_x, size_y))
        plt.barh(sorted_features, sorted_importance, color='skyblue')
        plt.xlabel('Average Feature Importance')
        plt.title(f'Top 10 Feature Importances - fold-Averaged over {k} Folds (Linear SVM)')
        font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
        font_prop = fm.FontProperties(fname=font_path)
        ax = plt.gca()
        for label in ax.get_yticklabels():
            label.set_fontproperties(font_prop)
        plt.savefig(os.path.join(path_figure, 'svm_normalized_feature_importances_kfCV_fold.png'))
        plt.show(); plt.close()
        print('')

    elif kernel_option == 'rbf':
        print('')
        print('・ Feature importance cannot be figured out from SVM with NON-LINEAR kernel due to the kernel trick.')
        print('')

    # Hold-out method for SHAP and LIME analyses
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = SVC(kernel=kernel_option, probability=True, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    print('')
    print('◇ SVM normalized の 各変数の SHAP値')
    print('')

    if show_shap:
        # For non-linear kernels, use KernelExplainer to calculate SHAP values
        explainer = shap.KernelExplainer(model_holdout.predict_proba, x_train_scaled)
        shap_values = explainer.shap_values(x_test_scaled)
        shap.summary_plot(shap_values, x_test_scaled, feature_names=df.columns[:-1], plot_type="bar")
        plt.title('SHAP summary plot for SVM with RBF kernel')
        plt.show()
    else:
        print('・　SVMのSHAP値の計算には長時間を要するため skip する。')
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'svm_normalized_lime_explanation_holdout_kfCV_fold',
            'svm_normalized_lime_feature_contributions_holdout_kfCV_fold',
            'SVM normalized',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([
        np.mean(auc_scores), np.mean(accuracy_scores),
        np.mean(precision_scores), np.mean(recall_scores),
        np.mean(f1_scores)
    ], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['SVM normalized']
    )

    return [res2, roc_SVMN]





# SVM Standardized kfCV with fold-Averaged metrics
def SVMKFoldS(k=k, show_shap=False, perform_lime=True):
    import numpy as np
    import pandas as pd
    from sklearn.svm import SVC
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import StandardScaler
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    # Hold-out 用のデータの分割
    x_train, x_test, t_train, t_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    print('')
    print('■■■ Support Vector Machine standardized kfCV with fold-Averaged metrics ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # データの線形性の確認
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X, y)
    y_pred_lr = lr.predict(X)
    r2 = r2_score(y, y_pred_lr)
    if r2 >= 0.8:
        linear = 1
    elif r2 < 0.2:
        linear = 0
    else:
        linear = 0

    kernel_option = 'linear' if linear == 1 else 'rbf'

    # Initialize StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # KFold cross-validation
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Initialize accumulator for feature importances
    feature_importances = np.zeros((df.shape[1] - 1,))

    # Lists for fold metrics
    auc_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    specs = []
    aps = []

    # For true fold-averaged confusion matrix
    tn_rates = []
    fp_rates = []
    fn_rates = []
    tp_rates = []

    # For ROC and Precision-Recall curves per fold
    fprs = []
    tprs = []
    precisions = []
    recalls = []

    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_pred_probs = []

    for train_index, test_index in kf.split(X_scaled, y):
        X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Initialize the SVM model with probability estimation enabled
        model = SVC(kernel=kernel_option, probability=True, random_state=random_state)
        model.fit(X_train_fold, y_train_fold)

        # Assuming a linear SVM, extract the coefficients
        # For non-linear SVM, this part is not applicable
        if kernel_option == 'linear':
            feature_importances += np.abs(model.coef_[0])

        # Predict classes and probabilities for evaluation
        y_pred = model.predict(X_test_fold)
        y_pred_prob = model.predict_proba(X_test_fold)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test_fold, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        # specificity
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        aps.append(average_precision_score(y_test_fold, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test_fold, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test_fold, y_pred_prob)
        precisions.append(prec_vals)
        recalls.append(rec_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test_fold)
        all_pred_probs.extend(y_pred_prob)

    # Compute fold-averaged scores
    mean_auc = np.mean(auc_scores)
    mean_acc = np.mean(accuracy_scores)
    mean_prec = np.mean(precision_scores)
    mean_rec = np.mean(recall_scores)
    mean_f1 = np.mean(f1_scores)
    mean_spec = np.mean(specs)
    mean_prerec = np.mean(aps)
    mean_cind = mean_auc

    # Compute true fold-averaged confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Support Vector Machine standardized"
    )

    # Print summary
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')

    # Paths for figures
    hmname = os.path.join(path_figure, 'svm_standardized_confusion_heatmap_kfCV.png')
    rocname = os.path.join(path_figure, 'svm_standardized_roc_curve_kfCV.png')
    prerecname = os.path.join(path_figure, 'svm_standardized_prerec_curve_kfCV.png')

    # Plot fold-averaged confusion matrix heatmap
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()

    # Plot fold-averaged ROC curve
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_SVMS = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'SVM standardized'], dtype=object)

    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # Plot fold-averaged Precision-Recall curve
    mean_rec_curve = np.linspace(0, 1, 100)
    pr_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # Initialize the SVM model with probability estimation enabled for full data
    model_full = SVC(kernel=kernel_option, probability=True, random_state=random_state)
    model_full.fit(X_scaled, y)

    # Ensure the path exists
    os.makedirs(path_model, exist_ok=True)
    model_filename = os.path.join(path_model, 'svm_standardized_full_kfCV.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Feature importance for SVM can be calculated only when the kernel is linear.
    if kernel_option == 'linear':
        # Average the feature importances over all folds
        feature_importances /= k
        # Get the feature names
        feature_names = df.columns[:-1]
        # Sort the features by importance
        sorted_idx = np.argsort(feature_importances)[-10:]
        sorted_importance = feature_importances[sorted_idx]
        sorted_features = feature_names[sorted_idx]

        # Export feature importances
        feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
        feature_importances_df['Abs_Importance'] = feature_importances_df['Importance'].abs()
        feature_importances_df = feature_importances_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
        excel_filename = os.path.join(path_table, 'svm_standardized_feature_importances_kfCV.xlsx')
        feature_importances_df.to_excel(excel_filename, index=False)

        print('')
        print(f'◇ {k}-fold Cross Validation で得られた SVM standardized kfCV における Feature Importance (fold-Averaged)')
        print('')
        plt.figure(figsize=(size_x, size_y))
        plt.barh(sorted_features, sorted_importance, color='skyblue')
        plt.xlabel('Average Feature Importance')
        plt.title(f'Top 10 Feature Importances - fold-Averaged over {k} Folds (Linear SVM)')
        font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
        font_prop = fm.FontProperties(fname=font_path)
        ax = plt.gca()
        for label in ax.get_yticklabels():
            label.set_fontproperties(font_prop)
        plt.savefig(os.path.join(path_figure, 'svm_standardized_feature_importances_kfCV.png'))
        plt.show(); plt.close()
        print('')

    elif kernel_option == 'rbf':
        print('')
        print('・ Feature importance cannot be figured out from SVM with NON-LINEAR kernel due to the kernel trick.')
        print('')

    # Hold-out method for SHAP and LIME analyses
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = SVC(kernel=kernel_option, probability=True, random_state=random_state)
    model_holdout.fit(x_train_scaled, t_train)

    print('')
    print('◇ SVM standardized の 各変数の SHAP値')
    print('')

    if show_shap:
        # For non-linear kernels, use KernelExplainer to calculate SHAP values
        explainer = shap.KernelExplainer(model_holdout.predict_proba, x_train_scaled)
        shap_values = explainer.shap_values(x_test_scaled)
        shap.summary_plot(shap_values, x_test_scaled, feature_names=df.columns[:-1], plot_type="bar")
        plt.title('SHAP summary plot for SVM with RBF kernel')
        plt.show()
    else:
        print('・　SVMのSHAP値の計算には長時間を要するため skip する。')
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'svm_standardized_lime_explanation_holdout_kfCV_fold',
            'svm_standardized_lime_feature_contributions_holdout_kfCV_fold',
            'SVM standardized',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([
        np.mean(auc_scores), np.mean(accuracy_scores),
        np.mean(precision_scores), np.mean(recall_scores),
        np.mean(f1_scores)
    ], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['SVM standardized']
    )

    return [res2, roc_SVMS]







# RandomForest kfCV with fold-Averaged metrics
def RandomForestKFold(k=k, show_shap=True, perform_lime=True, shap_sample_size=100):
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold, train_test_split
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm

    print('')
    print('■■■ Random Forest kfCV with fold-Averaged metrics ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')


    # Stratified KFold cross-validation
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for fold metrics
    auc_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    specs = []
    aps = []

    # For true fold-averaged confusion matrix
    tn_rates = []
    fp_rates = []
    fn_rates = []
    tp_rates = []

    # For ROC and Precision-Recall curves per fold
    fprs = []
    tprs = []
    precisions = []
    recalls = []

    # Accumulate feature importances per fold
    feature_importance_list = []

    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_pred_probs = []

    for train_index, test_index in kf.split(X, y):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Initialize the RandomForest model with default parameters
        model = RandomForestClassifier(random_state=random_state)
        model.fit(X_train_fold, y_train_fold)

        # Collect feature importances
        feature_importances = model.feature_importances_
        feature_importance_list.append(feature_importances)

        # Predict classes and probabilities for evaluation
        y_pred = model.predict(X_test_fold)
        y_pred_prob = model.predict_proba(X_test_fold)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test_fold, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        # specificity
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        aps.append(average_precision_score(y_test_fold, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test_fold, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)
        prec_vals, rec_vals, _ = precision_recall_curve(y_test_fold, y_pred_prob)
        precisions.append(prec_vals)
        recalls.append(rec_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test_fold)
        all_pred_probs.extend(y_pred_prob)

    # Compute fold-averaged scores
    mean_auc   = np.mean(auc_scores)
    mean_acc   = np.mean(accuracy_scores)
    mean_prec  = np.mean(precision_scores)
    mean_rec   = np.mean(recall_scores)
    mean_f1    = np.mean(f1_scores)
    mean_spec  = np.mean(specs)
    mean_prerec= np.mean(aps)
    mean_cind  = mean_auc

    # Compute true fold-averaged confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_prerec,
        "Random Forest"
    )

    # Print summary
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')

    # Paths for figures
    hmname      = os.path.join(path_figure, 'random_forest_confusion_heatmap_kfCV.png')
    rocname     = os.path.join(path_figure, 'random_forest_roc_curve_kfCV.png')
    prerecname  = os.path.join(path_figure, 'random_forest_prerec_curve_kfCV.png')

    # Plot fold-averaged confusion matrix heatmap
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(hmname); plt.show(); plt.close()

    # Plot fold-averaged ROC curve
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_randomforest = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'Random Forest'], dtype=object)

    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Fold-Average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # Plot fold-averaged Precision-Recall curve
    mean_rec_curve    = np.linspace(0, 1, 100)
    pr_interp         = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve   = np.mean(pr_interp, axis=0)

    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'Fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # Calculate the average feature importance across all folds
    mean_feature_importances = np.mean(feature_importance_list, axis=0)
    column_names = df.columns[:-1]
    feature_importances_df = pd.DataFrame({'Feature': column_names, 'Importance': mean_feature_importances})
    feature_importances_df['Abs_Importance'] = feature_importances_df['Importance'].abs()
    feature_importances_df = feature_importances_df.sort_values(by='Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    excel_filename = os.path.join(path_table, 'random_forest_feature_importances_kfCV.xlsx')
    feature_importances_df.to_excel(excel_filename, index=False)

    # Get the top 10 features and their importances
    indices       = np.argsort(mean_feature_importances)[-10:]
    top_features  = df.columns[indices]
    top_importances = mean_feature_importances[indices]

    print('')
    print(f'◇ {k}-fold Cross Validation における Random Forest Feature Importance (fold-Averaged)')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.barh(top_features, top_importances, align='center')
    plt.xlabel('Average Feature Importance')
    plt.title(f'Top 10 Feature Importances - fold-Averaged over {k} Folds')
    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    font_prop = fm.FontProperties(fname=font_path)
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.savefig(os.path.join(path_figure, 'random_forest_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # Optionally retrain the model on the full dataset
    model_full = RandomForestClassifier(random_state=random_state)
    model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    model_filename = os.path.join(path_model, 'random_forest_full_model.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out method for SHAP and LIME analyses
    model_holdout = RandomForestClassifier(random_state=random_state)
    model_holdout.fit(x_train, t_train)

    if show_shap:
        if len(df.columns) < 1000:
            print('')
            print('□ Random Forest model の 各変数のSHAP値')
            print('')
            shap_sample_size = min(shap_sample_size, len(x_train))
            print(f'◇ SHAPのサンプルサイズ: {shap_sample_size}')
            print('')
            print('----- Be patient. It takes a few minutes. -----')
            print('')

            background_data = shap.utils.sample(x_train, shap_sample_size)
            explainer = shap.KernelExplainer(model_holdout.predict_proba, background_data)
            shap_values = explainer.shap_values(x_test)

            print(f'◇ Random Forest の SHAP値の計算には時間がかかるため、{shap_sample_size}個のデータを抽出して計算します。')
            positive_shap = shap_values[:, :, 1]

            plt.figure(figsize=(size_x, size_y))
            shap.summary_plot(positive_shap, x_test, feature_names=df.columns[:-1], plot_type="bar", show=False)
            ax = plt.gca()
            for label in ax.get_yticklabels():
                label.set_fontproperties(font_prop)
            plt.savefig(os.path.join(path_figure, 'random_forest_positive_shap_values_holdout.png'))
            plt.show(); plt.close()
            print('')

            mean_abs_shap = np.abs(positive_shap).mean(axis=0)
            shap_df = pd.DataFrame(mean_abs_shap.reshape(-1, 1), columns=["SHAP Value"], index=df.columns[:-1])
            shap_df = shap_df.sort_values(by="SHAP Value", ascending=False)
            shap_excel = os.path.join(path_table, 'random_forest_shap_values_holdout.xlsx')
            shap_df.to_excel(shap_excel)

        else:
            print('◇ 説明変数が1,000個を超えるため、Random ForestのSHAP値の計算をskipします。')
            print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'random_forest_lime_explanation_holdout',
            'random_forest_lime_feature_contributions_holdout',
            'Random Forest',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([
        mean_auc, mean_acc,
        mean_prec, mean_rec,
        mean_f1
    ], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Random Forest']
    )

    return [res2, roc_randomforest]








def RandomForestOptunaKFold(
    k=k,
    show_shap=True,
    perform_lime=True,
    shap_sample_size=100,
    threshold=0.5,
    n_trials=30,
    target='logloss'
):
    import os
    import pickle
    import logging
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm
    import optuna
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )

    # 初期出力
    print('')
    print(f'■■■ Random Forest kfCV with Optuna (Foldwise averaged, Optimizing {target})　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): '
          + str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5))
          + ' %)')
    print('')
    print('----- Be patient. It would take a few minutes. -----')
    print('')

    # Optuna ログを INFO レベルで出力
    optuna.logging.set_verbosity(optuna.logging.INFO)
    logging.getLogger("optuna").propagate = False

    def objective(trial):
        # ハイパーパラメータ探索空間
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 500),
            'max_depth': trial.suggest_categorical('max_depth', [None]+list(range(2, 65))),
            # 'max_depth': trial.suggest_int('max_depth', 2, 32, log=True),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
            'random_state': random_state
        }
        model = RandomForestClassifier(**params)
        cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

        if target == 'auc':
            scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')
            return np.mean(scores)
        else:  # 'logloss'
            scores = cross_val_score(model, X, y, cv=cv, scoring='neg_log_loss')
            return np.mean(scores)

    # Optuna Study 作成
    study = optuna.create_study(direction='maximize')
    optuna.logging.enable_default_handler()
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # ベストパラメータ出力
    best_trial_no = study.best_trial.number
    print('')
    print('◇ Hyperparameter Optimization for Random Forest with Optuna (detailed logs below)')
    print(f'Best trial #{best_trial_no} for maximizing {target}:')
    print("Best hyperparameters: ", study.best_params)
    print('')

    # 最良モデルの準備
    best_params = study.best_params.copy()
    best_params['random_state'] = random_state
    best_model = RandomForestClassifier(**best_params)

    # # ホールドアウト分割
    # x_train, x_test, t_train, t_test = train_test_split(
    #     X, y, test_size=test_size, random_state=random_state
    # )

    # 各種蓄積用リスト
    aucs = []; accs = []; precs = []; recs = []; f1s = []; specs = []; aps = []
    tn_rates = []; fp_rates = []; fn_rates = []; tp_rates = []
    fprs = []; tprs = []; precisions = []; recalls = []
    feature_importances = np.zeros(X.shape[1])
    all_y_test = []; all_pred_probs = []

    # k-fold クロスバリデーションによる評価
    cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    for train_idx, test_idx in cv.split(X, y):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]

        model = RandomForestClassifier(**best_params)
        model.fit(X_tr, y_tr)

        # 予測確率および閾値0.5でのクラス予測
        probs = model.predict_proba(X_te)[:, 1]
        preds = (probs >= threshold).astype(int)

        # 混同行列レート
        tn, fp, fn, tp = confusion_matrix(y_te, preds).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total); fp_rates.append(fp/total)
        fn_rates.append(fn/total); tp_rates.append(tp/total)
        specs.append(tn/(tn+fp))

        # 各種メトリクス
        aucs.append(roc_auc_score(y_te, probs))
        accs.append(accuracy_score(y_te, preds))
        precs.append(precision_score(y_te, preds, zero_division=0))
        recs.append(recall_score(y_te, preds))
        f1s.append(f1_score(y_te, preds))
        aps.append(average_precision_score(y_te, probs))

        # ROC/PR 曲線データ
        fpr, tpr, _ = roc_curve(y_te, probs)
        fprs.append(fpr); tprs.append(tpr)
        p_vals, r_vals, _ = precision_recall_curve(y_te, probs)
        precisions.append(p_vals); recalls.append(r_vals)

        # 全体プロット用蓄積
        all_y_test.extend(y_te); all_pred_probs.extend(probs)

        # 特徴量重要度
        feature_importances += model.feature_importances_

    # 平均化
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_prerec = np.mean(aps)
    feature_importances /= k

    # 混同行列マクロ平均
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                         [fold_fp, fold_tp]]) * 100

    # display_fold_averages 呼び出し
    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_auc, mean_prerec,
        "Random Forest with Optuna"
    )

    # プロット設定共通
    fig_kwargs = dict(figsize=(size_x, size_y))
    font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')

    # ヒートマップ
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(**fig_kwargs)
    sns.heatmap(fold_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'randomforest_optuna_confusion_heatmap.png'))
    plt.show(); plt.close()

    # ROC 曲線
    mean_fpr = np.linspace(0,1,100)
    interp_tprs = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr = np.mean(interp_tprs, axis=0)
    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(**fig_kwargs)
    plt.plot(mean_fpr, mean_tpr, label=f'Fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'randomforest_optuna_roc_curve.png'))
    plt.show(); plt.close()

    # Precision-Recall 曲線
    mean_rec = np.linspace(0,1,100)
    interp_prec = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        interp_prec.append(np.interp(mean_rec, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(interp_prec, axis=0)
    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(**fig_kwargs)
    plt.plot(mean_rec, mean_prec_curve, label=f'Fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'randomforest_optuna_pr_curve.png'))
    plt.show(); plt.close()

    # Feature Importance
    fi_df = pd.DataFrame({
        'Feature': df.columns[:-1],
        'Importance': feature_importances
    })
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values('Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    fi_df.to_excel(os.path.join(path_table, 'randomforest_optuna_feature_importances.xlsx'), index=False)

    print('')
    print(f'◇ {k}-fold CV + Optuna による Random Forest の Feature Importance')
    print('')
    plt.figure(**fig_kwargs)
    plt.barh(fi_df['Feature'][:10], fi_df['Importance'][:10])
    plt.xlabel('Importance'); plt.title(f'Top 10 Feature Importances ({k}-fold CV + Optuna)')
    ax = plt.gca()
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    # ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'randomforest_optuna_feature_importances.png'))
    plt.show(); plt.close()

    # モデル保存
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'randomforest_optuna_best_model.pkl'), 'wb') as f:
        pickle.dump(best_model, f)

    # SHAP と LIME
    holdout_model = RandomForestClassifier(**best_params)
    holdout_model.fit(x_train, t_train)

    if show_shap:
        print('')
        print('□ Random Forest best model (Optuna) の SHAP 値')
        shap_sample = min(shap_sample_size, len(x_train))
        print(f'◇ SHAPのサンプルサイズ: {shap_sample}')
        background = shap.utils.sample(x_train, shap_sample)
        explainer = shap.KernelExplainer(holdout_model.predict_proba, background)
        shap_vals = explainer.shap_values(x_test)
        pos_shap = shap_vals[:, :, 1]
        plt.figure(**fig_kwargs)
        shap.summary_plot(pos_shap, x_test, feature_names=df.columns[:-1], plot_type="bar", show=False)
        ax = plt.gca()
        for lbl in ax.get_yticklabels():
            lbl.set_fontproperties(font_prop)
        plt.savefig(os.path.join(path_figure, 'randomforest_optuna_shap.png'))
        plt.show(); plt.close()

    if perform_lime:
        print('')
        print('□ Random Forest best model (Optuna) の LIME 分析')
        LimeContributionValueFunction(
            holdout_model, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'randomforest_optuna_lime_explain', 'randomforest_optuna_lime_features',
            'Random Forest kfCV with Optuna', task_type='classification', random_state=random_state
        )
        print('')

    # 結果返却
    res = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['Random Forest with Optuna']
    )
    optuna.logging.disable_default_handler()
    return [res, (mean_fpr, mean_tpr, mean_auc, 'Random Forest with Optuna')]





# XGBoost kfCV with fold-Averaged metrics
def XGBoostKFold(k=k, show_shap=True, perform_lime=True):
    from xgboost import XGBClassifier
    from collections import defaultdict
    import numpy as np
    import pandas as pd
    import os
    import pickle
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import (
        r2_score, roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold

    print('')
    print('■■■ XGBoost kfCV with fold-Averaged metrics ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): '
          + str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5))
          + ' %)')
    print('')

    # データの線形性の確認
    lr = LinearRegression()
    lr.fit(X, y)
    y_pred_lr = lr.predict(X)
    r2 = r2_score(y, y_pred_lr)
    linear = 1 if r2 >= 0.8 else 0

    booster_option = 'gblinear' if linear == 1 else 'gbtree'

    # Use StratifiedKFold instead of KFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for fold metrics
    auc_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    specs = []
    aps = []

    # For ROC and Precision-Recall curve plotting per fold
    fprs = []
    tprs = []
    precisions = []
    recalls = []

    # For true fold-averaged confusion matrix
    tn_rates = []
    fp_rates = []
    fn_rates = []
    tp_rates = []

    # For aggregation of test labels and predicted probabilities
    all_y_test = []
    all_predictions = []

    # Accumulate feature importances per fold
    feature_importances = defaultdict(list)
    feature_names = df.columns.tolist()
    feature_index_to_name = {f'f{i}': name for i, name in enumerate(feature_names)}

    # ----------------- Stratified K-fold Cross Validation -----------------
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = XGBClassifier(
            booster=booster_option,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=random_state
        )
        model.fit(X_train, y_train)

        # Retrieve feature importances based on booster type
        if linear == 0:
            weight_imp = model.get_booster().get_score(importance_type='weight')
            for idx, name in feature_index_to_name.items():
                feature_importances[name].append(weight_imp.get(idx, 0))
        else:
            coeffs = model.coef_.ravel()
            for i, name in enumerate(feature_names):
                feature_importances[name].append(abs(coeffs[i]))

        # Predict classes and probabilities for evaluation
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Compute per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)
        specs.append(tn / (tn + fp))

        # Compute per-fold metrics
        auc_scores.append(roc_auc_score(y_test, y_pred_prob))
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
        aps.append(average_precision_score(y_test, y_pred_prob))

        # Collect for fold ROC/PR curves
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)
        p_vals, r_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(p_vals)
        recalls.append(r_vals)

        # Aggregate for overall ROC call
        all_y_test.extend(y_test)
        all_predictions.extend(y_pred_prob)

    # -----------------Calculate fold-average scores-----------------
    mean_auc    = np.mean(auc_scores)
    mean_acc    = np.mean(accuracy_scores)
    mean_prec   = np.mean(precision_scores)
    mean_rec    = np.mean(recall_scores)
    mean_f1     = np.mean(f1_scores)
    mean_spec   = np.mean(specs)
    mean_prerec = np.mean(aps)

    # -----------------True fold-averaged confusion matrix-----------------
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]])
    fold_cm_pct = fold_cm * 100

    # Display fold-averaged table
    display_fold_averages(
        k, df, fold_cm_pct,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_auc, mean_prerec,
        "XGBoost kfCV"
    )

    # Common plot settings
    fig_kwargs = dict(figsize=(size_x, size_y))
    font_prop  = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')

    # -----------------Plot fold-averaged confusion matrix heatmap-----------------
    print('')
    print('□　fold-Averageによる混同行列のヒートマップ:')
    print('')
    plt.figure(**fig_kwargs)
    sns.heatmap(fold_cm_pct, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('fold-Averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'xgboost_confusion_heatmap_kfCV.png'))
    plt.show(); plt.close()

    # -----------------Plot fold-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    rocname        = os.path.join(path_figure, 'xgboost_roc_curve_kfCV.png')

    print('')
    print('□　ROC curve:')
    print('')
    plt.figure(**fig_kwargs)
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Fold-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)): plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot fold-averaged Precision-Recall curve-----------------
    mean_rec_curve  = np.linspace(0, 1, 100)
    pr_interp       = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        pr_interp.append(np.interp(mean_rec_curve, recalls[i][idx], precisions[i][idx]))
    mean_prec_curve = np.mean(pr_interp, axis=0)
    prerecname      = os.path.join(path_figure, 'xgboost_prerec_curve_kfCV.png')

    print('')
    print('□　Precision-Recall curve:')
    print('')
    plt.figure(**fig_kwargs)
    plt.plot(mean_rec_curve, mean_prec_curve, label=f'Fold-average PRC (AUC = {mean_prerec:.4f})')
    for i in range(len(recalls)): plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Fold-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # -----------------Fold-based Feature-importance extraction-----------------
    avg_imp = {f: np.mean(vals) for f, vals in feature_importances.items()}
    sorted_imp = dict(sorted(avg_imp.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({
        'Feature': list(sorted_imp.keys()),
        'Importance': list(sorted_imp.values())
    })
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values('Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    fi_df.to_excel(os.path.join(path_table, 'xgboost_feature_importances_kfCV.xlsx'), index=False)

    print('')
    print(f'□ {k}-fold Cross Validation における XGBoost Feature Importance (fold-Averaged)')
    print('')
    fig_kwargs['figsize'] = (12, 8)
    plt.figure(**fig_kwargs)
    plt.barh(fi_df['Feature'][:10], fi_df['Importance'][:10])
    plt.xlabel('Average Importance')
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of XGBoost')
    ax = plt.gca()
    ax.invert_yaxis()
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    ax.set_aspect('equal', adjustable='box')
    plt.savefig(os.path.join(path_figure, 'xgboost_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # -----------------Retrain the model on the full dataset-----------------
    model_full = XGBClassifier(
        booster=booster_option,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=random_state
    )
    model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'xgboost_kfCV_model.pkl'), 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out method for SHAP and LIME analyses
    model_holdout = XGBClassifier(
        booster=booster_option,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=random_state
    )
    model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'xgboost_shap_values_holdout_holdout',
            'xgboost_shap_values_holdout_holdout',
            'XGBoost', task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'xgboost_lime_explanation_holdout',
            'xgboost_lime_feature_contributions_holdout',
            'XGBoost', task_type='classification', random_state=random_state
        )
        print('')

    # -----------------Prepare and return results-----------------
    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame([res], columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'], index=['XGBoost'])
    roc_xgboost = (mean_fpr, mean_tpr_curve, mean_auc, 'XGBoost')

    return [res2, roc_xgboost]






def XGBoostOptunaKFold(k=k,
                       show_shap=True,
                       perform_lime=True,
                       target='auc',
                       n_trials=50):
    import xgboost as xgb
    import optuna
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        log_loss, roc_curve,
        precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    from sklearn.model_selection import StratifiedKFold
    import numpy as np
    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm
    from collections import defaultdict

    print('')
    print(f'■■■ XGBoost kfCV with Optuna (Foldwise averaged, Optimizing {target}) ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): '
          + str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5))
          + ' %)')
    print('')
    print('----- Be patient. It takes a whole. -----')
    print('')

    feature_names = df.columns[:-1]
    # Boosterの“f0, f1, …”インデックスを元の特徴量名にマッピング
    feature_index_to_name = {f'f{i}': name for i, name in enumerate(feature_names)}

    # データの線形性の確認
    lr = LinearRegression()
    lr.fit(X, y)
    y_pred_lr = lr.predict(X)
    r2 = r2_score(y, y_pred_lr)
    if r2 >= 0.8:
        linear = 1
    elif r2 < 0.2:
        linear = 0
    else:
        linear = 0

    booster_option = 'gblinear' if linear == 1 else 'gbtree'

    # right after you define `target`
    target2_map = {
        'auc': 'AUC',
        'logloss': 'minus_logloss'
    }

    if target not in target2_map:
        raise ValueError(f"Unknown target: {target}")
    display_name = target2_map[target]

    def objective(trial):
        # ハイパーパラメータ探索空間の定義
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 9),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True)
        }
        model = xgb.XGBClassifier(
            **params,
            booster=booster_option,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=random_state,
            early_stopping_rounds=50
        )
        # StratifiedKFold に変更
        kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

        fold_metrics = []
        for train_index, test_index in kf.split(X, y):
            X_tr, X_te = X[train_index], X[test_index]
            y_tr, y_te = y[train_index], y[test_index]

            model.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False)
            y_prob = model.predict_proba(X_te)[:, 1]

            # Optuna の目的関数を target パラメータで切り替え
            if target == 'auc':
                m = roc_auc_score(y_te, y_prob)
            elif target == 'logloss':
                m = -log_loss(y_te, y_prob)
            else:
                raise ValueError(f"Unknown target: {target}")
            fold_metrics.append(m)

        return np.mean(fold_metrics)

    study = optuna.create_study(direction='maximize')
    optuna.logging.enable_default_handler()
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_trial_no = study.best_trial.number
    print('')
    print('')
    print('◇ Hyperparameter Optimization with Optuna for XGBoost (detailed logs below)')
    print(f'   Best trial #{best_trial_no} for maximizing {display_name}')
    print(f'   The booster used: {booster_option}')
    print("   Best hyperparameters: ", study.best_params)
    print('')
    print('')

    # # 結果の表示（Foldwise-averaged）
    # print('')
    # print(f'□ {k}-fold Cross Validation を用いた XGBoost optimized with Optuna による2値分類の検定結果 (Foldwise-averaged)')

    # StratifiedKFold による最終評価
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # フォールド毎の混同行列率、各種指標、曲線データを格納するリスト
    tns, fps, fns, tps = [], [], [], []
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    feature_importances = defaultdict(list)

    # フォールド毎の評価ループ
    for train_index, test_index in kf.split(X, y):
        X_tr, X_te = X[train_index], X[test_index]
        y_tr, y_te = y[train_index], y[test_index]

        best_model = xgb.XGBClassifier(
            **study.best_params,
            booster=booster_option,
            use_label_encoder=False,
            eval_metric=target,
            random_state=random_state
        )
        best_model.fit(X_tr, y_tr)

        # Feature‐importance の抽出
        if linear == 0:
            imp = best_model.get_booster().get_score(importance_type='weight')
            for idx, fname in feature_index_to_name.items():
                feature_importances[fname].append(imp.get(idx, 0))
        else:
            coefs = best_model.coef_
            if coefs.ndim != 1:
                raise ValueError("Unexpected coefficient shape.")
            imp = {feature_names[i]: abs(c) for i, c in enumerate(coefs)}
            for fname in feature_names:
                feature_importances[fname].append(imp.get(fname, 0))

        # 予測と混同行列
        y_prob = best_model.predict_proba(X_te)[:, 1]
        y_pred = best_model.predict(X_te)
        tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
        total = tn + fp + fn + tp
        tns.append(tn/total); fps.append(fp/total)
        fns.append(fn/total); tps.append(tp/total)

        # 指標を計算
        aucs.append(roc_auc_score(y_te, y_prob))
        accs.append(accuracy_score(y_te, y_pred))
        precs.append(precision_score(y_te, y_pred, zero_division=0))
        recs.append(recall_score(y_te, y_pred))
        f1s.append(f1_score(y_te, y_pred))
        specs.append(tn/(tn+fp))
        aps.append(average_precision_score(y_te, y_prob))

        # ROC 曲線データ
        fpr, tpr, _ = roc_curve(y_te, y_prob)
        fprs.append(fpr); tprs.append(tpr)

        # Precision‐Recall 曲線データ
        p_vals, r_vals, _ = precision_recall_curve(y_te, y_prob)
        precisions.append(p_vals); recalls.append(r_vals)

    # Foldwise-averaged 混同行列
    macro_tn, macro_fp = np.mean(tns), np.mean(fps)
    macro_fn, macro_tp = np.mean(fns), np.mean(tps)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    display_fold_averages(
        k, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        np.mean(accs), np.mean(precs), np.mean(recs), np.mean(f1s),
        np.mean(specs), np.mean(aucs), np.mean(aucs), np.mean(aps),
        "XGBoost with Optuna"
    )

    # ヒートマップ表示
    print('')
    print('□　Foldwise-averaged Confusion Matrix (Heatmap):')
    print('')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(
        macro_cm, annot=True, fmt='.2f', cbar=False,
        xticklabels=['Pred 0', 'Pred 1'],
        yticklabels=['True 0', 'True 1']
    )
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title(f'Foldwise-averaged Confusion Matrix (%) over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    hmname = os.path.join(path_figure, 'xgboost_confusion_heatmap_kfCV_optuna.png')
    plt.savefig(hmname)
    plt.show(); plt.close()

    # Foldwise-averaged ROC 曲線
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)

    print('')
    print('□　ROC Curve (Foldwise-averaged):')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve,
             label=f'Foldwise-avg ROC (AUC = {np.mean(aucs):.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    rocname = os.path.join(path_figure, 'xgboost_roc_curve_kfCV_optuna.png')
    plt.savefig(rocname)
    plt.show(); plt.close()

    # Foldwise-averaged Precision‐Recall 曲線
    mean_recall = np.linspace(0, 1, 100)
    pr_interps = []
    for i in range(len(recalls)):
        idx_sort = np.argsort(recalls[i])
        rec_s = recalls[i][idx_sort]
        prec_s = precisions[i][idx_sort]
        pr_interps.append(np.interp(mean_recall, rec_s, prec_s))
    mean_precision_curve = np.mean(pr_interps, axis=0)

    print('')
    print('□　Precision-Recall Curve (Foldwise-averaged):')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_precision_curve,
             label=f'Foldwise-avg PRC (AUC = {np.mean(aps):.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    prerecname = os.path.join(path_figure, 'xgboost_prerec_curve_kfCV_optuna.png')
    plt.savefig(prerecname)
    plt.show(); plt.close()

    # ----------------- Foldwise-averaged Feature-importance extraction -----------------
    avg_imp = {f: np.mean(vals) for f, vals in feature_importances.items()}
    sorted_imp = dict(sorted(avg_imp.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({
        'Feature': list(sorted_imp.keys()),
        'Importance': list(sorted_imp.values())
    })
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values(
        'Abs_Importance', ascending=False
    ).drop('Abs_Importance', axis=1)
    fi_df.to_excel(
        os.path.join(path_table, 'xgboost_feature_importances_kfCV_optuna.xlsx'),
        index=False
    )

    print('')
    print(f'◇ {k}-fold Cross Validation を用いた '
          'XGBoost with Optuna による '
          'Feature Importance (Foldwise-averaged)')
    print('')

    # プロット
    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    font_prop = fm.FontProperties(fname=font_path)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(fi_df['Feature'][:10], fi_df['Importance'][:10], color='skyblue')
    ax.invert_yaxis()
    ax.set_xlabel('Average Importance')
    ax.set_title(
        f'Top 10 Feature Importances in {k}-fold CV of '
        'XGBoost with Optuna (Foldwise-averaged)'
    )
    for lbl in ax.get_yticklabels():
        lbl.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.savefig(
        os.path.join(path_figure, 'xgboost_feature_importances_kfCV_optuna.png')
    )
    plt.show(); plt.close()

    # Full データで再学習しモデル保存
    best_model_full = xgb.XGBClassifier(
        **study.best_params,
        booster=booster_option,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=random_state
    )
    best_model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(
        os.path.join(path_model, 'xgboost_kfCV_optuna_model_full.pkl'),
        'wb'
    ) as f:
        pickle.dump(best_model_full, f)

    # Hold-out モデル
    best_model_holdout = xgb.XGBClassifier(random_state=random_state)
    best_model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            best_model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'xgboost_shap_values_holdout_optuna',
            'xgboost_shap_values_holdout_optuna',
            'XGBoost with Optuna',
            task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            best_model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'xgboost_lime_explanation_holdout_optuna',
            'xgboost_lime_feature_contributions_holdout_optuna',
            'XGBoost with Optuna',
            task_type='classification',
            random_state=random_state
        )
        print('')

    # 結果を DataFrame で返却
    res = [
        np.mean(aucs),
        np.mean(accs),
        np.mean(precs),
        np.mean(recs),
        np.mean(f1s)
    ]
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['XGBoost with Optuna']
    )

    optuna.logging.disable_default_handler()
    return [res2,
            (mean_fpr, mean_tpr_curve, np.mean(aucs), 'XGBoost with Optuna')]








# LightGBM KFold Cross Validation
def LightGBMKFold(k=k, show_shap=True, perform_lime=True):
    from lightgbm import LGBMClassifier
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score
    )
    from collections import defaultdict
    import contextlib
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os
    import pickle
    import matplotlib.font_manager as fm

    print('')
    print('■■■ LightGBM kfCV (Foldwise-Averaged) ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    feature_names = df.columns[:-1]

    # Use StratifiedKFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for macro metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For ROC/PR curve plotting per fold
    fprs, tprs, precisions, recalls = [], [], [], []
    # For true macro-averaged confusion matrix
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    # For aggregated showscores (no longer used)
    all_y_test, all_predictions = [], []
    # For collecting feature importances
    feature_importances = defaultdict(list)

    # ----------------- Stratified K-fold Cross Validation -----------------
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = LGBMClassifier(boosting_type='gbdt', random_state=random_state, verbose=-1)
        with open(os.devnull, "w") as f_null, contextlib.redirect_stdout(f_null), contextlib.redirect_stderr(f_null):
            model.fit(X_train, y_train)

        y_pred      = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Metrics per fold
        auc_fold  = roc_auc_score           (y_test, y_pred_prob)
        acc_fold  = accuracy_score          (y_test, y_pred)
        prec_fold = precision_score         (y_test, y_pred, zero_division=0)
        rec_fold  = recall_score            (y_test, y_pred)
        f1_fold   = f1_score                (y_test, y_pred)
        ap_fold   = average_precision_score (y_test, y_pred_prob)
        spec_fold = tn / (tn + fp)

        aucs .append(auc_fold)
        accs .append(acc_fold)
        precs.append(prec_fold)
        recs .append(rec_fold)
        f1s .append(f1_fold)
        specs.append(spec_fold)
        aps .append(ap_fold)

        # ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall curve
        p_vals, r_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(p_vals); recalls.append(r_vals)

        # Feature importances
        for i, imp in enumerate(model.feature_importances_):
            feature_importances[feature_names[i]].append(imp)

    # -----------------True macro-averaged confusion matrix & macro metrics-----------------
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_cind   = mean_auc
    mean_pr_auc = np.mean(aps)

    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]])
    macro_cm_percent = macro_cm * 100

    display_fold_averages(
        k, df,
        macro_cm_percent,
        macro_tp, macro_fp,
        macro_fn, macro_tn,
        mean_acc, mean_prec,
        mean_rec, mean_f1,
        mean_spec, mean_auc,
        mean_cind, mean_pr_auc,
        "LightGBM"
    )

    # -----------------Plot foldwise-averaged ROC curve & build roc_lightgbm-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)

    rocname = os.path.join(path_figure, 'lightGBM_roc_curve_kfCV.png')
    roc_lightgbm = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'lightGBM'], dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Foldwise-average ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        recall_sorted    = recalls[i][idx]
        precision_sorted = precisions[i][idx]
        precisions_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))
    mean_precision_curve = np.mean(precisions_interp, axis=0)
    prerecname = os.path.join(path_figure, 'lightGBM_prerec_curve_kfCV.png')

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_precision_curve, label=f'Foldwise-average PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # -----------------Average feature importance & Excel export-----------------
    avg_importance = {feat: np.mean(imps) for feat, imps in feature_importances.items()}
    sorted_avg_imp = dict(sorted(avg_importance.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({
        'Feature': list(sorted_avg_imp.keys()),
        'Importance': list(sorted_avg_imp.values())
    })
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values('Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    excel_filename = os.path.join(path_table, 'lightgbm_feature_importances_kfold.xlsx')
    fi_df.to_excel(excel_filename, index=False)

    # -----------------Plot feature importances-----------------
    def plot_importance(title, imp_dict, xlabel):
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(title)
        keys   = list(imp_dict.keys())[:10]
        values = list(imp_dict.values())[:10]
        ax.barh(keys, values)
        ax.set_xlabel(xlabel)
        font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
        for lbl in ax.get_yticklabels():
            lbl.set_fontproperties(font_prop)
        ax.invert_yaxis(); plt.tight_layout()
        plt.savefig(os.path.join(path_figure, 'lightgbm_feature_importances_kfCV.png'))
        plt.show(); plt.close()

    print('')
    print(f'◆ {k}-fold Cross Validation を用いた LightGBM による Feature Importance (Foldwise-Averaged)')
    print('')
    plot_importance(f"Top 10 Feature Importances in {k}-fold CV of LightGBM", sorted_avg_imp, "Average Importance")

    # -----------------Retrain on full dataset & save model-----------------
    model_full = LGBMClassifier(boosting_type='gbdt', random_state=random_state, verbose=-1)
    with open(os.devnull, "w") as f_null, contextlib.redirect_stdout(f_null), contextlib.redirect_stderr(f_null):
        model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'lightgbm_full_model.pkl'), 'wb') as file:
        pickle.dump(model_full, file)

    # -----------------Hold-out for SHAP/LIME analyses-----------------
    model_holdout = LGBMClassifier(boosting_type='gbdt', random_state=random_state, verbose=-1)
    with open(os.devnull, "w") as f_null, contextlib.redirect_stdout(f_null), contextlib.redirect_stderr(f_null):
        model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_shap_values_holdout',
            'lightgbm_shap_values_holdout',
            'LightGBM',
            task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_lime_explanation_holdout',
            'lightgbm_lime_feature_contributions_holdout',
            'LightGBM',
            task_type='classification', random_state=random_state
        )
        print('')

    # -----------------Return results-----------------
    res  = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['LightGBM']
    )
    return [res2, roc_lightgbm]







# target: 'auc' or 'log_loss'
def LightGBMOptunaKFold(k=k, show_shap=True, perform_lime=True, n_trials=50, target='log_loss'):
    import lightgbm as lgb
    from lightgbm import early_stopping
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve,
        precision_recall_curve, average_precision_score,
        log_loss
    )
    import contextlib
    from contextlib import redirect_stderr
    import optuna
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from collections import defaultdict
    import pickle
    import matplotlib.font_manager as fm

    print('')
    print(f'■■■ LightGBM kfCV with Optuna (Foldwise-Averaged, Optimizing {target}) ■■■')
    print('')
    print('')
    print('----- Be patient. It may take a while. -----')
    print('')

    feature_names = df.columns[:-1]

    def objective(trial):
        import contextlib
        # Hyperparameter search space
        max_depth   = trial.suggest_int('max_depth', 3, 9)
        num_leaves  = trial.suggest_int(
            'num_leaves',
            int(2 ** max_depth / 2),
            2 ** max_depth,
            log=True
        )
        params = {
            'n_estimators':      trial.suggest_int('n_estimators', 100, 1000),
            'max_depth':         max_depth,
            'num_leaves':        num_leaves,
            'learning_rate':     trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
            'subsample':         trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_weight':  trial.suggest_int('min_child_weight', 1, 10),
            'reg_lambda':        trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'reg_alpha':         trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        }

        model = lgb.LGBMClassifier(
            **params,
            random_state=random_state,
            objective='binary',
            verbosity=0
        )

        # Stratified K-fold for objective
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        aucs, loglosses = [], []

        for train_idx, test_idx in skf.split(X, y):
            X_tr, X_te = X[train_idx], X[test_idx]
            y_tr, y_te = y[train_idx], y[test_idx]

            with open(os.devnull, "w") as f, \
                 contextlib.redirect_stdout(f), \
                 contextlib.redirect_stderr(f):
                model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_te, y_te)],
                    callbacks=[early_stopping(stopping_rounds=50, verbose=False)],
                )

            y_prob = model.predict_proba(X_te)[:, 1]
            aucs.append(roc_auc_score(y_te, y_prob))
            loglosses.append(log_loss(y_te, y_prob))

        if target == 'auc':
            return np.mean(aucs)
        else:  # 'log_loss'
            # maximize negative log loss
            return -np.mean(loglosses)

    # Enable Optuna log handler
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print('')
    print('◇ Best hyperparameters: ', study.best_params)
    print('')

    # Final foldwise evaluation with best params
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    feature_importances = defaultdict(list)

    for train_idx, test_idx in skf.split(X, y):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]

        best_model = lgb.LGBMClassifier(
            **study.best_params,
            random_state=random_state,
            objective='binary',
            verbosity=-1
        )

        with open(os.devnull, 'w') as err:
            with redirect_stderr(err):
                best_model.fit(X_tr, y_tr)

        y_pred      = best_model.predict(X_te)
        y_pred_prob = best_model.predict_proba(X_te)[:, 1]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Metrics per fold
        aucs .append(roc_auc_score           (y_te, y_pred_prob))
        accs .append(accuracy_score          (y_te, y_pred))
        precs.append(precision_score         (y_te, y_pred, zero_division=0))
        recs .append(recall_score            (y_te, y_pred))
        f1s .append(f1_score                  (y_te, y_pred))
        aps .append(average_precision_score  (y_te, y_pred_prob))
        specs.append(tn / (tn + fp))

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_te, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall curve data
        p_vals, r_vals, _ = precision_recall_curve(y_te, y_pred_prob)
        precisions.append(p_vals); recalls.append(r_vals)

        # Feature importances
        for i, imp in enumerate(best_model.feature_importances_):
            feature_importances[feature_names[i]].append(imp)

    # Compute foldwise-averaged scores
    mean_auc     = np.mean(aucs)
    mean_acc     = np.mean(accs)
    mean_prec    = np.mean(precs)
    mean_rec     = np.mean(recs)
    mean_f1      = np.mean(f1s)
    mean_spec    = np.mean(specs)
    mean_pr_auc  = np.mean(aps)
    mean_cind    = mean_auc

    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # Display foldwise-averaged confusion & metrics table
    display_fold_averages(
        k, df,
        macro_cm,
        macro_tp, macro_fp,
        macro_fn, macro_tn,
        mean_acc, mean_prec,
        mean_rec, mean_f1,
        mean_spec, mean_auc,
        mean_cind, mean_pr_auc,
        "LightGBM with Optuna"
    )

    # Plot foldwise-averaged ROC & build return object
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    rocname        = os.path.join(path_figure, 'lightGBM_roc_curve_kfCV_optuna.png')
    roc_lightgbmOptuna = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'lightGBM with Optuna'], dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()

    # Plot foldwise-averaged Precision-Recall curve
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'lightGBM_prerec_curve_kfCV_optuna.png')

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # Average feature importances & export
    avg_imp = {f: np.mean(imps) for f, imps in feature_importances.items()}
    sorted_imp = dict(sorted(avg_imp.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({'Feature': list(sorted_imp.keys()), 'Importance': list(sorted_imp.values())})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values('Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    fi_df.to_excel(os.path.join(path_table, 'lightgbm_feature_importances_kfCV_optuna.xlsx'), index=False)

    # Plot feature importances
    def plot_importance(title, imp_dict, xlabel):
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(title)
        keys   = list(imp_dict.keys())[:10]
        values = list(imp_dict.values())[:10]
        ax.barh(keys, values)
        ax.set_xlabel(xlabel)
        font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
        for lbl in ax.get_yticklabels():
            lbl.set_fontproperties(font_prop)
        ax.invert_yaxis()
        plt.savefig(os.path.join(path_figure, 'lightgbm_feature_importances_kfCV_optuna.png'))
        plt.show(); plt.close()

    print('')
    print(f'◆ {k}-fold CV による LightGBM Feature Importance (Optuna)')
    print('')
    plot_importance(
        f"Top 10 Feature Importances in {k}-fold CV of LightGBM (Optuna)",
        sorted_imp, "Average Importance"
    )

    # Retrain best model on full data & save
    best_full = lgb.LGBMClassifier(
        **study.best_params,
        random_state=random_state,
        objective='binary',
        verbosity=-1
    )
    with open(os.devnull, "w") as f, \
         contextlib.redirect_stdout(f), \
         contextlib.redirect_stderr(f):
        best_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'lightgbm_full_model_optuna.pkl'), 'wb') as file:
        pickle.dump(best_full, file)

    # Hold-out for SHAP & LIME
    holdout = lgb.LGBMClassifier(
        **study.best_params,
        random_state=random_state,
        objective='binary',
        verbosity=-1
    )
    holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_shap_values_holdout_optuna',
            'lightgbm_shap_values_holdout_optuna',
            'LightGBM kfCV with Optuna',
            task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_lime_explanation_holdout_optuna',
            'lightgbm_lime_feature_contributions_holdout_optuna',
            'LightGBM kfCV with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    # Prepare and return
    res = [mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]
    res2 = pd.DataFrame([res],
                        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
                        index=['LightGBM with Optuna'])

    optuna.logging.disable_default_handler()
    return [res2, roc_lightgbmOptuna]







def LightGBMTunerOptunaKFold(k=k, show_shap=True, perform_lime=True, target='log_loss'):
    import os
    import pickle
    import contextlib
    from contextlib import redirect_stderr
    from collections import defaultdict

    import lightgbm as lgb
    from optuna.integration import LightGBMTunerCV
    import optuna
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
        confusion_matrix, roc_curve, precision_recall_curve, average_precision_score
    )
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm

    print('')
    print(f'■■■ LightGBMTuner k-Fold CV with Optuna (Foldwise-Averaged, optimizing {target}) ■■■')
    print('')
    print('----- Be patient. It may take a while. -----')
    print('')

    feature_names = df.columns[:-1].tolist()
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    # Stratified K-Fold cross-validation setup
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    folds = list(skf.split(X, y))

    # Prepare LightGBM dataset
    train_data = lgb.Dataset(X, label=y)

    # Fixed parameters with chosen metric
    metric_name = 'auc' if target == 'auc' else 'binary_logloss'
    fixed_params = {
        "objective": "binary",
        "metric": metric_name,
        "verbosity": -1,
        "seed": random_state,
    }

    # Initialize LightGBMTunerCV
    tuner = LightGBMTunerCV(
        params=fixed_params,
        train_set=train_data,
        folds=folds,
        num_boost_round=1000
    )

    # Enable Optuna INFO logs, silence LightGBM warnings
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    with open(os.devnull, 'w') as err:
        with redirect_stderr(err):
            tuner.run()

    # Best hyperparameters
    best_params = tuner.best_params
    print('')
    print("◇ Best hyperparameters: ", best_params)
    print('')

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    feature_importances = defaultdict(list)

    # Foldwise evaluation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = lgb.LGBMClassifier(**best_params, random_state=random_state)
        with open(os.devnull, 'w') as err:
            with redirect_stderr(err):
                model.fit(X_train, y_train)

        y_pred      = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Metrics per fold
        aucs .append(roc_auc_score            (y_test, y_pred_prob))
        accs .append(accuracy_score            (y_test, y_pred))
        precs.append(precision_score           (y_test, y_pred, zero_division=0))
        recs .append(recall_score              (y_test, y_pred))
        f1s .append(f1_score                    (y_test, y_pred))
        aps .append(average_precision_score    (y_test, y_pred_prob))
        specs.append(tn / (tn + fp))

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall curve data
        p_vals, r_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(p_vals); recalls.append(r_vals)

        # Feature importances
        for i, imp in enumerate(model.feature_importances_):
            feature_importances[feature_names[i]].append(imp)

    # Compute foldwise-averaged scores
    mean_auc     = np.mean(aucs)
    mean_acc     = np.mean(accs)
    mean_prec    = np.mean(precs)
    mean_rec     = np.mean(recs)
    mean_f1      = np.mean(f1s)
    mean_spec    = np.mean(specs)
    mean_pr_auc  = np.mean(aps)
    mean_cind    = mean_auc

    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # Display foldwise-averaged confusion & metrics table
    display_fold_averages(
        k, df,
        macro_cm,
        macro_tp, macro_fp,
        macro_fn, macro_tn,
        mean_acc, mean_prec,
        mean_rec, mean_f1,
        mean_spec, mean_auc,
        mean_cind, mean_pr_auc,
        "LightGBMTuner with Optuna"
    )

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    rocname        = os.path.join(path_figure, 'lightGBM_tuner_roc_curve_kfCV_optuna.png')
    roc_lightgbm_tuner_optuna = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'lightGBMtuner with Optuna'], dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocname)
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        recall_sorted    = recalls[i][idx]
        precision_sorted = precisions[i][idx]
        precisions_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'lightGBM_tuner_prerec_curve_kfCV_optuna.png')

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecname)
    plt.show()
    plt.close()

    # -----------------Average feature importances & export-----------------
    avg_imp = {f: np.mean(imps) for f, imps in feature_importances.items()}
    sorted_imp = dict(sorted(avg_imp.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({'Feature': list(sorted_imp.keys()), 'Importance': list(sorted_imp.values())})
    fi_df['Abs_Importance'] = fi_df['Importance'].abs()
    fi_df = fi_df.sort_values('Abs_Importance', ascending=False).drop('Abs_Importance', axis=1)
    fi_df.to_excel(os.path.join(path_table, 'lightgbm_tuner_feature_importances_kfCV_optuna.xlsx'), index=False)

    # -----------------Plot feature importances-----------------
    def plot_importance(title, imp_dict, xlabel):
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.set_title(title)
        keys   = list(imp_dict.keys())[:10]
        values = list(imp_dict.values())[:10]
        ax.barh(keys, values)
        ax.set_xlabel(xlabel)
        font_prop = fm.FontProperties(fname='/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc')
        for lbl in ax.get_yticklabels():
            lbl.set_fontproperties(font_prop)
        ax.invert_yaxis()
        plt.savefig(os.path.join(path_figure, 'lightgbm_tuner_feature_importances_kfCV_optuna.png'))
        plt.show()
        plt.close()

    print('')
    print(f'◆ {k}-Fold CV による LightGBM Feature Importance (Optuna)')
    print('')
    plot_importance(
        f"Top 10 Feature Importances in {k}-Fold CV optimized with LightGBMTunerCV from Optuna",
        sorted_imp, "Average Importance"
    )

    # Retrain on full dataset & save model
    best_model_full = lgb.LGBMClassifier(**best_params, random_state=random_state)
    with open(os.devnull, "w") as f, \
         contextlib.redirect_stdout(f), \
         contextlib.redirect_stderr(f):
        best_model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'lightgbm_tuner_full_model_optuna.pkl'), 'wb') as file:
        pickle.dump(best_model_full, file)

    # Hold-out for SHAP & LIME
    best_model_holdout = lgb.LGBMClassifier(**best_params, random_state=random_state)
    best_model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            best_model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_tuner_shap_values_holdout_optuna',
            'lightgbm_tuner_shap_values_holdout_optuna',
            'LightGBMTuner kfCV with Optuna',
            task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            best_model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'lightgbm_tuner_lime_explanation_holdout_optuna',
            'ligthgbm_tuner_lime_feature_contributions_holdout_optuna',
            'LightGBMTuner kfCV with Optuna',
            task_type='classification', random_state=random_state
        )
        print('')

    optuna.logging.disable_default_handler()

    res = [mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['LightGBMTuner with Optuna']
    )
    return [res2, roc_lightgbm_tuner_optuna]






# CatBoost K-Fold Cross Validation
def CatBoostKFold(k=k, show_shap=True, perform_lime=True):
    from catboost import CatBoostClassifier
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, roc_curve,
        precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    from collections import defaultdict
    import numpy as np
    import os
    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    print('')
    print('■■■ CatBoost kfCV　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(df.iloc[:, -1] > 0) / len(df)), 5)) + ' %)')
    print('')
    print('----- Be patient. It might take a little while. -----')
    print('')

    # X = df.iloc[:, :-1].values
    # y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]

    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for foldwise-averaged metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # ROC/PR plotting data
    fprs, tprs, precisions, recalls = [], [], [], []
    # confusion rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    # Feature importances per fold
    feature_importances = defaultdict(list)

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X, y), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = CatBoostClassifier(random_state=random_state, verbose=False)
        model.fit(X_train, y_train)

        y_pred      = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # per-fold metrics
        auc_fold  = roc_auc_score(y_test, y_pred_prob)
        acc_fold  = accuracy_score(y_test, y_pred)
        prec_fold = precision_score(y_test, y_pred, zero_division=0)
        rec_fold  = recall_score(y_test, y_pred)
        f1_fold   = f1_score(y_test, y_pred)
        ap_fold   = average_precision_score(y_test, y_pred_prob)
        spec_fold = tn / (tn + fp)

        aucs .append(auc_fold)
        accs .append(acc_fold)
        precs.append(prec_fold)
        recs .append(rec_fold)
        f1s .append(f1_fold)
        specs.append(spec_fold)
        aps  .append(ap_fold)

        # ROC curve data for this fold
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall curve data for this fold
        p_vals, r_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions.append(p_vals); recalls.append(r_vals)

        print(f"Fold {fold_idx:>2}: "
              f"AUC={auc_fold:.4f}, Acc={acc_fold:.4f}, "
              f"Prec={prec_fold:.4f}, Rec={rec_fold:.4f}, F1={f1_fold:.4f}")

        # collect feature importances
        for i, imp in enumerate(model.feature_importances_):
            feature_importances[feature_names[i]].append(imp)

    # -----------------Calculate foldwise-averaged scores-----------------
    mean_auc   = np.mean(aucs)
    mean_acc   = np.mean(accs)
    mean_prec  = np.mean(precs)
    mean_rec   = np.mean(recs)
    mean_f1    = np.mean(f1s)
    mean_spec  = np.mean(specs)
    mean_pr_auc= np.mean(aps)
    mean_cind  = mean_auc

    # foldwise-averaged confusion matrix
    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # display foldwise-averaged table
    display_fold_averages(
        k, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "CatBoost"
    )

    # -----------------Plot foldwise-averaged confusion heatmap-----------------
    print('\n□ Foldwise-averaged Confusion Matrix (%):')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(macro_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'],
                yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Foldwise-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    hmname = os.path.join(path_figure, 'catboost_confusion_heatmap_kfCV.png')
    plt.savefig(hmname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr    = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr    = np.mean(tprs_interp, axis=0)
    rocname     = os.path.join(path_figure, 'catboost_roc_curve_kfCV.png')
    roc_catboost = np.array([mean_fpr, mean_tpr, mean_auc, 'Catboost'], dtype=object)

    print('\n□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'catboost_prerec_curve_kfCV.png')

    print('\n□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {k} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # -----------------Retrain on full data & save model-----------------
    model_full = CatBoostClassifier(random_state=random_state, verbose=False)
    model_full.fit(X, y)
    os.makedirs(path_model, exist_ok=True)
    with open(os.path.join(path_model, 'catboost_kfCV_full.pkl'), 'wb') as f:
        pickle.dump(model_full, f)

    # -----------------Aggregate & save feature importances-----------------
    avg_imp = {f: np.mean(imps) for f, imps in feature_importances.items()}
    sorted_imp = dict(sorted(avg_imp.items(), key=lambda x: x[1], reverse=True))
    fi_df = pd.DataFrame({'Feature': list(sorted_imp.keys()),
                          'Importance': list(sorted_imp.values())})
    fi_df['Abs'] = fi_df.Importance.abs()
    fi_df = fi_df.sort_values('Abs', ascending=False).drop('Abs', axis=1)
    fi_df.to_excel(os.path.join(path_table, 'catboost_feature_importances_kfCV.xlsx'), index=False)

    # -----------------Plot top-10 feature importances-----------------
    fig, ax = plt.subplots(figsize=(10, 6))
    top10 = fi_df.head(10)
    ax.barh(top10.Feature, top10.Importance)
    ax.set_xlabel('Average Importance')
    ax.set_title(f'Top 10 Feature Importances in {k}-fold CV of CatBoost')
    ax.invert_yaxis()
    # no aspect enforcement for importances
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'catboost_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # SHAP & LIME (unchanged)
    model_holdout = CatBoostClassifier(random_state=random_state, verbose=False)
    model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'catboost_shap_values_holdout',
            'catboost_shap_values_holdout',
            'CatBoost', task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'catboost_lime_explanation_holdout',
            'catboost_lime_feature_contributions_holdout',
            'CatBoost', task_type='classification', random_state=random_state
        )
        print('')

    # -----------------Prepare and return results-----------------
    res = [mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]
    res2 = pd.DataFrame([res],
                        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
                        index=['CatBoost'])

    return [res2, roc_catboost]








# Yggdrasil Decision Forest (K-Fold CV版, truly bullet-proof importance parsing with foldwise-averaged metrics)
def YggdrasilDecisionForestKFold(
    n_splits=k,
    show_shap=False,
    perform_lime=True,
    importance_key_preference=None,
    random_state=random_state
):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        roc_curve, precision_recall_curve, confusion_matrix, average_precision_score
    )
    import ydf
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    print('')
    print('■■■ Yggdrasil Decision Forest kfCV (Foldwise averaged)　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(df.iloc[:, -1] > 0) / len(df)), 5)) + ' %)')
    print('')

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]
    df[label_column] = df[label_column].astype(int)

    # Use StratifiedKFold instead of KFold for balanced splits
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    feature_importance_list = []

    default_keys = ['TRAINING_GAIN','SUM_SCORE','NUM_NODES','NUM_AS_ROOT','GAIN_SUM']
    keys_to_try = importance_key_preference or default_keys
    printed_key = False

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df, df[label_column]), 1):
        train_df = df.iloc[train_idx]
        test_df  = df.iloc[test_idx]

        # train model
        model = ydf.GradientBoostedTreesLearner(
            label=label_column,
            num_trees=400, max_depth=10,
            shrinkage=0.02, min_examples=2,
            subsample=0.9, sampling_method="RANDOM"
        ).train(train_df)

        # --- bullet-proof importance parsing ---
        var_imp = model.variable_importances()
        tuple_key = None
        for key in keys_to_try:
            cand = var_imp.get(key)
            if isinstance(cand, (list, tuple)) and cand and isinstance(cand[0], tuple) and len(cand[0]) >= 2:
                tuple_key = key; break
        if tuple_key is None:
            for key, cand in var_imp.items():
                if isinstance(cand, (list, tuple)) and cand and isinstance(cand[0], tuple) and len(cand[0]) >= 2:
                    tuple_key = key; break

        if tuple_key:
            imp_list = var_imp[tuple_key]
            if not printed_key:
                print(f"◆ Using importance key: {tuple_key}")
                printed_key = True
            norm = {}
            for item in imp_list:
                if isinstance(item, (list, tuple)) and len(item) >= 2:
                    if isinstance(item[0], str) and isinstance(item[1], (int, float, np.floating, np.integer)):
                        feat, score = item[0], item[1]
                    elif isinstance(item[1], str) and isinstance(item[0], (int, float, np.floating, np.integer)):
                        feat, score = item[1], item[0]
                    else:
                        continue
                elif hasattr(item, 'feature') and hasattr(item, 'importance'):
                    feat, score = item.feature, item.importance
                else:
                    continue
                norm[str(feat).strip().lower()] = float(score)
            imp_vector = np.array(
                [norm.get(str(f).strip().lower(), 0.0) for f in feature_names],
                dtype=float
            )
        else:
            imp_vector = None
            for key, cand in var_imp.items():
                if isinstance(cand, (list, tuple, np.ndarray)) and len(cand) == len(feature_names):
                    if all(isinstance(v, (int, float, np.floating, np.integer)) for v in cand):
                        imp_vector = np.array(cand, dtype=float)
                        if not printed_key:
                            print(f"◆ Using numeric importances from key: {key}")
                            printed_key = True
                        break
            if imp_vector is None:
                print("◇ Warning: no valid importances found; using zeros.")
                imp_vector = np.zeros(len(feature_names), dtype=float)
        feature_importance_list.append(imp_vector)

        # --- predictions & metrics (batch) ---
        y_true = test_df[label_column].to_numpy()
        X_dict = {col: test_df[col].values for col in feature_names}
        distr = model.predict(X_dict)
        if isinstance(distr, dict):
            probs = np.array(distr['probabilities'])[:, 1]
        elif isinstance(distr, np.ndarray):
            if distr.ndim == 2 and distr.shape[1] >= 2:
                probs = distr[:, 1]
            else:
                probs = distr
        else:
            probs = distr.probabilities_[:, 1]
        bins = (probs >= 0.5).astype(int)

        # per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_true, bins).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # per-fold metrics
        aucs .append(roc_auc_score(y_true, probs))
        accs .append(accuracy_score(y_true, bins))
        precs.append(precision_score(y_true, bins, zero_division=0))
        recs .append(recall_score(y_true, bins))
        f1s .append(f1_score(y_true, bins))
        specs.append(tn / (tn + fp))
        aps .append(average_precision_score(y_true, probs))

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_true, probs)
        fprs.append(fpr); tprs.append(tpr)
        # Precision-Recall curve data
        p_vals, r_vals, _ = precision_recall_curve(y_true, probs)
        precisions.append(p_vals); recalls.append(r_vals)

        print(
            f"Fold {fold_idx:>2} — "
            f"AUC: {aucs[-1]:.4f}, Acc: {accs[-1]:.4f}, "
            f"Prec: {precs[-1]:.4f}, Rec: {recs[-1]:.4f}, F1: {f1s[-1]:.4f}"
        )

    # -----------------Calculate foldwise-averaged scores-----------------
    mean_auc   = np.mean(aucs)
    mean_acc   = np.mean(accs)
    mean_prec  = np.mean(precs)
    mean_rec   = np.mean(recs)
    mean_f1    = np.mean(f1s)
    mean_spec  = np.mean(specs)
    mean_pr_auc= np.mean(aps)
    mean_cind  = mean_auc

    # foldwise-averaged confusion matrix
    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # display foldwise-averaged table
    display_fold_averages(
        n_splits, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "Yggdrasil Decision Forest"
    )

    # -----------------Plot foldwise-averaged confusion heatmap-----------------
    print('')
    print('□ Foldwise-averaged Confusion Matrix (%):')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(macro_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Foldwise-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    hmname = os.path.join(path_figure, 'YDF_confusion_heatmap_kfCV.png')
    plt.savefig(hmname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    rocname = os.path.join(path_figure, 'YDF_roc_curve_kfCV.png')
    roc_array = np.array([mean_fpr, mean_tpr_curve, mean_auc, 'Ygggdrasil Decision Forest'], dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {n_splits} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname = os.path.join(path_figure, 'YDF_prerec_curve_kfCV.png')

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {n_splits} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # retrain on full data
    model_full = ydf.GradientBoostedTreesLearner(label=label_column).train(df)
    os.makedirs(path_model, exist_ok=True)
    model_full.save(os.path.join(path_model, "YDF_KFold_CV_full"))

    # aggregate & save importances
    mean_imp = np.mean(feature_importance_list, axis=0)
    imp_df = pd.DataFrame({'Feature': feature_names, 'YDF_Importance': mean_imp})
    imp_df['Abs'] = imp_df.YDF_Importance.abs()
    imp_df = imp_df.sort_values('Abs', ascending=False).drop('Abs', axis=1)
    imp_df.to_excel(os.path.join(path_table, 'YDF_feature_importances_kfCV.xlsx'), index=False)

    # plot top-10 importances
    top10 = imp_df.head(10)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(top10.Feature, top10.YDF_Importance)
    ax.set_xlabel('Relative Importance')
    ax.set_title(f'Top 10 Feature Importances ({n_splits}-Fold CV)')
    ax.invert_yaxis()
    # ax.set_aspect('equal', adjustable='box')
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'YDF_feature_importances_kfCV.png'))
    plt.show(); plt.close()

    # SHAP & LIME (unchanged)
    if show_shap:
        ShapValueFunction(
            model_made=model_full, df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='YDF_SHAP', excel_name='YDF_SHAP_values',
            method_name='Yggdrasil Decision Forest',
            task_type='classification', model_type='ydf'
        )
    if perform_lime:
        X = df[feature_names].to_numpy()
        y = df[label_column].to_numpy()
        LimeContributionValueFunction(
            model_made=model_full, df_data=df,
            x_train_data=X, x_test_data=X, t_test_data=y,
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='YDF_lime_explanation_full',
            excel_name='YDF_lime_feature_contributions_full',
            method_name='Yggdrasil Decision Forest',
            task_type='classification', random_state=random_state,
            model_type='general'
        )

    # return metrics
    metrics_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['Yggdrasil Decision Forest']
    )
    return [metrics_df, roc_array]









# Yggdrasil Decision Forest (K-Fold CV + Optuna版, foldwise-averaged metrics)
def YggdrasilDecisionForestOptunaKFold(
    n_trials=20,
    n_splits=k,
    show_shap=False,
    perform_lime=True,
    importance_key_preference=None,
    random_state=random_state,
    target='log_loss'
):
    import ydf
    import optuna
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, log_loss, roc_curve,
        precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    from collections import defaultdict
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    print('')
    print(f'■■■ Yggdrasil Decision Forest kfCV with Optuna (Foldwise-Averaged, optimizing {target}, Trial count {n_trials}) ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(df.iloc[:, -1] > 0) / len(df)), 5)) + ' %)')
    print('')
    print('----- Be patient. It takes a while.-----')
    print('')

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]

    # --- Optuna Objective: maximize CV AUC or minimize CV LogLoss over k folds ---
    def objective(trial):
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        all_y, all_preds = [], []

        # suggest hyperparameters
        num_trees    = trial.suggest_int('num_trees', 100, 500)
        max_depth    = trial.suggest_int('max_depth', 4, 16)
        shrinkage    = trial.suggest_float('shrinkage', 0.01, 0.30, log=True)
        min_examples = trial.suggest_int('min_examples', 1, 10)
        subsample    = trial.suggest_float('subsample', 0.60, 1.00)

        for train_idx, test_idx in kf.split(df, df[label_column]):
            train_df = df.iloc[train_idx]
            test_df  = df.iloc[test_idx]

            model = ydf.GradientBoostedTreesLearner(
                label=label_column,
                num_trees=num_trees,
                max_depth=max_depth,
                shrinkage=shrinkage,
                min_examples=min_examples,
                subsample=subsample,
                sampling_method="RANDOM"
            ).train(train_df, verbose=0)

            # batch predict: true probabilities
            X_dict = {col: test_df[col].values for col in feature_names}
            distr = model.predict(X_dict)
            if isinstance(distr, dict):
                probs = np.array(distr['probabilities'])[:, 1]
            elif isinstance(distr, np.ndarray):
                if distr.ndim == 2 and distr.shape[1] >= 2:
                    probs = distr[:, 1]
                else:
                    probs = distr.astype(float)
            else:
                probs = distr.probabilities_[:, 1]

            all_preds.extend(probs.tolist())
            all_y.extend(test_df[label_column].to_numpy().tolist())

        if target == 'auc':
            return roc_auc_score(all_y, all_preds)
        else:
            return -log_loss(all_y, all_preds)

    # Enable Optuna logging
    optuna.logging.enable_default_handler()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    optuna.logging.disable_default_handler()

    print("Best Hyperparameters:")
    for hk, hv in study.best_params.items():
        print(f"  {hk}: {hv}")
    if target == 'auc':
        print(f"Best CV AUC: {study.best_value:.4f}\n")
    else:
        print(f"Best CV LogLoss: {-study.best_value:.4f}\n")

    # --- Final k-fold CV with best params ---
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    all_y_test = []
    all_prob_preds = []
    all_bin_preds  = []
    feature_importance_list = []
    fold_aucs, fold_accs, fold_precs, fold_recs, fold_f1s = [], [], [], [], []
    specs, aps = [], []

    # for ROC/PR plotting
    fprs, tprs, precisions, recalls = [], [], [], []
    # for confusion matrix rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    printed_key = False
    keys_to_try = importance_key_preference or ['TRAINING_GAIN','SUM_SCORE','NUM_NODES','NUM_AS_ROOT','GAIN_SUM']

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df, df[label_column]), 1):
        train_df = df.iloc[train_idx]
        test_df  = df.iloc[test_idx]

        # train with best params
        p = study.best_params
        model = ydf.GradientBoostedTreesLearner(
            label=label_column,
            num_trees=p['num_trees'],
            max_depth=p['max_depth'],
            shrinkage=p['shrinkage'],
            min_examples=p['min_examples'],
            subsample=p['subsample'],
            sampling_method="RANDOM"
        ).train(train_df, verbose=0)

        # --- bullet-proof importance parsing ---
        var_imp = model.variable_importances()
        tuple_key = None
        for key in keys_to_try:
            cand = var_imp.get(key)
            if isinstance(cand, (list, tuple)) and cand and isinstance(cand[0], tuple) and len(cand[0]) >= 2:
                tuple_key = key; break
        if tuple_key is None:
            for key, cand in var_imp.items():
                if isinstance(cand, (list, tuple)) and cand and isinstance(cand[0], tuple) and len(cand[0]) >= 2:
                    tuple_key = key; break
        if tuple_key:
            imp_list = var_imp[tuple_key]
            if not printed_key:
                print(f"◆ Using importance key: {tuple_key}"); printed_key = True
            norm = {}
            for item in imp_list:
                if isinstance(item, (list, tuple)) and len(item) >= 2:
                    if isinstance(item[0], str) and isinstance(item[1], (int, float, np.floating, np.integer)):
                        feat, score = item[0], item[1]
                    elif isinstance(item[1], str) and isinstance(item[0], (int, float, np.floating, np.integer)):
                        feat, score = item[1], item[0]
                    else: continue
                elif hasattr(item, 'feature') and hasattr(item, 'importance'):
                    feat, score = item.feature, item.importance
                else:
                    continue
                norm[str(feat).strip().lower()] = float(score)
            imp_vector = np.array([norm.get(f.strip().lower(),0.0) for f in feature_names], dtype=float)
        else:
            imp_vector = None
            for key, cand in var_imp.items():
                if isinstance(cand, (list, tuple, np.ndarray)) and len(cand)==len(feature_names) and all(isinstance(v,(int,float,np.floating,np.integer)) for v in cand):
                    imp_vector = np.array(cand, dtype=float)
                    if not printed_key:
                        print(f"◆ Using numeric importances from key: {key}"); printed_key=True
                    break
            if imp_vector is None:
                print("◇ Warning: no valid importances found; using zeros.")
                imp_vector = np.zeros(len(feature_names), dtype=float)
        feature_importance_list.append(imp_vector)

        # --- batch predictions & metrics ---
        X_dict = {col: test_df[col].values for col in feature_names}
        distr = model.predict(X_dict)
        if isinstance(distr, dict):
            probs = np.array(distr['probabilities'])[:, 1]
        elif isinstance(distr, np.ndarray):
            if distr.ndim == 2 and distr.shape[1] >= 2:
                probs = distr[:, 1]
            else:
                probs = distr.astype(float)
        else:
            probs = distr.probabilities_[:, 1]
        bins = (probs >= 0.5).astype(int)

        y_true = test_df[label_column].to_numpy()
        all_y_test.extend(y_true.tolist())
        all_prob_preds.extend(probs.tolist())
        all_bin_preds.extend(bins.tolist())

        # confusion rates
        tn, fp, fn, tp = confusion_matrix(y_true, bins).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # per-fold metrics
        fold_aucs .append(roc_auc_score(y_true, probs))
        fold_accs .append(accuracy_score(y_true, bins))
        fold_precs.append(precision_score(y_true, bins, zero_division=0))
        fold_recs .append(recall_score(y_true, bins))
        fold_f1s  .append(f1_score(y_true, bins))
        specs    .append(tn/(tn+fp))
        aps      .append(average_precision_score(y_true, probs))

        # ROC/PR data
        fpr, tpr, _ = roc_curve(y_true, probs)
        fprs.append(fpr); tprs.append(tpr)
        p_vals, r_vals, _ = precision_recall_curve(y_true, probs)
        precisions.append(p_vals); recalls.append(r_vals)

        print(f"Fold {fold_idx:>2}: AUC={fold_aucs[-1]:.4f}, Acc={fold_accs[-1]:.4f}, Prec={fold_precs[-1]:.4f}, Rec={fold_recs[-1]:.4f}, F1={fold_f1s[-1]:.4f}")

    # -----------------Calculate foldwise-averaged scores-----------------
    mean_auc   = np.mean(fold_aucs)
    mean_acc   = np.mean(fold_accs)
    mean_prec  = np.mean(fold_precs)
    mean_rec   = np.mean(fold_recs)
    mean_f1    = np.mean(fold_f1s)
    mean_spec  = np.mean(specs)
    mean_pr_auc= np.mean(aps)
    mean_cind  = mean_auc

    # foldwise-averaged confusion matrix
    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # display foldwise-averaged table
    display_fold_averages(
        n_splits, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "Yggdrasil Decision Forest with Optuna"
    )

    # -----------------Plot foldwise-averaged confusion heatmap-----------------
    print('\n□ Foldwise-averaged Confusion Matrix (%):')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(macro_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'], yticklabels=['True 0','True 1'])
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Foldwise-averaged Confusion Matrix (%)')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    hmname = os.path.join(path_figure, 'YDF_confusion_heatmap_kfCV_optuna.png')
    plt.savefig(hmname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr    = np.linspace(0,1,100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr    = np.mean(tprs_interp, axis=0)
    rocname     = os.path.join(path_figure, 'YDF_roc_curve_kfCV_optuna.png')
    roc_array   = np.array([mean_fpr, mean_tpr, mean_auc, 'Yggdrasil Decision Forest with Optuna'], dtype=object)

    print('\n□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {n_splits} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0,1,100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'YDF_prerec_curve_kfCV_optuna.png')

    print('\n□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {n_splits} Folds')
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right'); plt.savefig(prerecname); plt.show(); plt.close()

    # retrain on full data with best params
    p = study.best_params
    model_full = ydf.GradientBoostedTreesLearner(
        label=label_column,
        num_trees=p['num_trees'],
        max_depth=p['max_depth'],
        shrinkage=p['shrinkage'],
        min_examples=p['min_examples'],
        subsample=p['subsample'],
        sampling_method="RANDOM"
    ).train(df, verbose=0)
    os.makedirs(path_model, exist_ok=True)
    model_full.save(os.path.join(path_model, "YDF_KFoldOptuna_full"))

    # aggregate & save importances
    mean_imp = np.mean(feature_importance_list, axis=0)
    imp_df = pd.DataFrame({
        'Feature': feature_names,
        'YDF_Importance': mean_imp
    })
    imp_df['Abs'] = imp_df.YDF_Importance.abs()
    imp_df = imp_df.sort_values('Abs', ascending=False).drop('Abs', axis=1)
    imp_df.to_excel(os.path.join(path_table, 'YDF_feature_importances_KFoldOptuna.xlsx'), index=False)

    # plot top-10 importances
    fig, ax = plt.subplots(figsize=(10, 6))
    top10 = imp_df.head(10)
    ax.barh(top10.Feature, top10.YDF_Importance)
    ax.set_xlabel('Relative Importance')
    ax.set_title(f'Top 10 Feature Importances ({n_splits}-Fold Optuna)')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'YDF_feature_importances_KFoldOptuna.png'))
    plt.show(); plt.close()

    # SHAP & LIME (unchanged)
    if show_shap:
        ShapValueFunction(
            model_made=model_full, df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='YDF_SHAP_Optuna', excel_name='YDF_SHAP_values_Optuna',
            method_name='Yggdrasil Decision Forest with Optuna',
            task_type='classification', model_type='ydf'
        )
    if perform_lime:
        X = df[feature_names].to_numpy()
        y = df[label_column].to_numpy()
        LimeContributionValueFunction(
            model_made=model_full, df_data=df,
            x_train_data=X, x_test_data=X, t_test_data=y,
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='YDF_lime_explanation_full_optuna',
            excel_name='YDF_lime_feature_contributions_full_optuna',
            method_name='Yggdrasil Decision Forest with Optuna',
            task_type='classification', random_state=random_state,
            model_type='general'
        )

    # return metrics
    metrics_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['Yggdrasil Decision Forest with Optuna']
    )
    return [metrics_df, roc_array]






# TabNet を用いた K-Fold CV版（Foldwise-Averaged Metrics, Normalization）
def TabNetKFoldN(
    n_splits=k,
    perform_lime=True,
    n1=16,
    n2=16,
    step_size=20,
    max_epochs=20,
    batch_size=32,
    show_shap=False,
    random_state=random_state
):
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import contextlib
    import tempfile
    import shutil
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        roc_curve, precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier

    # --- PyTorch TabNet Wrapper with scaling & true validation split ---
    class TorchTabNetWrapper:
        def __init__(self, feature_names, label_column):
            self.feature_names = feature_names
            self.label_column  = label_column
            self.model         = None
            self.scaler        = None

        def __getstate__(self):
            state = self.__dict__.copy()
            if getattr(self, 'model', None) is not None:
                tmp = tempfile.mkdtemp()
                base = os.path.join(tmp, "model")
                self.model.save_model(base)
                with open(base + ".zip", "rb") as f:
                    state['model_bytes'] = f.read()
                shutil.rmtree(tmp)
                del state['model']
            return state

        def __setstate__(self, state):
            self.__dict__.update(state)
            if 'model_bytes' in state:
                tmp = tempfile.mkdtemp()
                base = os.path.join(tmp, "model")
                with open(base + ".zip", "wb") as f:
                    f.write(state['model_bytes'])
                self.model = TabNetClassifier(
                    n_d=n1, n_a=n2, n_steps=5, gamma=1.5,
                    lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
                    optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=2e-2),
                    scheduler_params={"step_size": step_size, "gamma": 0.9},
                    scheduler_fn=torch.optim.lr_scheduler.StepLR,
                    mask_type='entmax'
                )
                self.model.load_model(base)
                shutil.rmtree(tmp)

        def train(self, train_df):
            X_full = train_df[self.feature_names].values.astype(np.float32)
            y_full = train_df[self.label_column].values.astype(np.int64)
            X_tr, X_val, y_tr, y_val = train_test_split(
                X_full, y_full,
                test_size=0.2,
                stratify=y_full,
                random_state=random_state
            )
            self.scaler = MinMaxScaler().fit(X_tr)
            X_tr = self.scaler.transform(X_tr)
            X_val = self.scaler.transform(X_val)
            self.model = TabNetClassifier(
                n_d=n1, n_a=n2, n_steps=5, gamma=1.5,
                lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=2e-2),
                scheduler_params={"step_size": step_size, "gamma": 0.9},
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                mask_type='entmax'
            )
            with open(os.devnull, "w") as f, \
                 contextlib.redirect_stdout(f), \
                 contextlib.redirect_stderr(f):
                self.model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs,
                    patience=10,
                    batch_size=min(batch_size, X_tr.shape[0]),
                    virtual_batch_size=min(batch_size, X_tr.shape[0]),
                    num_workers=0,
                    drop_last=False
                )
            return self

        def predict_proba(self, data):
            if isinstance(data, pd.DataFrame):
                X = data[self.feature_names].values.astype(np.float32)
            else:
                arr = np.array(data, dtype=np.float32)
                X = np.atleast_2d(arr)
            X = self.scaler.transform(X)
            return self.model.predict_proba(X)

        def predict(self, data):
            proba = self.predict_proba(data)
            return (proba[:,1] >= 0.5).astype(int)

        def variable_importances(self):
            imp = self.model.feature_importances_
            return [(name, float(imp[i])) for i, name in enumerate(self.feature_names)]

        def save_model(self, file_name):
            if self.model is None:
                raise ValueError("No model to save. Train first.")
            os.makedirs(os.path.dirname(file_name), exist_ok=True)
            self.model.save_model(file_name)

    # ————————————————————————————————————————————————
    # 1) 準備
    print('')
    print('■■■ TabNet (PyTorch) with Normalization kfCV ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(df.iloc[:, -1] > 0) / len(df)), 5)) + ' %)')
    print('')

    if gpu_available:
        using_device = "GPU " + gpu_name
    else:
        using_device = "CPU"
    print(f"Using device: {using_device}\n")

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    splits = list(skf.split(df, df[label_column]))


    # merge tiny folds (既存の処理)
    cleaned_splits = []
    for idx, (tr_idx, te_idx) in enumerate(splits):
        if idx == 0:
            cleaned_splits.append((tr_idx, te_idx))
            continue
        p_tr, p_te = cleaned_splits[-1]
        if len(tr_idx)<2 or len(te_idx)<2:
            merged_te = np.concatenate([p_te, te_idx])
            merged_tr = np.setdiff1d(np.arange(len(df)), merged_te)
            cleaned_splits[-1] = (merged_tr, merged_te)
        else:
            cleaned_splits.append((tr_idx, te_idx))
    splits = cleaned_splits
    n_effective = len(splits)

    # Lists for foldwise-averaged metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    imps_list = []
    printed_key = False

    # 2) CV ループ
    for fold_no, (tr_idx, te_idx) in enumerate(splits, 1):
        train_df = df.iloc[tr_idx].reset_index(drop=True)
        test_df  = df.iloc[te_idx].reset_index(drop=True)

        mdl = TorchTabNetWrapper(feature_names, label_column).train(train_df)

        imp_pairs = mdl.variable_importances()
        if not printed_key:
            print("◆ Using importance key: TABNET_IMPORTANCE")
            printed_key = True
        norm = {k.lower():v for k,v in imp_pairs}
        imps_list.append(np.array([norm.get(f.lower(),0) for f in feature_names]))

        proba     = mdl.predict_proba(test_df)[:,1]
        preds_bin = (proba>=0.5).astype(int)
        y_true    = test_df[label_column].to_numpy()

        # per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_true, preds_bin).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # per-fold metrics
        auc_fold  = roc_auc_score(y_true, proba)
        acc_fold  = accuracy_score(y_true, preds_bin)
        prec_fold = precision_score(y_true, preds_bin, zero_division=0)
        rec_fold  = recall_score(y_true, preds_bin)
        f1_fold   = f1_score(y_true, preds_bin)
        ap_fold   = average_precision_score(y_true, proba)
        spec_fold = tn / (tn + fp)

        aucs .append(auc_fold)
        accs .append(acc_fold)
        precs.append(prec_fold)
        recs .append(rec_fold)
        f1s .append(f1_fold)
        specs.append(spec_fold)
        aps  .append(ap_fold)

        # ROC curve for this fold
        fpr, tpr, _ = roc_curve(y_true, proba)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall for this fold
        p_vals, r_vals, _ = precision_recall_curve(y_true, proba)
        precisions.append(p_vals); recalls.append(r_vals)

        print(f"Fold {fold_no:>2} — "
              f"AUC: {auc_fold:.4f}, Acc: {acc_fold:.4f}, "
              f"Prec: {prec_fold:.4f}, Rec: {rec_fold:.4f}, F1: {f1_fold:.4f}")


    # -----------------Calculate foldwise-averaged scores-----------------
    mean_auc   = np.mean(aucs)
    mean_acc   = np.mean(accs)
    mean_prec  = np.mean(precs)
    mean_rec   = np.mean(recs)
    mean_f1    = np.mean(f1s)
    mean_spec  = np.mean(specs)
    mean_pr_auc= np.mean(aps)
    mean_cind  = mean_auc

    # foldwise-averaged confusion matrix
    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # display foldwise-averaged table
    display_fold_averages(
        n_effective, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "TabNet with Normalization"
    )


    # -----------------Plot foldwise-averaged confusion heatmap-----------------
    print('\n□ Foldwise-averaged Confusion Matrix (%):')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(macro_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'],
                yticklabels=['True 0','True 1'])
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Foldwise-averaged Confusion Matrix (%)')
    plt.savefig(os.path.join(path_figure, 'TabNet_confusion_heatmap_kfCV.png'))
    plt.show(); plt.close()


    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr    = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr    = np.mean(tprs_interp, axis=0)
    rocname     = os.path.join(path_figure, 'TabNet_roc_curve_kfCV.png')
    roc_tabnet  = np.array([mean_fpr, mean_tpr, mean_auc, 'TabNet normalized'], dtype=object)

    print('\n□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {n_effective} Folds')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()


    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'TabNet_prerec_curve_kfCV.png')

    print('\n□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {n_effective} Folds')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # -----------------Retrain full model & save-----------------
    full_mdl = TorchTabNetWrapper(feature_names, label_column).train(df)
    full_mdl.save_model(os.path.join(path_model, "TabNet_normalized_full_model_kFoldCV"))

    # -----------------Aggregate & save feature importances-----------------
    mean_imp = np.mean(imps_list, axis=0)
    imp_df = pd.DataFrame({
        'Feature': feature_names,
        'TabNet_Importance': mean_imp
    })
    imp_df['Abs'] = imp_df.TabNet_Importance.abs()
    imp_df = imp_df.sort_values('Abs', ascending=False).drop('Abs', axis=1)
    imp_df.to_excel(os.path.join(path_table, 'TabNet_normalized_feature_importances_KFold_CV.xlsx'), index=False)

    # -----------------Plot top-10 feature importances-----------------
    fig, ax = plt.subplots(figsize=(10, 6))
    top10 = imp_df.head(10)
    ax.barh(top10.Feature, top10.TabNet_Importance)
    ax.set_xlabel('Average Importance')
    ax.set_title(f'Top 10 Feature Importances ({n_effective}-Fold CV of TabNet)')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'TabNet_normalized_feature_importances_KFold_CV.png'))
    plt.show(); plt.close()

    # SHAP & LIME (unchanged)
    if show_shap:
        print('')
        ShapValueFunction(
            model_made=full_mdl,
            df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x,
            size_y_value=size_y,
            path_figure_info=path_figure,
            path_table_info=path_table,
            figure_name='TabNet_SHAP_normalized',
            excel_name='TabNet_SHAP_values_normalized',
            method_name='TabNet normalized',
            task_type='classification',
            model_type='TabNet'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_made=full_mdl,
            df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            t_test_data=df[label_column].to_numpy(),
            size_x_value=size_x,
            size_y_value=size_y,
            path_figure_info=path_figure,
            path_table_info=path_table,
            figure_name='TabNet_normalized_lime_explanation_full',
            excel_name='TabNet_normalized_lime_feature_contributions_full',
            method_name='TabNet normazlied',
            task_type='classification',
            random_state=random_state,
            model_type='TabNet'
        )

    # -----------------Prepare and return results-----------------
    metrics_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['TabNet normalized']
    )
    return [metrics_df, roc_tabnet]








# TabNet を用いた K-Fold CV版（Foldwise-Averaged Metrics, Standardization）
def TabNetKFoldS(
    n_splits=k,
    perform_lime=True,
    n1=16,
    n2=16,
    step_size=20,
    max_epochs=20,
    batch_size=32,
    show_shap=False,
    random_state=random_state
):
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import contextlib
    import tempfile
    import shutil
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import (
        roc_auc_score, accuracy_score,
        precision_score, recall_score, f1_score,
        roc_curve, precision_recall_curve, confusion_matrix,
        average_precision_score
    )
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier

    # --- PyTorch TabNet Wrapper with standardization & true validation split ---
    class TorchTabNetWrapper:
        def __init__(self, feature_names, label_column):
            self.feature_names = feature_names
            self.label_column  = label_column
            self.model         = None
            self.scaler        = None

        def __getstate__(self):
            state = self.__dict__.copy()
            if getattr(self, 'model', None) is not None:
                tmp = tempfile.mkdtemp()
                base = os.path.join(tmp, "model")
                self.model.save_model(base)
                with open(base + ".zip", "rb") as f:
                    state['model_bytes'] = f.read()
                shutil.rmtree(tmp)
                del state['model']
            return state

        def __setstate__(self, state):
            self.__dict__.update(state)
            if 'model_bytes' in state:
                tmp = tempfile.mkdtemp()
                base = os.path.join(tmp, "model")
                with open(base + ".zip", "wb") as f:
                    f.write(state['model_bytes'])
                self.model = TabNetClassifier(
                    n_d=n1, n_a=n2, n_steps=5, gamma=1.5,
                    lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
                    optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=2e-2),
                    scheduler_params={"step_size": step_size, "gamma": 0.9},
                    scheduler_fn=torch.optim.lr_scheduler.StepLR,
                    mask_type='entmax'
                )
                self.model.load_model(base)
                shutil.rmtree(tmp)

        def train(self, train_df):
            X_full = train_df[self.feature_names].values.astype(np.float32)
            y_full = train_df[self.label_column].values.astype(np.int64)
            X_tr, X_val, y_tr, y_val = train_test_split(
                X_full, y_full,
                test_size=0.2,
                stratify=y_full,
                random_state=random_state
            )
            self.scaler = StandardScaler().fit(X_tr)
            X_tr = self.scaler.transform(X_tr)
            X_val = self.scaler.transform(X_val)
            self.model = TabNetClassifier(
                n_d=n1, n_a=n2, n_steps=5, gamma=1.5,
                lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=2e-2),
                scheduler_params={"step_size": step_size, "gamma": 0.9},
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                mask_type='entmax'
            )
            with open(os.devnull, "w") as f, \
                 contextlib.redirect_stdout(f), \
                 contextlib.redirect_stderr(f):
                self.model.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs,
                    patience=10,
                    batch_size=min(batch_size, X_tr.shape[0]),
                    virtual_batch_size=min(batch_size, X_tr.shape[0]),
                    num_workers=0,
                    drop_last=False
                )
            return self

        def predict_proba(self, data):
            if isinstance(data, pd.DataFrame):
                X = data[self.feature_names].values.astype(np.float32)
            else:
                arr = np.array(data, dtype=np.float32)
                X = np.atleast_2d(arr)
            X = self.scaler.transform(X)
            return self.model.predict_proba(X)

        def predict(self, data):
            proba = self.predict_proba(data)
            return (proba[:,1] >= 0.5).astype(int)

        def variable_importances(self):
            imp = self.model.feature_importances_
            return [(name, float(imp[i])) for i, name in enumerate(self.feature_names)]

        def save_model(self, file_name):
            if self.model is None:
                raise ValueError("No model to save. Train first.")
            os.makedirs(os.path.dirname(file_name), exist_ok=True)
            self.model.save_model(file_name)

    # ————————————————————————————————————————————————
    # 1) 準備
    print('')
    print('■■■ TabNet (PyTorch) with Standardization kfCV ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(df.iloc[:, -1] > 0) / len(df)), 5)) + ' %)')
    print('')
    if gpu_available:
        using_device = "GPU " + gpu_name
    else:
        using_device = "CPU"
    print(f"Using device: {using_device}\n")

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    splits = list(skf.split(df, df[label_column]))

    # merge tiny folds (既存の処理)
    cleaned_splits = []
    for idx, (tr_idx, te_idx) in enumerate(splits):
        if idx == 0:
            cleaned_splits.append((tr_idx, te_idx))
            continue
        p_tr, p_te = cleaned_splits[-1]
        if len(tr_idx)<2 or len(te_idx)<2:
            merged_te = np.concatenate([p_te, te_idx])
            merged_tr = np.setdiff1d(np.arange(len(df)), merged_te)
            cleaned_splits[-1] = (merged_tr, merged_te)
        else:
            cleaned_splits.append((tr_idx, te_idx))
    splits = cleaned_splits
    n_effective = len(splits)

    # Lists for foldwise-averaged metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs, precisions, recalls = [], [], [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []
    imps_list = []
    printed_key = False

    # 2) CV ループ
    for fold_no, (tr_idx, te_idx) in enumerate(splits, 1):
        train_df = df.iloc[tr_idx].reset_index(drop=True)
        test_df  = df.iloc[te_idx].reset_index(drop=True)

        mdl = TorchTabNetWrapper(feature_names, label_column).train(train_df)

        imp_pairs = mdl.variable_importances()
        if not printed_key:
            print("◆ Using importance key: TABNET_IMPORTANCE")
            printed_key = True
        norm = {k.lower():v for k,v in imp_pairs}
        imps_list.append(np.array([norm.get(f.lower(),0) for f in feature_names]))

        proba     = mdl.predict_proba(test_df)[:,1]
        preds_bin = (proba>=0.5).astype(int)
        y_true    = test_df[label_column].to_numpy()

        # per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_true, preds_bin).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn/total)
        fp_rates.append(fp/total)
        fn_rates.append(fn/total)
        tp_rates.append(tp/total)

        # per-fold metrics
        auc_fold  = roc_auc_score(y_true, proba)
        acc_fold  = accuracy_score(y_true, preds_bin)
        prec_fold = precision_score(y_true, preds_bin, zero_division=0)
        rec_fold  = recall_score(y_true, preds_bin)
        f1_fold   = f1_score(y_true, preds_bin)
        ap_fold   = average_precision_score(y_true, proba)
        spec_fold = tn / (tn + fp)

        aucs .append(auc_fold)
        accs .append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s .append(f1_fold)
        specs.append(spec_fold)
        aps  .append(ap_fold)

        # ROC curve for this fold
        fpr, tpr, _ = roc_curve(y_true, proba)
        fprs.append(fpr); tprs.append(tpr)

        # Precision-Recall for this fold
        p_vals, r_vals, _ = precision_recall_curve(y_true, proba)
        precisions.append(p_vals); recalls.append(r_vals)

        print(f"Fold {fold_no:>2} — "
              f"AUC: {auc_fold:.4f}, Acc: {acc_fold:.4f}, "
              f"Prec: {prec_fold:.4f}, Rec: {rec_fold:.4f}, F1: {f1_fold:.4f}")

    # -----------------Calculate foldwise-averaged scores-----------------
    mean_auc   = np.mean(aucs)
    mean_acc   = np.mean(accs)
    mean_prec  = np.mean(precs)
    mean_rec   = np.mean(recs)
    mean_f1    = np.mean(f1s)
    mean_spec  = np.mean(specs)
    mean_pr_auc= np.mean(aps)
    mean_cind  = mean_auc

    # foldwise-averaged confusion matrix
    macro_tn = np.mean(tn_rates)
    macro_fp = np.mean(fp_rates)
    macro_fn = np.mean(fn_rates)
    macro_tp = np.mean(tp_rates)
    macro_cm = np.array([[macro_tn, macro_fn],
                         [macro_fp, macro_tp]]) * 100

    # display foldwise-averaged table
    display_fold_averages(
        n_effective, df, macro_cm,
        macro_tp, macro_fp, macro_fn, macro_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "TabNet standardizated"
    )

    # -----------------Plot foldwise-averaged confusion heatmap-----------------
    print('\n□ Foldwise-averaged Confusion Matrix (%):')
    plt.figure(figsize=(size_x, size_y))
    sns.heatmap(macro_cm, annot=True, fmt='.2f', cbar=False,
                xticklabels=['Pred 0','Pred 1'],
                yticklabels=['True 0','True 1'])
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('Predicted label'); plt.ylabel('True label')
    plt.title('Foldwise-averaged Confusion Matrix (%)')
    plt.savefig(os.path.join(path_figure, 'TabNet_standardized_confusion_heatmap_kfCV.png'))
    plt.show(); plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr    = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr    = np.mean(tprs_interp, axis=0)
    rocname     = os.path.join(path_figure, 'TabNet_standardized_roc_curve_kfCV.png')
    roc_tabnet  = np.array([mean_fpr, mean_tpr, mean_auc, 'TabNet standardized'], dtype=object)

    print('\n□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-averaged over {n_effective} Folds')
    plt.legend(loc='lower right')
    plt.savefig(rocname); plt.show(); plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall       = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls)):
        idx = np.argsort(recalls[i])
        precisions_interp.append(np.interp(mean_recall, recalls[i][idx], precisions[i][idx]))
    mean_pr_curve = np.mean(precisions_interp, axis=0)
    prerecname    = os.path.join(path_figure, 'TabNet_standardized_prerec_curve_kfCV.png')

    print('\n□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve, label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls)):
        plt.plot(recalls[i], precisions[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    ax = plt.gca(); ax.set_aspect('equal', adjustable='box')
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-averaged over {n_effective} Folds')
    plt.legend(loc='lower right')
    plt.savefig(prerecname); plt.show(); plt.close()

    # -----------------Retrain full model & save-----------------
    full_mdl = TorchTabNetWrapper(feature_names, label_column).train(df)
    full_mdl.save_model(os.path.join(path_model, "TabNet_standardized_full_model"))

    # -----------------Aggregate & save feature importances-----------------
    mean_imp = np.mean(imps_list, axis=0)
    imp_df = pd.DataFrame({
        'Feature': feature_names,
        'TabNet_Importance': mean_imp
    })
    imp_df['Abs'] = imp_df.TabNet_Importance.abs()
    imp_df = imp_df.sort_values('Abs', ascending=False).drop('Abs', axis=1)
    imp_df.to_excel(os.path.join(path_table, 'TabNet_standardized_feature_importances_KFold_CV.xlsx'), index=False)

    # -----------------Plot top-10 feature importances-----------------
    fig, ax = plt.subplots(figsize=(10, 6))
    top10 = imp_df.head(10)
    ax.barh(top10.Feature, top10.TabNet_Importance)
    ax.set_xlabel('Average Importance')
    ax.set_title(f'Top 10 Feature Importances ({n_effective}-Fold CV of TabNet)')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig(os.path.join(path_figure, 'TabNet_standardized_feature_importances_KFold_CV.png'))
    plt.show(); plt.close()

    # SHAP & LIME (unchanged)
    if show_shap:
        print('')
        ShapValueFunction(
            model_made=full_mdl,
            df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x,
            size_y_value=size_y,
            path_figure_info=path_figure,
            path_table_info=path_table,
            figure_name='TabNet_SHAP_standardized',
            excel_name='TabNet_SHAP_values_standardized',
            method_name='TabNet standardized',
            task_type='classification',
            model_type='TabNet'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_made=full_mdl,
            df_data=df,
            x_train_data=df[feature_names].to_numpy(),
            x_test_data=df[feature_names].to_numpy(),
            t_test_data=df[label_column].to_numpy(),
            size_x_value=size_x,
            size_y_value=size_y,
            path_figure_info=path_figure,
            path_table_info=path_table,
            figure_name='TabNet_standardized_lime_explanation_full',
            excel_name='TabNet_standardized_lime_feature_contributions_full',
            method_name='TabNet standardized',
            task_type='classification',
            random_state=random_state,
            model_type='TabNet'
        )

    # -----------------Prepare and return results-----------------
    metrics_df = pd.DataFrame(
        [[mean_auc, mean_acc, mean_prec, mean_rec, mean_f1]],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=['TabNet standardized']
    )
    return [metrics_df, roc_tabnet]







# With true early stopping mechanism by validation
def TabNetOptunaKFoldN(
     n_splits=k,
     show_shap=False,
     perform_lime=True,
     max_epochs=100,
     batch_size=32,
     random_state=random_state,
     n_trials=20,     # Number of Optuna trials
     target='log_loss'     # 'auc' or 'log_loss'
 ):
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import contextlib
    import tempfile
    import shutil
    from datetime import datetime
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import (
        roc_auc_score,
        accuracy_score,
        precision_score,
        recall_score,
        f1_score,
        roc_curve,
        auc,
        precision_recall_curve,
        average_precision_score,
        log_loss,
        confusion_matrix
    )
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier
    import optuna
    import warnings
    # suppress absolutely all warnings
    warnings.filterwarnings("ignore")
    optuna.logging.set_verbosity(optuna.logging.WARNING)


    # ────── PERFORMING K-FOLD CV WITH OPTUNA TO FIND OUT BEST PARAMS ──────
    print('')
    print(f'■■■ TabNet (PyTorch) {n_splits}-fold Cross Validation with Optuna (Optimizing {target}, Trial count {n_trials})　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if gpu_available:
        using_device = "GPU " + gpu_name
    else:
        using_device = "CPU"
    print(f"Using device: {using_device}")
    print('\n----- Be patient. It will take several tens of minutes. -----\n')
    if using_device == "CPU":
        print('Warning: Since CPU is selected, it will take many hours. GPU is highly recommended.\n')

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]

    # — prepare outer folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    raw_splits = list(skf.split(df, df[label_column]))
    cleaned_splits = []
    for idx, (train_idx, test_idx) in enumerate(raw_splits):
        if idx == 0 or (len(train_idx) >= 2 and len(test_idx) >= 2):
            cleaned_splits.append((train_idx, test_idx))
        else:
            prev_tr, prev_te = cleaned_splits[-1]
            merged_te = np.concatenate([prev_te, test_idx])
            merged_tr = np.setdiff1d(np.arange(len(df)), merged_te)
            cleaned_splits[-1] = (merged_tr, merged_te)
    splits = cleaned_splits

    trial_history = []

    def objective(trial):
        params = {
            'n_d': trial.suggest_int('n_d', 8, 64),
            'n_a': trial.suggest_int('n_a', 8, 64),
            'n_steps': trial.suggest_int('n_steps', 3, 10),
            'gamma': trial.suggest_float('gamma', 1.0, 2.0),
            'lambda_sparse': trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-2),
            'momentum': trial.suggest_float('momentum', 0.0, 0.9),
            'clip_value': trial.suggest_float('clip_value', 1.0, 5.0),
            'optimizer_fn': torch.optim.Adam,
            'optimizer_params': {
                'lr': trial.suggest_float('lr', 1e-3, 1e-1, log=True),
                'weight_decay': trial.suggest_float('wd', 1e-6, 1e-2, log=True)
            },
            'scheduler_fn': torch.optim.lr_scheduler.StepLR,
            'scheduler_params': {
                'step_size': trial.suggest_int('step_size', 10, 50),
                'gamma': trial.suggest_float('scheduler_gamma', 0.5, 0.99)
            },
            'mask_type': 'entmax',
            'device_name': device
        }

        aucs, loglosses = [], []
        for tr_idx, te_idx in splits:
            if len(tr_idx) < 2:
                continue

            X_tr = df.iloc[tr_idx][feature_names].values.astype(np.float32)
            y_tr = df.iloc[tr_idx][label_column].values.astype(np.int64)
            X_te = df.iloc[te_idx][feature_names].values.astype(np.float32)
            y_te = df.iloc[te_idx][label_column].values.astype(np.int64)

            scaler = MinMaxScaler()
            X_tr = scaler.fit_transform(X_tr)
            X_te = scaler.transform(X_te)

            X_fit, X_val, y_fit, y_val = train_test_split(
                X_tr, y_tr, test_size=0.1, stratify=y_tr, random_state=random_state
            )

            bs_eff = min(batch_size, X_fit.shape[0])
            if bs_eff < 2:
                continue

            mdl = TabNetClassifier(**params)
            with open(os.devnull, "w") as f, \
                contextlib.redirect_stdout(f), \
                contextlib.redirect_stderr(f):
                mdl.fit(
                    X_fit, y_fit,
                    eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs,
                    patience=10,
                    batch_size=bs_eff,
                    virtual_batch_size=bs_eff,
                    num_workers=0,
                    drop_last=True
                )

            prob = mdl.predict_proba(X_te)[:, 1]
            aucs.append(roc_auc_score(y_te, prob))
            loglosses.append(log_loss(y_te, prob))

        mean_auc_value = float(np.mean(aucs)) if aucs else 0.0
        mean_logloss_value = float(np.mean(loglosses)) if loglosses else 0.0

        rec = {
            'trial': trial.number,
            'auc': mean_auc_value,
            'log_loss': mean_logloss_value,
            'n_d': params['n_d'],
            'n_a': params['n_a'],
            'n_steps': params['n_steps'],
            'gamma': params['gamma'],
            'lambda_sparse': params['lambda_sparse'],
            'momentum': params['momentum'],
            'clip_value': params['clip_value'],
            'lr': params['optimizer_params']['lr'],
            'weight_decay': params['optimizer_params']['weight_decay'],
            'step_size': params['scheduler_params']['step_size'],
            'scheduler_gamma': params['scheduler_params']['gamma'],
        }
        trial_history.append(rec)
        return (mean_auc_value if target=='auc' else -mean_logloss_value)

    def print_callback(study, trial):
        t = datetime.now().strftime("%Y-%m-%d %H:%M:%S, %f")[:-3]
        best = study.best_trial
        mname = 'AUC' if target=='auc' else '-LogLoss'
        print(f"[I {t}] Trial {trial.number} finished with {mname}={trial.value:.6f}; params={trial.params}. Best#{best.number} {mname}={best.value:.6f}.")

    overall_start = datetime.now()
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])
    df_history = pd.DataFrame(trial_history).sort_values('trial').reset_index(drop=True)
    df_history.to_excel(os.path.join(path_table, 'tabnet_normalized_optuna_hyperparameters.xlsx'), index=False)
    print(f"Exported {len(df_history)} trials.")

    elapsed = datetime.now() - overall_start
    hrs, rem = divmod(int(elapsed.total_seconds()), 3600)
    mins, secs = divmod(rem, 60)
    print(f"\n◆ Total optimization time: {hrs}h {mins}m {secs}s\n")
    print(f"◆ Best trial {'AUC' if target=='auc' else 'LogLoss'}: {study.best_value:.4f}")
    print(f"◆ Best params: {study.best_params}\n")

    tuned = study.best_params
    tuned_params = {
        'n_d': tuned['n_d'],'n_a': tuned['n_a'],'n_steps': tuned['n_steps'],
        'gamma': tuned['gamma'],'lambda_sparse': tuned['lambda_sparse'],
        'momentum': tuned['momentum'],'clip_value': tuned['clip_value'],
        'optimizer_fn': torch.optim.Adam,
        'optimizer_params': {'lr': tuned['lr'],'weight_decay': tuned['wd']},
        'scheduler_fn': torch.optim.lr_scheduler.StepLR,
        'scheduler_params': {'step_size': tuned['step_size'],'gamma': tuned['scheduler_gamma']},
        'mask_type': 'entmax','device_name': device
    }

    class TorchTabNetWrapper:
        def __init__(self, features, label, params):
            self.feature_names = features
            self.label_column  = label
            self.tabnet_params = params
            self.model = None
            self.scaler = None

        def train(self, train_df):
            X = train_df[self.feature_names].values.astype(np.float32)
            y = train_df[self.label_column].values.astype(np.int64)
            self.scaler = MinMaxScaler().fit(X)
            X = self.scaler.transform(X)
            X_fit, X_val, y_fit, y_val = train_test_split(
                X, y, test_size=0.1, stratify=y, random_state=random_state
            )
            bs = min(batch_size, X_fit.shape[0])
            self.model = TabNetClassifier(**self.tabnet_params)
            with open(os.devnull, "w") as f, \
                contextlib.redirect_stdout(f), \
                contextlib.redirect_stderr(f):
                self.model.fit(
                    X_fit, y_fit, eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs, patience=10,
                    batch_size=bs, virtual_batch_size=bs,
                    num_workers=0, drop_last=True
                )
            return self

        def predict(self, df_):
            X = df_[self.feature_names].values.astype(np.float32)
            X = self.scaler.transform(X)
            return self.model.predict_proba(X)

        def variable_importances(self):
            imp = self.model.feature_importances_
            return [(n, float(imp[i])) for i,n in enumerate(self.feature_names)]

        def save_model(self, fname):
            os.makedirs(os.path.dirname(fname), exist_ok=True)
            self.model.save_model(fname)


    def compute_avg_metrics(
        df, splits, feature_names, label_column,
        tuned_params, batch_size, random_state,
        max_epochs, device, threshold=0.5
    ):
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        from sklearn.metrics import (
            confusion_matrix, roc_auc_score, accuracy_score,
            precision_score, recall_score, f1_score,
            roc_curve, auc, precision_recall_curve,
            average_precision_score
        )

        mets = {"auc":[], "acc":[], "prec":[], "rec":[], "f1":[]}
        tprs, aucs_list = [], []
        pr_interp_list, ap_list = [], []

        tn_rates, fp_rates, fn_rates, tp_rates = [],[],[],[]
        mean_fpr    = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        for tr_idx, te_idx in splits:
            tr_df = df.iloc[tr_idx].reset_index(drop=True)
            te_df = df.iloc[te_idx].reset_index(drop=True)
            mdl = TorchTabNetWrapper(feature_names, label_column, tuned_params).train(tr_df)

            y_true = te_df[label_column].to_numpy()
            probs  = mdl.predict(te_df)[:,1]
            preds  = (probs>=threshold).astype(int)

            tn, fp, fn, tp = confusion_matrix(y_true,preds).ravel()
            total = tn+fp+fn+tp
            tn_rates.append(tn/total); fp_rates.append(fp/total)
            fn_rates.append(fn/total); tp_rates.append(tp/total)

            mets["auc"].append(roc_auc_score(y_true,probs))
            mets["acc"].append(accuracy_score(y_true,preds))
            mets["prec"].append(precision_score(y_true,preds,zero_division=0))
            mets["rec"].append(recall_score(y_true,preds))
            mets["f1"].append(f1_score(y_true,preds))

            fpr, tpr, _ = roc_curve(y_true,probs)
            interp_tpr = np.interp(mean_fpr, fpr, tpr); interp_tpr[0]=0.0
            tprs.append(interp_tpr); aucs_list.append(auc(fpr,tpr))

            prec, rec, _ = precision_recall_curve(y_true,probs)
            # interpolate onto uniform recall grid
            idx = np.argsort(rec)
            pr_interp = np.interp(mean_recall, rec[idx], prec[idx])
            pr_interp_list.append(pr_interp)
            ap_list.append(average_precision_score(y_true,probs))

        # confusion matrix
        fold_tn = np.mean(tn_rates); fold_fp = np.mean(fp_rates)
        fold_fn = np.mean(fn_rates); fold_tp = np.mean(tp_rates)
        macro_cm = np.array([[fold_tn, fold_fn],[fold_fp, fold_tp]])*100
        mean_spec = fold_tn/(fold_tn+fold_fp) if (fold_tn+fold_fp)>0 else 0.0

        mean_auc   = np.mean(mets["auc"])
        mean_acc   = np.mean(mets["acc"])
        mean_prec  = np.mean(mets["prec"])
        mean_rec   = np.mean(mets["rec"])
        mean_f1    = np.mean(mets["f1"])
        mean_pr_auc= np.mean(ap_list)

        display_fold_averages(
            len(splits), df, macro_cm,
            fold_tp, fold_fp, fold_fn, fold_tn,
            mean_acc, mean_prec, mean_rec, mean_f1,
            mean_spec, mean_auc, mean_auc, mean_pr_auc,
            "TabNet normalized with Optuna"
        )

        # foldwise ROC
        mean_tpr = np.mean(tprs,axis=0)
        roc_label = "TabNet normalized with Optuna"
        roc_arr = np.array([mean_fpr, mean_tpr, mean_auc, roc_label],dtype=object)

        print('\n□ Foldwise-averaged ROC curve:')
        plt.figure()
        plt.plot(mean_fpr,mean_tpr,label=f'Foldwise-averaged ROC (AUC={mean_auc:.4f})')
        for tr in tprs: plt.plot(mean_fpr,tr,alpha=0.3)
        plt.plot([0,1],[0,1],'--',alpha=0.5)
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
        plt.title('TabNet Normalized foldwise-averaged ROC Curve')
        plt.legend(loc='lower right')
        plt.gca().set_aspect('equal',adjustable='box')
        plt.savefig(os.path.join(path_figure,roc_label+'.png')); plt.show(); plt.close()

        # foldwise PR
        mean_pr = np.mean(pr_interp_list,axis=0)
        pr_label="TabNet_normalized_pr_curve_kfCV_optuna"
        print('\n□ Foldwise-averaged Precision-Recall curve:')
        plt.figure()
        plt.plot(mean_recall,mean_pr,label=f'Foldwise-averaged PRC (AUC={mean_pr_auc:.4f})')
        for pr in pr_interp_list: plt.plot(mean_recall,pr,alpha=0.3)
        plt.plot([0,1],[0,1],'--',alpha=0.5)
        plt.xlabel('Recall'); plt.ylabel('Precision')
        plt.title('TabNet Normalized foldwise-averaged Precision-Recall Curve')
        plt.legend(loc='lower right')
        plt.gca().set_aspect('equal',adjustable='box')
        plt.savefig(os.path.join(path_figure,pr_label+'.png')); plt.show(); plt.close()

        res = np.array([mean_auc,mean_acc,mean_prec,mean_rec,mean_f1],dtype=object)
        res2= pd.DataFrame([res],columns=['AUC','Accuracy','Precision','Recall','f1-score'],
                            index=['TabNet normalized with Optuna'])
        return res2, roc_arr

    print('')
    fold_metrics_df, roc_arr = compute_avg_metrics(
        df=df, splits=splits, feature_names=feature_names,
        label_column=label_column, tuned_params=tuned_params,
        batch_size=batch_size, random_state=random_state,
        max_epochs=max_epochs, device=device, threshold=0.5
    )

    full_mdl = TorchTabNetWrapper(feature_names,label_column,tuned_params).train(df)
    full_mdl.save_model(os.path.join(path_model,"TabNet_normalized_full_model_optuna"))

    if show_shap:
        ShapValueFunction(
            model_made=full_mdl, df_data=df,
            x_train_data=df[feature_names].to_numpy(), x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='TabNet_normalized_SHAP_optuna',
            excel_name='TabNet_normalized_SHAP_values_optuna',
            method_name='TabNet normalized with Optuna',
            task_type='classification', model_type='TabNet'
        )

    if perform_lime:
        LimeContributionValueFunction(
            model_made=full_mdl, df_data=df,
            x_train_data=df[feature_names].to_numpy(), x_test_data=df[feature_names].to_numpy(),
            t_test_data=df[label_column].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='TabNet_normalized_lime_explanation_optuna',
            excel_name='TabNet_normalized_lime_feature_contributions_optuna',
            method_name='TabNet normalized with Optuna',
            task_type='classification', random_state=random_state, model_type='TabNet'
        )

    return [fold_metrics_df, roc_arr]







# With true early stopping mechanism by validation
def TabNetOptunaKFoldS(
     n_splits=k,
     show_shap=False,
     perform_lime=True,
     max_epochs=100,
     batch_size=32,
     random_state=random_state,
     n_trials=20,     # Number of Optuna trials
     target='log_loss'     # 'auc' or 'log_loss'
 ):
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import contextlib
    import tempfile
    import shutil
    from datetime import datetime
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import (
        roc_auc_score,
        accuracy_score,
        precision_score,
        recall_score,
        f1_score,
        roc_curve,
        auc,
        precision_recall_curve,
        average_precision_score,
        log_loss,
        confusion_matrix
    )
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier
    import optuna
    import warnings
    # suppress all warnings
    warnings.filterwarnings("ignore")
    optuna.logging.set_verbosity(optuna.logging.WARNING)


    # ────── PERFORMING K-FOLD CV WITH OPTUNA TO FIND BEST PARAMS ──────
    print('')
    print(f'■■■ TabNet (PyTorch) standardized {n_splits}-fold Cross Validation with Optuna (Optimizing {target}, Trial count {n_trials})　■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): '
          + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if gpu_available:
        using_device = "GPU " + gpu_name
    else:
        using_device = "CPU"
    print(f"Using device: {using_device}")
    print('\n----- Be patient. It will take several tens of minutes. -----\n')
    if using_device == "CPU":
        print('Warning: CPU selected, this may take many hours. GPU recommended.\n')

    feature_names = df.columns[:-1]
    label_column  = df.columns[-1]

    # prepare outer folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    raw_splits = list(skf.split(df, df[label_column]))
    cleaned_splits = []
    for idx, (train_idx, test_idx) in enumerate(raw_splits):
        if idx == 0 or (len(train_idx) >= 2 and len(test_idx) >= 2):
            cleaned_splits.append((train_idx, test_idx))
        else:
            prev_tr, prev_te = cleaned_splits[-1]
            merged_te = np.concatenate([prev_te, test_idx])
            merged_tr = np.setdiff1d(np.arange(len(df)), merged_te)
            cleaned_splits[-1] = (merged_tr, merged_te)
    splits = cleaned_splits

    trial_history = []

    def objective(trial):
        params = {
            'n_d': trial.suggest_int('n_d', 8, 64),
            'n_a': trial.suggest_int('n_a', 8, 64),
            'n_steps': trial.suggest_int('n_steps', 3, 10),
            'gamma': trial.suggest_float('gamma', 1.0, 2.0),
            'lambda_sparse': trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-2),
            'momentum': trial.suggest_float('momentum', 0.0, 0.9),
            'clip_value': trial.suggest_float('clip_value', 1.0, 5.0),
            'optimizer_fn': torch.optim.Adam,
            'optimizer_params': {
                'lr': trial.suggest_float('lr', 1e-3, 1e-1, log=True),
                'weight_decay': trial.suggest_float('wd', 1e-6, 1e-2, log=True)
            },
            'scheduler_fn': torch.optim.lr_scheduler.StepLR,
            'scheduler_params': {
                'step_size': trial.suggest_int('step_size', 10, 50),
                'gamma': trial.suggest_float('scheduler_gamma', 0.5, 0.99)
            },
            'mask_type': 'entmax',
            'device_name': device
        }

        aucs, loglosses = [], []
        for tr_idx, te_idx in splits:
            if len(tr_idx) < 2:
                continue

            X_tr = df.iloc[tr_idx][feature_names].values.astype(np.float32)
            y_tr = df.iloc[tr_idx][label_column].values.astype(np.int64)
            X_te = df.iloc[te_idx][feature_names].values.astype(np.float32)
            y_te = df.iloc[te_idx][label_column].values.astype(np.int64)

            scaler = StandardScaler()
            X_tr = scaler.fit_transform(X_tr)
            X_te = scaler.transform(X_te)

            X_fit, X_val, y_fit, y_val = train_test_split(
                X_tr, y_tr, test_size=0.1, stratify=y_tr, random_state=random_state
            )

            bs_eff = min(batch_size, X_fit.shape[0])
            if bs_eff < 2:
                continue

            mdl = TabNetClassifier(**params)
            with open(os.devnull, "w") as f, \
                 contextlib.redirect_stdout(f), \
                 contextlib.redirect_stderr(f):
                mdl.fit(
                    X_fit, y_fit,
                    eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs,
                    patience=10,
                    batch_size=bs_eff,
                    virtual_batch_size=bs_eff,
                    num_workers=0,
                    drop_last=True
                )

            prob = mdl.predict_proba(X_te)[:, 1]
            aucs.append(roc_auc_score(y_te, prob))
            loglosses.append(log_loss(y_te, prob))

        mean_auc_value = float(np.mean(aucs)) if aucs else 0.0
        mean_logloss_value = float(np.mean(loglosses)) if loglosses else 0.0

        trial_history.append({
            'trial': trial.number,
            'auc': mean_auc_value,
            'log_loss': mean_logloss_value,
            'n_d': params['n_d'],
            'n_a': params['n_a'],
            'n_steps': params['n_steps'],
            'gamma': params['gamma'],
            'lambda_sparse': params['lambda_sparse'],
            'momentum': params['momentum'],
            'clip_value': params['clip_value'],
            'lr': params['optimizer_params']['lr'],
            'weight_decay': params['optimizer_params']['weight_decay'],
            'step_size': params['scheduler_params']['step_size'],
            'scheduler_gamma': params['scheduler_params']['gamma'],
        })

        return mean_auc_value if target=='auc' else -mean_logloss_value

    def print_callback(study, trial):
        t = datetime.now().strftime("%Y-%m-%d %H:%M:%S, %f")[:-3]
        best = study.best_trial
        mname = 'AUC' if target=='auc' else '-LogLoss'
        print(f"[I {t}] Trial {trial.number} finished with {mname}={trial.value:.6f}; "
              f"params={trial.params}. Best#{best.number} {mname}={best.value:.6f}.")

    overall_start = datetime.now()
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

    df_history = pd.DataFrame(trial_history).sort_values('trial').reset_index(drop=True)
    df_history.to_excel(os.path.join(path_table, 'tabnet_standard_optuna_hyperparameters.xlsx'), index=False)
    print(f"Exported {len(df_history)} trials to tabnet_standard_optuna_hyperparameters.xlsx")

    elapsed = datetime.now() - overall_start
    hrs, rem = divmod(int(elapsed.total_seconds()), 3600)
    mins, secs = divmod(rem, 60)
    print(f"\n◆ Total optimization time: {hrs}h {mins}m {secs}s\n")
    print(f"◆ Best trial {'AUC' if target=='auc' else 'LogLoss'}: {study.best_value:.4f}")
    print(f"◆ Best params: {study.best_params}\n")

    tuned = study.best_params
    tuned_params = {
        'n_d': tuned['n_d'],
        'n_a': tuned['n_a'],
        'n_steps': tuned['n_steps'],
        'gamma': tuned['gamma'],
        'lambda_sparse': tuned['lambda_sparse'],
        'momentum': tuned['momentum'],
        'clip_value': tuned['clip_value'],
        'optimizer_fn': torch.optim.Adam,
        'optimizer_params': {'lr': tuned['lr'], 'weight_decay': tuned['wd']},
        'scheduler_fn': torch.optim.lr_scheduler.StepLR,
        'scheduler_params': {'step_size': tuned['step_size'], 'gamma': tuned['scheduler_gamma']},
        'mask_type': 'entmax',
        'device_name': device
    }

    class TorchTabNetWrapper:
        def __init__(self, features, label, params):
            self.feature_names = features
            self.label_column  = label
            self.tabnet_params = params
            self.model = None
            self.scaler = None

        def train(self, train_df):
            X = train_df[self.feature_names].values.astype(np.float32)
            y = train_df[self.label_column].values.astype(np.int64)

            self.scaler = StandardScaler().fit(X)
            X = self.scaler.transform(X)

            X_fit, X_val, y_fit, y_val = train_test_split(
                X, y, test_size=0.1, stratify=y, random_state=random_state
            )

            bs = min(batch_size, X_fit.shape[0])
            self.model = TabNetClassifier(**self.tabnet_params)
            with open(os.devnull, "w") as f, \
                 contextlib.redirect_stdout(f), \
                 contextlib.redirect_stderr(f):
                self.model.fit(
                    X_fit, y_fit,
                    eval_set=[(X_val, y_val)],
                    max_epochs=max_epochs,
                    patience=10,
                    batch_size=bs,
                    virtual_batch_size=bs,
                    num_workers=0,
                    drop_last=True
                )
            return self

        def predict(self, df_):
            X = df_[self.feature_names].values.astype(np.float32)
            X = self.scaler.transform(X)
            return self.model.predict_proba(X)

        def variable_importances(self):
            imp = self.model.feature_importances_
            return [(n, float(imp[i])) for i,n in enumerate(self.feature_names)]

        def save_model(self, fname):
            os.makedirs(os.path.dirname(fname), exist_ok=True)
            self.model.save_model(fname)


    def compute_avg_metrics(
        df, splits, feature_names, label_column,
        tuned_params, batch_size, random_state,
        max_epochs, device, threshold=0.5
    ):
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        from sklearn.metrics import (
            confusion_matrix, roc_auc_score, accuracy_score,
            precision_score, recall_score, f1_score,
            roc_curve, auc, precision_recall_curve,
            average_precision_score
        )

        mets = {"auc":[], "acc":[], "prec":[], "rec":[], "f1":[]}
        tprs, aucs_list = [], []
        pr_interp_list, ap_list = [], []

        tn_rates, fp_rates, fn_rates, tp_rates = [],[],[],[]
        mean_fpr    = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        for tr_idx, te_idx in splits:
            tr_df = df.iloc[tr_idx].reset_index(drop=True)
            te_df = df.iloc[te_idx].reset_index(drop=True)
            mdl = TorchTabNetWrapper(feature_names, label_column, tuned_params).train(tr_df)

            y_true = te_df[label_column].to_numpy()
            probs  = mdl.predict(te_df)[:,1]
            preds  = (probs>=threshold).astype(int)

            tn, fp, fn, tp = confusion_matrix(y_true,preds).ravel()
            total = tn+fp+fn+tp
            tn_rates.append(tn/total); fp_rates.append(fp/total)
            fn_rates.append(fn/total); tp_rates.append(tp/total)

            mets["auc"].append(roc_auc_score(y_true,probs))
            mets["acc"].append(accuracy_score(y_true,preds))
            mets["prec"].append(precision_score(y_true,preds,zero_division=0))
            mets["rec"].append(recall_score(y_true,preds))
            mets["f1"].append(f1_score(y_true,preds))

            fpr, tpr, _ = roc_curve(y_true,probs)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs_list.append(auc(fpr,tpr))

            prec, rec, _ = precision_recall_curve(y_true,probs)
            idx = np.argsort(rec)
            pr_interp = np.interp(mean_recall, rec[idx], prec[idx])
            pr_interp_list.append(pr_interp)
            ap_list.append(average_precision_score(y_true,probs))

        fold_tn = np.mean(tn_rates)
        fold_fp = np.mean(fp_rates)
        fold_fn = np.mean(fn_rates)
        fold_tp = np.mean(tp_rates)
        macro_cm = np.array([[fold_tn, fold_fn],[fold_fp, fold_tp]])*100
        mean_spec = fold_tn/(fold_tn+fold_fp) if (fold_tn+fold_fp)>0 else 0.0

        mean_auc   = np.mean(mets["auc"])
        mean_acc   = np.mean(mets["acc"])
        mean_prec  = np.mean(mets["prec"])
        mean_rec   = np.mean(mets["rec"])
        mean_f1    = np.mean(mets["f1"])
        mean_pr_auc= np.mean(ap_list)

        display_fold_averages(
            len(splits), df, macro_cm,
            fold_tp, fold_fp, fold_fn, fold_tn,
            mean_acc, mean_prec, mean_rec, mean_f1,
            mean_spec, mean_auc, mean_auc, mean_pr_auc,
            "TabNet standardized with Optuna"
        )

        # ROC
        mean_tpr = np.mean(tprs,axis=0)
        roc_label = "TabNet standardized with Optuna"
        roc_arr = np.array([mean_fpr, mean_tpr, mean_auc, roc_label],dtype=object)

        print('\n□ Foldwise-averaged ROC curve:')
        plt.figure()
        plt.plot(mean_fpr,mean_tpr,label=f'Foldwise-averaged ROC (AUC={mean_auc:.4f})')
        for tr in tprs: plt.plot(mean_fpr,tr,alpha=0.3)
        plt.plot([0,1],[0,1],'--',alpha=0.5)
        plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
        plt.title('TabNet Standardized foldwise-averaged ROC Curve')
        plt.legend(loc='lower right')
        plt.gca().set_aspect('equal',adjustable='box')
        plt.savefig(os.path.join(path_figure,roc_label+'.png')); plt.show(); plt.close()

        # PR
        mean_pr = np.mean(pr_interp_list,axis=0)
        pr_label="TabNet_standardized_pr_curve_kfCV_optuna"
        print('\n□ Foldwise-averaged Precision-Recall curve:')
        plt.figure()
        plt.plot(mean_recall,mean_pr,label=f'Foldwise-averaged PRC (AUC={mean_pr_auc:.4f})')
        for pr in pr_interp_list: plt.plot(mean_recall,pr,alpha=0.3)
        plt.plot([0,1],[0,1],'--',alpha=0.5)
        plt.xlabel('Recall'); plt.ylabel('Precision')
        plt.title('TabNet Standardized foldwise-averaged Precision-Recall Curve')
        plt.legend(loc='lower right')
        plt.gca().set_aspect('equal',adjustable='box')
        plt.savefig(os.path.join(path_figure,pr_label+'.png')); plt.show(); plt.close()

        res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1],dtype=object)
        res2 = pd.DataFrame([res],columns=['AUC','Accuracy','Precision','Recall','f1-score'],
                            index=['TabNet standardized with Optuna'])
        return res2, roc_arr

    print('')
    fold_metrics_df, roc_arr = compute_avg_metrics(
        df=df, splits=splits, feature_names=feature_names,
        label_column=label_column, tuned_params=tuned_params,
        batch_size=batch_size, random_state=random_state,
        max_epochs=max_epochs, device=device, threshold=0.5
    )

    full_mdl = TorchTabNetWrapper(feature_names,label_column,tuned_params).train(df)
    full_mdl.save_model(os.path.join(path_model,"TabNet_standardized_full_model_optuna"))

    if show_shap:
        ShapValueFunction(
            model_made=full_mdl, df_data=df,
            x_train_data=df[feature_names].to_numpy(), x_test_data=df[feature_names].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='TabNet_standardized_SHAP_optuna',
            excel_name='TabNet_standardized_SHAP_values_optuna',
            method_name='TabNet standardized with Optuna',
            task_type='classification', model_type='TabNet'
        )

    if perform_lime:
        LimeContributionValueFunction(
            model_made=full_mdl, df_data=df,
            x_train_data=df[feature_names].to_numpy(), x_test_data=df[feature_names].to_numpy(),
            t_test_data=df[label_column].to_numpy(),
            size_x_value=size_x, size_y_value=size_y,
            path_figure_info=path_figure, path_table_info=path_table,
            figure_name='TabNet_standardized_lime_explanation_optuna',
            excel_name='TabNet_standardized_lime_feature_contributions_optuna',
            method_name='TabNet standardized with Optuna',
            task_type='classification', random_state=random_state, model_type='TabNet'
        )

    return [fold_metrics_df, roc_arr]






# MLP with selectable scaler
def MLPKFold(k=k, scaler='norm', n1=128, n2=32, max_iter=500, perform_lime=True):
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score, recall_score,
        f1_score, confusion_matrix, roc_curve, precision_recall_curve,
        average_precision_score
    )

    if scaler == 'norm':
        scaling_method = "Normalization"
        scale = 'normalized'
    elif scaler == 'stand':
        scaling_method = "Standardization"
        scale = 'standardized'
    else:
        raise ValueError(f"Unknown scaler: {scaler}. Either 'norm' or 'stand' is available.")

    print('')
    print(f'■■■ Multi-Layer Perceptron kfCV with {scaling_method} ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Choose scaler based on parameter
    def _make_scaler():
        if scaler == 'norm':
            return MinMaxScaler()
        elif scaler == 'stand':
            return StandardScaler()
        else:
            raise ValueError(f"Unknown scaler: {scaler}")

    # Use StratifiedKFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For ROC & PR curves per fold
    fprs, tprs = [], []
    precisions_list, recalls_list = [], []
    # For normalized confusion rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    loss_curves = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler_obj = _make_scaler()
        x_train_scaled = scaler_obj.fit_transform(X_train)
        x_test_scaled  = scaler_obj.transform(X_test)

        model = MLPClassifier(
            hidden_layer_sizes=(n1, n2),
            random_state=random_state,
            max_iter=max_iter
        )
        model.fit(x_train_scaled, y_train)
        loss_curves.append(model.loss_curve_)

        y_pred      = model.predict(x_test_scaled)
        y_pred_prob = model.predict_proba(x_test_scaled)[:, 1]

        # Per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Per-fold metrics
        auc_fold  = roc_auc_score(y_test, y_pred_prob)
        acc_fold  = accuracy_score(y_test, y_pred)
        prec_fold = precision_score(y_test, y_pred, zero_division=0)
        rec_fold  = recall_score(y_test, y_pred)
        f1_fold   = f1_score(y_test, y_pred)
        ap_fold   = average_precision_score(y_test, y_pred_prob)
        spec_fold = tn / (tn + fp)

        aucs.append(auc_fold)
        accs.append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s.append(f1_fold)
        aps.append(ap_fold)
        specs.append(spec_fold)

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)

        # Precision-Recall curve data
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions_list.append(prec_vals)
        recalls_list.append(rec_vals)

    # Compute foldwise-averaged metrics
    mean_auc     = np.mean(aucs)
    mean_acc     = np.mean(accs)
    mean_prec    = np.mean(precs)
    mean_rec     = np.mean(recs)
    mean_f1      = np.mean(f1s)
    mean_spec    = np.mean(specs)
    mean_pr_auc  = np.mean(aps)
    mean_cind    = mean_auc

    # Foldwise-averaged confusion matrix rates
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]]) * 100

    # Display foldwise-averaged binary-classification table
    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "Multi-Layer Perceptron " + scale
    )

    # Plot loss curves
    print('')
    print('◇ Training completed - Loss curves below.')
    max_length = max(len(curve) for curve in loss_curves)
    extended = [curve + [curve[-1]] * (max_length - len(curve))
                for curve in loss_curves]
    plt.figure(figsize=(size_x, size_y))
    for curve in extended:
        plt.plot(curve, color='blue', alpha=0.2)
    plt.plot(np.mean(extended, axis=0),
             color='red', label='Average Loss Curve')
    plt.title('Loss Curves per Fold and Average for MLP ' + scale)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(path_figure, 'MLP_loss_curve_kfCV_' + scale + '.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i])
                      for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_arr = np.array([mean_fpr,
                        mean_tpr_curve,
                        mean_auc,
                        'Multi-Layer Perceptron' + scale],
                       dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve,
             label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1],
             linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'MLP_roc_curve_kfCV_' + scale + '.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls_list)):
        idx_sort = np.argsort(recalls_list[i])
        recall_sorted    = recalls_list[i][idx_sort]
        precision_sorted = precisions_list[i][idx_sort]
        precisions_interp.append(
            np.interp(mean_recall, recall_sorted, precision_sorted)
        )
    mean_pr_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve,
             label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls_list)):
        plt.plot(recalls_list[i], precisions_list[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'MLP_prerec_curve_kfCV_' + scale + '.png'))
    plt.show()
    plt.close()

    # Optionally retrain the model on the full dataset
    scaler_obj = _make_scaler()
    X_scaled = scaler_obj.fit_transform(X)
    model_full = MLPClassifier(
        hidden_layer_sizes=(n1, n2),
        random_state=random_state,
        max_iter=max_iter
    )
    model_full.fit(X_scaled, y)

    # Save the full model
    model_filename = os.path.join(path_model, 'MLP_full_' + scale + '.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out training for LIME
    scaler_obj = _make_scaler()
    x_train_scaled = scaler_obj.fit_transform(x_train)
    x_test_scaled  = scaler_obj.transform(x_test)
    model_holdout = MLPClassifier(
        hidden_layer_sizes=(n1, n2),
        random_state=random_state,
        max_iter=max_iter
    )
    model_holdout.fit(x_train_scaled, t_train)

    print('')
    print('・　Feature importance cannot be extracted from the MLP ' + scale + ' model.')
    print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df,
            x_train_scaled, x_test_scaled, t_test,
            size_x, size_y,
            path_figure, path_table,
            'MLP_lime_explanation_holdout_' + scale,
            'MLP_lime_feature_contributions_holdout_' + scale,
            'MLP ' + scale, task_type='classification',
            random_state=random_state
        )
        print('')

    # Prepare and return results
    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1],
                   dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['MLP ' + scale]
    )

    return [res2, roc_arr]







# Gradient Boosting Machine kfCV
def GradientBoostingKFold(k=k, show_shap=True, perform_lime=True):
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve, average_precision_score
    )
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import pickle

    print('')
    print('■■■ Gradient Boosting Machine kfCV ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For ROC & PR curves per fold
    fprs, tprs = [], []
    precisions_list, recalls_list = [], []
    # For normalized confusion rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    # For loss (training deviance) curves per fold
    loss_curves = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = GradientBoostingClassifier(random_state=random_state)
        model.fit(X_train, y_train)

        # collect training deviance per iteration
        loss_curves.append(list(model.train_score_))

        # Predict classes and probabilities
        y_pred      = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Per-fold metrics
        auc_fold  = roc_auc_score(y_test, y_pred_prob)
        acc_fold  = accuracy_score(y_test, y_pred)
        prec_fold = precision_score(y_test, y_pred, zero_division=0)
        rec_fold  = recall_score(y_test, y_pred)
        f1_fold   = f1_score(y_test, y_pred)
        ap_fold   = average_precision_score(y_test, y_pred_prob)
        spec_fold = tn / (tn + fp) if (tn + fp) > 0 else 0.0

        aucs.append(auc_fold)
        accs.append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s.append(f1_fold)
        aps.append(ap_fold)
        specs.append(spec_fold)

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)

        # Precision-Recall curve data
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions_list.append(prec_vals)
        recalls_list.append(rec_vals)

    # Compute foldwise-averaged metrics
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_pr_auc = np.mean(aps)
    mean_cind   = mean_auc

    # Foldwise-averaged confusion matrix rates
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]]) * 100

    # Display foldwise-averaged binary-classification table
    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "Gradient Boosting Machine"
    )

    # Plot loss curves
    print('')
    print('◇ Training completed - Loss curves below.')
    max_length = max(len(curve) for curve in loss_curves)
    extended = [curve + [curve[-1]] * (max_length - len(curve))
                for curve in loss_curves]
    plt.figure(figsize=(size_x, size_y))
    for curve in extended:
        plt.plot(curve, color='blue', alpha=0.2)
    plt.plot(np.mean(extended, axis=0),
             color='red', label='Average Deviance Curve')
    plt.title('Deviance Curves per Fold and Average for Gradient Boosting')
    plt.xlabel('Iterations')
    plt.ylabel('Deviance')
    plt.legend()
    plt.savefig(os.path.join(path_figure, 'GradientBoosting_loss_curve_kfCV.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i])
                      for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_arr = np.array([mean_fpr,
                        mean_tpr_curve,
                        mean_auc,
                        'Gradient Boosting Machine'],
                       dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve,
             label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1],
             linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'GradientBoosting_roc_curve_kfCV.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls_list)):
        idx_sort = np.argsort(recalls_list[i])
        recall_sorted    = recalls_list[i][idx_sort]
        precision_sorted = precisions_list[i][idx_sort]
        precisions_interp.append(
            np.interp(mean_recall, recall_sorted, precision_sorted)
        )
    mean_pr_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve,
             label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls_list)):
        plt.plot(recalls_list[i], precisions_list[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'GradientBoosting_prerec_curve_kfCV.png'))
    plt.show()
    plt.close()

    # Optionally retrain the model on the full dataset
    model_full = GradientBoostingClassifier(random_state=random_state)
    model_full.fit(X, y)

    # Save the full model
    model_filename = os.path.join(path_model, 'GradientBoosting_kfCV.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out model for SHAP and LIME
    model_holdout = GradientBoostingClassifier(random_state=random_state)
    model_holdout.fit(x_train, t_train)

    if show_shap == True:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'GradientBoosting_shap_values_holdout',
            'GradientBoosting_shap_values_holdout',
            'Gradient Boosting Machine', task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'GradientBoosting_lime_explanation_holdout',
            'GradientBoosting_lime_feature_contributions_holdout',
            'Gradient Boosting Machine', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['Gradient Boosting Machine']
    )

    return [res2, roc_arr]






# AdaBoost Machine kfCV
def AdaBoostKFold(k=k, show_shap=True, perform_lime=True):
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve, average_precision_score
    )
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import pickle

    print('')
    print('■■■ AdaBoost Machine kfCV ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For ROC & PR curves per fold
    fprs, tprs = [], []
    precisions_list, recalls_list = [], []
    # For normalized confusion rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    # For staged-error curves per fold
    error_curves = []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = AdaBoostClassifier(random_state=random_state)
        model.fit(X_train, y_train)

        # collect training error per iteration
        error_curves.append([1 - score for score in model.staged_score(X_train, y_train)])

        # Predict classes and probabilities
        y_pred      = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Per-fold metrics
        auc_fold  = roc_auc_score(y_test, y_pred_prob)
        acc_fold  = accuracy_score(y_test, y_pred)
        prec_fold = precision_score(y_test, y_pred, zero_division=0)
        rec_fold  = recall_score(y_test, y_pred)
        f1_fold   = f1_score(y_test, y_pred)
        ap_fold   = average_precision_score(y_test, y_pred_prob)
        spec_fold = tn / (tn + fp) if (tn + fp) > 0 else 0.0

        aucs.append(auc_fold)
        accs.append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s.append(f1_fold)
        aps.append(ap_fold)
        specs.append(spec_fold)

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)

        # Precision-Recall curve data
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions_list.append(prec_vals)
        recalls_list.append(rec_vals)

    # Compute foldwise-averaged metrics
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_pr_auc = np.mean(aps)
    mean_cind   = mean_auc

    # Foldwise-averaged confusion matrix rates
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]]) * 100

    # Display foldwise-averaged binary-classification table
    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "AdaBoost"
    )

    # Plot error curves
    print('')
    print('◇ Training completed - Error curves below.')
    max_length = max(len(curve) for curve in error_curves)
    extended = [curve + [curve[-1]] * (max_length - len(curve))
                for curve in error_curves]
    plt.figure(figsize=(size_x, size_y))
    for curve in extended:
        plt.plot(curve, color='blue', alpha=0.2)
    plt.plot(np.mean(extended, axis=0),
             color='red', label='Average Training Error Curve')
    plt.title('Training Error per Iteration - AdaBoost')
    plt.xlabel('Iteration')
    plt.ylabel('Error')
    plt.legend()
    plt.savefig(os.path.join(path_figure, 'AdaBoost_error_curve_kfCV.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i])
                      for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_arr = np.array([mean_fpr,
                        mean_tpr_curve,
                        mean_auc,
                        'AdaBoost'],
                       dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve,
             label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1],
             linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'AdaBoost_roc_curve_kfCV.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls_list)):
        idx_sort = np.argsort(recalls_list[i])
        recall_sorted    = recalls_list[i][idx_sort]
        precision_sorted = precisions_list[i][idx_sort]
        precisions_interp.append(
            np.interp(mean_recall, recall_sorted, precision_sorted)
        )
    mean_pr_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve,
             label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls_list)):
        plt.plot(recalls_list[i], precisions_list[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, 'AdaBoost_prerec_curve_kfCV.png'))
    plt.show()
    plt.close()

    # Optionally retrain the model on the full dataset
    model_full = AdaBoostClassifier(random_state=random_state)
    model_full.fit(X, y)

    # Save the full model
    model_filename = os.path.join(path_model, 'AdaBoost_kfCV.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out model for SHAP and LIME
    model_holdout = AdaBoostClassifier(random_state=random_state)
    model_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'AdaBoost_shap_values_holdout',
            'AdaBoost_shap_values_holdout',
            'AdaBoost', task_type='classification', model_type='tree'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'AdaBoost_lime_explanation_holdout',
            'AdaBoost_lime_feature_contributions_holdout',
            'AdaBoost', task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=['AdaBoost']
    )

    return [res2, roc_arr]



# Linear Discriminant Analysis kfCV with selectable scaler
def LinearDiscriminantAnalysisKFold(k=k, scaler='norm', show_shap=True, perform_lime=True):
    # 'norm' = MinMaxScaler, 'stand' = StandardScaler
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve, average_precision_score
    )
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import pickle

    # Choose scaler based on parameter by the value of scaler, either 'norm' or 'stand'
    if scaler == 'norm':
        Scaler = MinMaxScaler
        scale_name = 'normalized'
    elif scaler == 'stand':
        Scaler = StandardScaler
        scale_name = 'standardized'
    else:
        raise ValueError(f"Unknown scale: {scaler}. Use 'norm' or 'stand'.")

    print('')
    print(f'■■■ Linear Discriminant Analysis kfCV ({scale_name}) ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
    print('')

    # Use StratifiedKFold for balanced splits
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Lists for foldwise metrics
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    # For ROC & PR curves per fold
    fprs, tprs = [], []
    precisions_list, recalls_list = [], []
    # For normalized confusion rates
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Scale features
        scaler_method = Scaler()
        X_train_scaled = scaler_method.fit_transform(X_train)
        X_test_scaled  = scaler_method.transform(X_test)

        # Initialize the Linear Discriminant Analysis model
        model = LinearDiscriminantAnalysis()
        model.fit(X_train_scaled, y_train)

        # Predict classes and probabilities
        y_pred      = model.predict(X_test_scaled)
        y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]

        # Per-fold confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Per-fold metrics
        auc_fold  = roc_auc_score(y_test, y_pred_prob)
        acc_fold  = accuracy_score(y_test, y_pred)
        prec_fold = precision_score(y_test, y_pred, zero_division=0)
        rec_fold  = recall_score(y_test, y_pred)
        f1_fold   = f1_score(y_test, y_pred)
        ap_fold   = average_precision_score(y_test, y_pred_prob)
        spec_fold = tn / (tn + fp) if (tn + fp) > 0 else 0.0

        aucs.append(auc_fold)
        accs.append(acc_fold)
        precs.append(prec_fold)
        recs.append(rec_fold)
        f1s.append(f1_fold)
        aps.append(ap_fold)
        specs.append(spec_fold)

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr)
        tprs.append(tpr)

        # Precision-Recall curve data
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions_list.append(prec_vals)
        recalls_list.append(rec_vals)

    # Compute foldwise-averaged metrics
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_pr_auc = np.mean(aps)
    mean_cind   = mean_auc

    # Foldwise-averaged confusion matrix rates
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]]) * 100

    # Display foldwise-averaged binary-classification table
    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        f"Linear Discriminant Analysis ({scale_name})"
    )

    # -----------------Plot foldwise-averaged ROC curve-----------------
    mean_fpr       = np.linspace(0, 1, 100)
    tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i])
                      for i in range(len(fprs))]
    mean_tpr_curve = np.mean(tprs_interp, axis=0)
    roc_arr = np.array([mean_fpr,
                        mean_tpr_curve,
                        mean_auc,
                        f'Linear Discriminant Analysis {scale_name}'],
                       dtype=object)

    print('')
    print('□ Foldwise-averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr_curve,
             label=f'Foldwise-averaged ROC (AUC = {mean_auc:.4f})')
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i], alpha=0.3)
    plt.plot([0, 1], [0, 1],
             linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, f'LDA_roc_curve_kfCV_{scale_name}.png'))
    plt.show()
    plt.close()

    # -----------------Plot foldwise-averaged Precision-Recall curve-----------------
    mean_recall = np.linspace(0, 1, 100)
    precisions_interp = []
    for i in range(len(recalls_list)):
        idx_sort = np.argsort(recalls_list[i])
        recall_sorted    = recalls_list[i][idx_sort]
        precision_sorted = precisions_list[i][idx_sort]
        precisions_interp.append(
            np.interp(mean_recall, recall_sorted, precision_sorted)
        )
    mean_pr_curve = np.mean(precisions_interp, axis=0)

    print('')
    print('□ Foldwise-averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_recall, mean_pr_curve,
             label=f'Foldwise-averaged PRC (AUC = {mean_pr_auc:.4f})')
    for i in range(len(recalls_list)):
        plt.plot(recalls_list[i], precisions_list[i], alpha=0.3)
    plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - Foldwise-Averaged over {k} Folds')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, f'LDA_pr_curve_kfCV_{scale_name}.png'))
    plt.show()
    plt.close()

    # Optionally retrain the model on the full dataset
    scaler_method = Scaler()
    X_scaled = scaler_method.fit_transform(X)
    model_full = LinearDiscriminantAnalysis()
    model_full.fit(X_scaled, y)

    # Save the full model
    model_filename = os.path.join(path_model, f'LDA_kfCV_{scale_name}.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Hold-out model for SHAP and LIME
    scaler_method = Scaler()
    x_train_scaled = scaler_method.fit_transform(x_train)
    x_test_scaled  = scaler_method.transform(x_test)
    model_holdout = LinearDiscriminantAnalysis()
    model_holdout.fit(x_train_scaled, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled,
            size_x, size_y, path_figure, path_table,
            'LDA_shap_values_holdout',
            'LDA_shap_values_holdout',
            'Linear Discriminant Analysis' + scale_name, task_type='classification', model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        LimeContributionValueFunction(
            model_holdout, df, x_train_scaled, x_test_scaled, t_test,
            size_x, size_y, path_figure, path_table,
            'LDA_lime_explanation_holdout',
            'LDA_lime_feature_contributions_holdout',
            'Linear Discriminant Analysis' + scale_name, task_type='classification', random_state=random_state
        )
        print('')

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
        index=[f'Linear Discriminant Analysis {scale_name}']
    )

    return [res2, roc_arr]







def GaussianProcessKFold(
    k=k,
    show_shap=True,
    perform_lime=True,
    scaler='norm'   # 'norm' = MinMaxScaler, 'stand' = StandardScaler, else no scaling
):
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import MinMaxScaler, StandardScaler
    from sklearn.gaussian_process import GaussianProcessClassifier
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import (
        roc_auc_score, accuracy_score, precision_score,
        recall_score, f1_score, confusion_matrix,
        roc_curve, precision_recall_curve, average_precision_score
    )
    import numpy as np
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import pickle

    print('')
    print('■■■ Gaussian Process Classifier kfCV ■■■')
    print('')
    print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
          str(round(100 * (np.count_nonzero(y > 0) / len(y)), 5)) + ' %)')
    print('')

    # Build a Pipeline: scaler + GP
    if scaler == 'norm':
        scaler_step = MinMaxScaler()
        scaler_name = 'normalized'
    elif scaler == 'stand':
        scaler_step = StandardScaler()
        scaler_name = 'standardized'
    else:
        scaler_step = 'passthrough'

    pipeline = Pipeline([
        ('scaler', scaler_step),
        ('gp', GaussianProcessClassifier())
    ])

    # Stratified k-fold
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

    # Storage for metrics and curve data
    aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
    fprs, tprs = [], []
    precisions_list, recalls_list = [], []
    tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Fit pipeline (scale + GP)
        pipeline.fit(X_train, y_train)

        # Predictions
        y_pred      = pipeline.predict(X_test)
        y_pred_prob = pipeline.predict_proba(X_test)[:, 1]

        # Confusion rates
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total = tn + fp + fn + tp
        tn_rates.append(tn / total)
        fp_rates.append(fp / total)
        fn_rates.append(fn / total)
        tp_rates.append(tp / total)

        # Metrics
        aucs.append(roc_auc_score(y_test, y_pred_prob))
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred, zero_division=0))
        recs.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        aps.append(average_precision_score(y_test, y_pred_prob))
        specs.append(tn / (tn + fp) if (tn + fp) > 0 else 0.0)

        # ROC curve points
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprs.append(fpr); tprs.append(tpr)

        # PR curve points
        prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
        precisions_list.append(prec_vals)
        recalls_list.append(rec_vals)

    # Aggregate metrics
    mean_auc    = np.mean(aucs)
    mean_acc    = np.mean(accs)
    mean_prec   = np.mean(precs)
    mean_rec    = np.mean(recs)
    mean_f1     = np.mean(f1s)
    mean_spec   = np.mean(specs)
    mean_pr_auc = np.mean(aps)
    mean_cind   = mean_auc

    # Aggregate confusion matrix
    fold_tn = np.mean(tn_rates)
    fold_fp = np.mean(fp_rates)
    fold_fn = np.mean(fn_rates)
    fold_tp = np.mean(tp_rates)
    fold_cm = np.array([[fold_tn, fold_fn],
                        [fold_fp, fold_tp]]) * 100

    display_fold_averages(
        k, df, fold_cm,
        fold_tp, fold_fp, fold_fn, fold_tn,
        mean_acc, mean_prec, mean_rec, mean_f1,
        mean_spec, mean_auc, mean_cind, mean_pr_auc,
        "Gaussian Process " + scaler_name
    )

    # Prepare averaged ROC curve
    mean_fpr    = np.linspace(0, 1, 100)
    tprs_interp = [np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(fprs))]
    mean_tpr    = np.mean(tprs_interp, axis=0)

    print('')
    print('□ Foldwise‐averaged ROC curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_fpr, mean_tpr, label=f'AUC = {mean_auc:.4f}')
    for fpr, tpr in zip(fprs, tprs):
        plt.plot(fpr, tpr, alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve over {k} folds')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, f'GaussianProcess_roc_curve_kfCV_{scaler_name}.png'))
    plt.show(); plt.close()

    # Prepare averaged PR curve
    mean_rec    = np.linspace(0, 1, 100)
    precs_interp = []
    for precs_i, recs_i in zip(precisions_list, recalls_list):
        idx = np.argsort(recs_i)
        precs_interp.append(np.interp(mean_rec, recs_i[idx], precs_i[idx]))
    mean_pr = np.mean(precs_interp, axis=0)

    print('')
    print('□ Foldwise‐averaged Precision-Recall curve:')
    plt.figure(figsize=(size_x, size_y))
    plt.plot(mean_rec, mean_pr, label=f'PR AUC = {mean_pr_auc:.4f}')
    for recs_i, precs_i in zip(recalls_list, precisions_list):
        plt.plot(recs_i, precs_i, alpha=0.3)
    plt.plot([0,1],[0,1], linestyle='--', alpha=0.5)
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall over {k} folds')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend(loc='lower right')
    plt.savefig(os.path.join(path_figure, f'GaussianProcess_pr_curve_kfCV_{scaler_name}.png'))
    plt.show(); plt.close()

    # Retrain on full data
    pipeline_full = Pipeline([
        ('scaler', scaler_step),
        ('gp', GaussianProcessClassifier())
    ])
    pipeline_full.fit(X, y)
    with open(os.path.join(path_model, f'GaussianProcess_kfCV_{scaler_name}.pkl'), 'wb') as f:
        pickle.dump(pipeline_full, f)

    # Hold‐out for SHAP & LIME: use the same pipeline
    pipeline_holdout = Pipeline([
        ('scaler', scaler_step),
        ('gp', GaussianProcessClassifier())
    ])
    pipeline_holdout.fit(x_train, t_train)

    if show_shap:
        print('')
        ShapValueFunction(
            pipeline_holdout, df, x_train, x_test,
            size_x, size_y, path_figure, path_table,
            'GaussianProcess_shap_values_holdout_' + scaler_name,
            'GaussianProcess_shap_values_holdout_' + scaler_name,
            'Gaussian Process ' + scaler_name,
            task_type='classification',
            model_type='general'
        )
        print('')

    if perform_lime:
        print('')
        # Pass the pipeline to LIME so it scales internally
        LimeContributionValueFunction(
            pipeline_holdout,    # now a Pipeline(scaler + GP)
            df, x_train, x_test, t_test,
            size_x, size_y, path_figure, path_table,
            'GaussianProcess_lime_explanation_holdout_' + scaler_name,
            'GaussianProcess_lime_feature_contributions_holdout_' + scaler_name,
            'Gaussian Process ' + scaler_name,
            task_type='classification',
            random_state=random_state,
            model_type='general'  # Pipeline has a predict_proba method
        )
        print('')

    # Ensure roc_arr is defined
    roc_arr = np.array([
        mean_fpr,
        mean_tpr,
        mean_auc,
        f'Gaussian Process {scaler_name}'
    ], dtype=object)

    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame(
        [res],
        columns=['AUC','Accuracy','Precision','Recall','f1-score'],
        index=[f'Gaussian Process {scaler_name}']
    )

    return [res2, roc_arr]





# # Gaussian Process Classifier kfCV with per‐feature ARD length‐scales
# def GaussianProcessKFold(k=k, scaler='norm', show_shap=True, perform_lime=True):
#     from sklearn.gaussian_process import GaussianProcessClassifier
#     from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
#     from sklearn.model_selection import StratifiedKFold
#     from sklearn.metrics import (
#         roc_auc_score, accuracy_score, precision_score,
#         recall_score, f1_score, confusion_matrix,
#         roc_curve, precision_recall_curve, average_precision_score
#     )
#     from sklearn.preprocessing import MinMaxScaler, StandardScaler
#     import numpy as np
#     import os
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import pickle


#     # Choose scaler based on parameter by the value of scaler, either 'norm' or 'stand'
#     if scaler == 'norm':
#         scale_name = 'normalized'
#     elif scaler == 'stand':
#         scale_name = 'standardized'
#     else:
#         raise ValueError(f"Unknown scale: {scaler}. Use 'norm' or 'stand'.")


#     print('')
#     print(f'■■■ Gaussian Process Classifier kfCV ({scale_name}) ■■■')
#     print('')
#     print('◆ 二値分類　(陽性症例の割合(事前確率): ' +
#           str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %)')
#     print('')
#     print(f'----- Be patient. It takes a few minutes. ------')
#     print('')

#     # Build ARD RBF kernel: one length‐scale per feature
#     n_features = X.shape[1]
#     kernel = C(1.0, (1e-3, 1e3)) * RBF(
#         length_scale=[1.0] * n_features,
#         length_scale_bounds=(1e-2, 1e2)
#     )

#     # Use StratifiedKFold for balanced splits
#     kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)

#     # Lists for foldwise metrics
#     aucs, accs, precs, recs, f1s, specs, aps = [], [], [], [], [], [], []
#     # For ROC & PR curves per fold
#     fprs, tprs = [], []
#     precisions_list, recalls_list = [], []
#     # For normalized confusion rates
#     tn_rates, fp_rates, fn_rates, tp_rates = [], [], [], []

#     for train_index, test_index in kf.split(X, y):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]

#         # --- ここで前処理 ---
#         if scaler == 'norm':
#             scaler_obj = MinMaxScaler()
#         elif scaler == 'stand':
#             scaler_obj = StandardScaler()
#         else:
#             scaler_obj = None

#         if scaler_obj is not None:
#             X_train = scaler_obj.fit_transform(X_train)
#             X_test  = scaler_obj.transform(X_test)

#         # Initialize the Gaussian Process Classifier with ARD kernel
#         model = GaussianProcessClassifier(
#             kernel=kernel,
#             n_restarts_optimizer=5,
#             warm_start=True
#         )
#         model.fit(X_train, y_train)

#         # Predict classes and probabilities
#         y_pred      = model.predict(X_test)
#         y_pred_prob = model.predict_proba(X_test)[:, 1]

#         # Per‐fold confusion rates
#         tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
#         total = tn + fp + fn + tp
#         tn_rates.append(tn / total)
#         fp_rates.append(fp / total)
#         fn_rates.append(fn / total)
#         tp_rates.append(tp / total)

#         # Per‐fold metrics
#         auc_fold  = roc_auc_score(y_test, y_pred_prob)
#         acc_fold  = accuracy_score(y_test, y_pred)
#         prec_fold = precision_score(y_test, y_pred, zero_division=0)
#         rec_fold  = recall_score(y_test, y_pred)
#         f1_fold   = f1_score(y_test, y_pred)
#         ap_fold   = average_precision_score(y_test, y_pred_prob)
#         spec_fold = tn / (tn + fp) if (tn + fp) > 0 else 0.0

#         aucs.append(auc_fold)
#         accs.append(acc_fold)
#         precs.append(prec_fold)
#         recs.append(rec_fold)
#         f1s.append(f1_fold)
#         aps.append(ap_fold)
#         specs.append(spec_fold)

#         # ROC curve data
#         fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
#         fprs.append(fpr)
#         tprs.append(tpr)

#         # Precision‐Recall curve data
#         prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_pred_prob)
#         precisions_list.append(prec_vals)
#         recalls_list.append(rec_vals)

#     # Compute foldwise‐averaged metrics
#     mean_auc    = np.mean(aucs)
#     mean_acc    = np.mean(accs)
#     mean_prec   = np.mean(precs)
#     mean_rec    = np.mean(recs)
#     mean_f1     = np.mean(f1s)
#     mean_spec   = np.mean(specs)
#     mean_pr_auc = np.mean(aps)
#     mean_cind   = mean_auc

#     # Foldwise‐averaged confusion matrix rates
#     fold_tn = np.mean(tn_rates)
#     fold_fp = np.mean(fp_rates)
#     fold_fn = np.mean(fn_rates)
#     fold_tp = np.mean(tp_rates)
#     fold_cm = np.array([[fold_tn, fold_fn],
#                         [fold_fp, fold_tp]]) * 100

#     # Display foldwise‐averaged binary‐classification table
#     display_fold_averages(
#         k, df, fold_cm,
#         fold_tp, fold_fp, fold_fn, fold_tn,
#         mean_acc, mean_prec, mean_rec, mean_f1,
#         mean_spec, mean_auc, mean_cind, mean_pr_auc,
#         "Gaussian Process Classifier (ARD)"
#     )

#     # -----------------Plot foldwise‐averaged ROC curve-----------------
#     mean_fpr       = np.linspace(0, 1, 100)
#     tprs_interp    = [np.interp(mean_fpr, fprs[i], tprs[i])
#                       for i in range(len(fprs))]
#     mean_tpr_curve = np.mean(tprs_interp, axis=0)
#     roc_arr = np.array([mean_fpr,
#                         mean_tpr_curve,
#                         mean_auc,
#                         'GaussianProcess_roc_curve_kfCV.png'],
#                        dtype=object)

#     print('')
#     print('□ Foldwise‐averaged ROC curve:')
#     plt.figure(figsize=(size_x, size_y))
#     plt.plot(mean_fpr, mean_tpr_curve,
#              label=f'Foldwise‐averaged ROC (AUC = {mean_auc:.4f})')
#     for i in range(len(fprs)):
#         plt.plot(fprs[i], tprs[i], alpha=0.3)
#     plt.plot([0, 1], [0, 1],
#              linestyle='--', alpha=0.5)
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title(f'ROC Curve - Foldwise‐Averaged over {k} Folds')
#     ax = plt.gca()
#     ax.set_aspect('equal', adjustable='box')
#     plt.legend(loc='lower right')
#     plt.savefig(os.path.join(path_figure, 'GaussianProcess_roc_curve_kfCV.png'))
#     plt.show()
#     plt.close()

#     # -----------------Plot foldwise‐averaged Precision‐Recall curve-----------------
#     mean_recall = np.linspace(0, 1, 100)
#     precisions_interp = []
#     for i in range(len(recalls_list)):
#         idx_sort = np.argsort(recalls_list[i])
#         recall_sorted    = recalls_list[i][idx_sort]
#         precision_sorted = precisions_list[i][idx_sort]
#         precisions_interp.append(
#             np.interp(mean_recall, recall_sorted, precision_sorted)
#         )
#     mean_pr_curve = np.mean(precisions_interp, axis=0)

#     print('')
#     print('□ Foldwise‐averaged Precision‐Recall curve:')
#     plt.figure(figsize=(size_x, size_y))
#     plt.plot(mean_recall, mean_pr_curve,
#              label=f'Foldwise‐averaged PRC (AUC = {mean_pr_auc:.4f})')
#     for i in range(len(recalls_list)):
#         plt.plot(recalls_list[i], precisions_list[i], alpha=0.3)
#     plt.plot([0, 1], [0, 1], linestyle='--', alpha=0.5)
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.title(f'Precision‐Recall Curve - Foldwise‐Averaged over {k} Folds')
#     ax = plt.gca()
#     ax.set_aspect('equal', adjustable='box')
#     plt.legend(loc='lower right')
#     plt.savefig(os.path.join(path_figure, 'GaussianProcess_pr_curve_kfCV.png'))
#     plt.show()
#     plt.close()

#     # Optionally retrain the model on the full dataset
#     # （必要に応じて同様にスケーリングを挟んでください）
#     model_full = GaussianProcessClassifier(
#         kernel=kernel,
#         n_restarts_optimizer=5,
#         warm_start=True
#     )
#     model_full.fit(X, y)

#     # Save the full model
#     model_filename = os.path.join(path_model, 'GaussianProcess_kfCV.pkl')
#     with open(model_filename, 'wb') as file:
#         pickle.dump(model_full, file)

#     # Hold‐out model for SHAP and LIME
#     model_holdout = GaussianProcessClassifier(
#         kernel=kernel,
#         n_restarts_optimizer=5,
#         warm_start=True
#     )
#     model_holdout.fit(x_train, t_train)

#     if show_shap:
#         print('')
#         ShapValueFunction(
#             model_holdout, df, x_train, x_test,
#             size_x, size_y, path_figure, path_table,
#             'GaussianProcess_shap_values_holdout',
#             'GaussianProcess_shap_values_holdout',
#             'GaussianProcessClassifier', task_type='classification', model_type='kernel'
#         )
#         print('')

#     if perform_lime:
#         print('')
#         LimeContributionValueFunction(
#             model_holdout, df, x_train, x_test, t_test,
#             size_x, size_y, path_figure, path_table,
#             'GaussianProcess_lime_explanation_holdout',
#             'GaussianProcess_lime_feature_contributions_holdout',
#             'GaussianProcessClassifier', task_type='classification', random_state=random_state
#         )
#         print('')

#     res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
#     res2 = pd.DataFrame(
#         [res],
#         columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'],
#         index=['Gaussian Process Classifier']
#     )

#     return [res2, roc_arr]












#----------------------------------  以下はMicro-Averaged Metricsによる評価がされている  ------------------------------------------------


# Decision Tree
def DecisionTreeKFold(k=k, perform_lime=True):
    from sklearn.tree import DecisionTreeClassifier

    print('')
    print('■ Decision Tree')
    print('')


    # KFold cross-validation
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    auc_scores, accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], [], []

    # For ROC and Precision-Recall curve plotting
    all_y_test = []
    all_predictions = []

    # Initialize array to store feature importances
    feature_importances = np.zeros((df.shape[1] - 1,))

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize the Decision Tree model
        model = DecisionTreeClassifier(random_state=random_state)
        model.fit(X_train, y_train)

        # Feature importances are directly available for decision trees
        feature_importances += model.feature_importances_

        # Predict classes and probabilities for evaluation
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]

        # Compute metrics for the current fold
        auc = roc_auc_score(y_test, y_pred_prob)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Append scores
        auc_scores.append(auc)
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        all_y_test.extend(y_test)
        all_predictions.extend(y_pred_prob)

    # Calculate mean scores
    mean_auc = np.mean(auc_scores)
    mean_acc = np.mean(accuracy_scores)
    mean_prec = np.mean(precision_scores)
    mean_rec = np.mean(recall_scores)
    mean_f1 = np.mean(f1_scores)

    # Calculate and print the mean of each metric
    print(f'□ {k}-fold Cross Validation を用いた Decision Tree による2値分類の検定結果')
    print('')
    print(f'Mean AUC: {mean_auc:.4f}')
    print(f'Mean Accuracy: {mean_acc:.4f}')
    print(f'Mean Precision: {mean_prec:.4f}')
    print(f'Mean Recall: {mean_rec:.4f}')
    print(f'Mean F1 Score: {mean_f1:.4f}')
    print('')

    hmname=os.path.join(path_figure, 'decision_tree_confusion_heatmap_kfCV.png')
    rocname=os.path.join(path_figure, 'decision_tree_roc_curve_kfCV.png')
    prerecname=os.path.join(path_figure, 'decision_tree_prerec_curve_kfCV.png')

    roc_decisiontree=showscores(all_y_test, all_predictions, hmname, rocname, prerecname)

    # Average the feature importances over all folds
    feature_importances /= k
    # Get the feature names
    feature_names = df.columns[:-1]

    # Sort the features by importance in descending order
    sorted_idx = np.argsort(feature_importances)[::-1]
    sorted_importance = feature_importances[sorted_idx]
    sorted_features = feature_names[sorted_idx]

    # Export feature importances
    feature_importances_df = pd.DataFrame({'Feature': sorted_features, 'Importance': sorted_importance})
    excel_filename = os.path.join(path_table, 'decision_tree_feature_importances_kfold.xlsx')
    feature_importances_df.to_excel(excel_filename, index=False)

    # Plot only the top 10 features
    top_features = sorted_features[:10]
    top_importances = sorted_importance[:10]

    # Plot only the top 10 features
    top_features = sorted_features[::-1]
    top_importances = sorted_importance[::-1]

    print('')
    print(f'◇ {k}-fold Cross Validation で得られた Decision Tree における Feature Importance')
    print('')
    plt.figure(figsize=(size_x, size_y))
    plt.barh(top_features, top_importances, color='skyblue')
    plt.xlabel('Average Feature Importance')
    plt.title(f'Top 10 Feature Importances in {k}-fold CV of Decision Tree')

    # フォントファイルのパスを確認
    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    # フォントプロパティを設定
    font_prop = fm.FontProperties(fname=font_path)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)

    plt.savefig(os.path.join(path_figure, 'decision_tree_feature_importances_kfCV.png'))
    plt.show()
    plt.close()



    # Optionally retrain the model on the full dataset
    model_full = DecisionTreeClassifier(random_state=random_state)
    model_full.fit(X, y)

    # Save the full model
    model_filename = os.path.join(path_model, 'decision_tree_kfCV.pkl')
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Holtout data
    model_holdout = DecisionTreeClassifier(random_state=random_state)
    model_holdout.fit(x_train, t_train)


    if len(df.columns)<250:

        print('')
        ShapValueFunction(model_holdout, df, x_train, x_test, size_x, size_y, path_figure, path_table, 'decision_treee_shap_values_holdout', 'decision_tree_shap_values_holdout', 'DecisionTree', task_type='classification', model_type='tree')
        print('')

    else:
        print('')
        print('◇ 説明変数の数が250以上あるため、Decision Tree のSHAP値の計算は回避されました。')
        print('')

    if perform_lime == True:
        print('')
        LimeContributionValueFunction(model_holdout, df, x_train, x_test, t_test, size_x, size_y, path_figure, path_table, 'decision_tree_lime_explanation_holdout', 'decision_tree_lime_feature_contributions_holdout', 'DecisionTree', task_type='classification', random_state=random_state)
        print('')


    res = np.array([mean_auc, mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)
    res2 = pd.DataFrame([res], columns=['AUC', 'Accuracy', 'Precision', 'Recall', 'f1-score'], index=['Decision Tree'])

    return [res2, roc_decisiontree]



# k-Nearest Neighbors
def KNearestNeighborsKFoldN(k=k, perform_lime=True):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import MinMaxScaler

    print('')
    print('■ k-Nearest Neighbors normalized')
    print('')


    # Initialize MinMaxScaler
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # KFold cross-validation
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    auc_scores, accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], [], []

    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize the k-Nearest Neighbors model
        model = KNeighborsClassifier()
        model.fit(X_train, y_train)

        # Predict classes for evaluation
        y_pred = model.predict(X_test)
        # k-NN does not directly provide probability estimates for AUC calculation in a consistent way like decision trees,
        # hence it's commented out. If needed, use `predict_proba` method carefully considering k-NN's behavior.
        # y_pred_prob = model.predict_proba(X_test)[:, 1]
        # auc = roc_auc_score(y_test, y_pred_prob)

        # Compute metrics for the current fold
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Append scores (excluding AUC due to the above comment)
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    # Calculate mean scores
    mean_acc = np.mean(accuracy_scores)
    mean_prec = np.mean(precision_scores)
    mean_rec = np.mean(recall_scores)
    mean_f1 = np.mean(f1_scores)

    # Calculate and print the mean of each metric
    print(f'□ {k}-fold Cross Validation を用いた k-Nearest Neighbors normalized による2値分類の検定結果')
    print('')
    print(f'Mean Accuracy: {mean_acc:.4f}')
    print(f'Mean Precision: {mean_prec:.4f}')
    print(f'Mean Recall: {mean_rec:.4f}')
    print(f'Mean F1 Score: {mean_f1:.4f}')
    print('')

    # k-NN does not use feature importances, so related output is removed

    print('')
    print('・ AUC and feature importance cannot be figured out in the k-Nearest Neighbors in a consistent way like other models.')
    print('')

    # Optionally retrain the model on the full dataset
    model_full = KNeighborsClassifier()
    model_full.fit(X_scaled, y)

    # Ensure the path exists
    model_filename = os.path.join(path_model, 'kNN_normalized_full.pkl')
    # Save the full model
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Holdout model
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = KNeighborsClassifier()
    model_holdout.fit(x_train_scaled, t_train)

    if perform_lime == True:
        print('')
        LimeContributionValueFunction(model_holdout, df, x_train_scaled, x_test_scaled, t_test, size_x, size_y, path_figure, path_table, 'kNN_normalized_lime_explanation_holdout', 'kNN_normalized_lime_feature_contributions_holdout', 'k-Nearst Neighbors normalized', task_type='classification', random_state=random_state)
        print('')

    res = np.array([mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)  # Note: AUC is excluded
    res2 = pd.DataFrame([res], columns=['Accuracy', 'Precision', 'Recall', 'f1-score'], index=['k-Nearest Neighbors normalized'])

    return [res2, np.array([np.nan, np.nan, np.nan, np.nan], dtype=object)]




# k-Nearest Neighbors
def KNearestNeighborsKFoldS(k=k, perform_lime=True):
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import StandardScaler

    print('')
    print('■ k-Nearest Neighbors standardized')
    print('')

    # Initialize StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # KFold cross-validation
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    auc_scores, accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], [], []

    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize the k-Nearest Neighbors model
        model = KNeighborsClassifier()
        model.fit(X_train, y_train)

        # Predict classes for evaluation
        y_pred = model.predict(X_test)
        # k-NN does not directly provide probability estimates for AUC calculation in a consistent way like decision trees,
        # hence it's commented out. If needed, use `predict_proba` method carefully considering k-NN's behavior.
        # y_pred_prob = model.predict_proba(X_test)[:, 1]
        # auc = roc_auc_score(y_test, y_pred_prob)

        # Compute metrics for the current fold
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Append scores (excluding AUC due to the above comment)
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    # Calculate mean scores
    mean_acc = np.mean(accuracy_scores)
    mean_prec = np.mean(precision_scores)
    mean_rec = np.mean(recall_scores)
    mean_f1 = np.mean(f1_scores)

    # Calculate and print the mean of each metric
    print(f'□ {k}-fold Cross Validation を用いた k-Nearest Neighbors standardized による2値分類の検定結果')
    print('')
    print(f'Mean Accuracy: {mean_acc:.4f}')
    print(f'Mean Precision: {mean_prec:.4f}')
    print(f'Mean Recall: {mean_rec:.4f}')
    print(f'Mean F1 Score: {mean_f1:.4f}')
    print('')

    # k-NN does not use feature importances, so related output is removed

    print('')
    print('・ AUC and feature importance cannot be figured out in the k-Nearest Neighbors in a consistent way like other models.')
    print('')

    # Optionally retrain the model on the full dataset
    model_full = KNeighborsClassifier()
    model_full.fit(X_scaled, y)

    # Ensure the path exists
    model_filename = os.path.join(path_model, 'kNN_standardized_full.pkl')
    # Save the full model
    with open(model_filename, 'wb') as file:
        pickle.dump(model_full, file)

    # Holdtou model
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    model_holdout = KNeighborsClassifier()
    model_holdout.fit(x_train_scaled, t_train)

    if perform_lime == True:
        print('')
        LimeContributionValueFunction(model_holdout, df, x_train_scaled, x_test_scaled, t_test, size_x, size_y, path_figure, path_table, 'kNN_standardized_lime_explanation_holdout', 'kNN_standardized_lime_feature_contributions_holdout', 'k-Nearst Neighbors standardized', task_type='classification', random_state=random_state)
        print('')


    res = np.array([mean_acc, mean_prec, mean_rec, mean_f1], dtype=object)  # Note: AUC is excluded
    res2 = pd.DataFrame([res], columns=['Accuracy', 'Precision', 'Recall', 'f1-score'], index=['k-Nearest Neighbors standardized'])

    return [res2, np.array([np.nan, np.nan, np.nan, np.nan], dtype=object)]




# ■ 実行コマンド

## 1) 簡易モデル Set (９個の機械学習モデル)

In [ ]:
print('')
print('■ 二値分類の簡易Verson')
print('')
print(f'◆ The sample size of dataset: {n_samples}')
print('')

# 説明変数と目的変数を分離: Pandas形式
X0 = df.iloc[:, :-1]
y0 = df.iloc[:, -1]

# データの大きさ
print('◆　行と列の数')
rows, cols = df.shape
print(f'データの行数(サンプル数) {rows},  データの列数(説明変数＋目的変数の数) {cols}')
if rows >= 5000 or cols >= 5000:
    bigdata = 1
    print('')
    print('◇ データは Big Data のため、Random Forest と LightGBM への Optuna適用 と CatBoost の利用が省かれる。')
elif rows < 5000 and cols < 5000:
    bigdata = 0
    print('')
    print('◇ データは Big Data ではないため、Optuna は適用可能なモデルには適用する。')
print('')

print('')
# データの分割数 k
print(f"◆ Chosen value of k for k-fold cross-validation: {k}")
print('')
print('')

# 目的変数の分布の表示
print('◆　目的変数の分布')
sns.displot(df.iloc[:,-1].dropna())
print('　　　　　陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %')
plt.show()
print('')
print('')


import matplotlib.font_manager as fm
# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)




# データの線形性の確認

# 線形回帰モデルによる決定係数R2
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# (二値分類のため)Logistic回帰モデルによる擬似決定係数R2: NagelkerkeのR2 (Cox-Snellの擬似R2を0〜1にscalingしたもの)
import numpy as np
from sklearn.linear_model import LogisticRegression
def nagelkerke_r2(X, y, **logistic_kwargs):
    """
    X: array-like of shape (n_samples, n_features)
    y: array-like of shape (n_samples,), binary target (0 or 1)
    logistic_kwargs:
        追加の LogisticRegression 引数（solver, C, penalty など）

    Returns
    -------
    float
        Nagelkerke の R²
    """
    # 1) モデルのフィッティング
    model = LogisticRegression(**logistic_kwargs)
    model.fit(X, y)

    # 2) 予測確率（正クラス）の取得
    p = model.predict_proba(X)[:, 1]
    # log(0) を防ぐため eps 分だけクリップ
    eps = np.finfo(float).eps
    p = np.clip(p, eps, 1 - eps)

    # 3) フルモデルの対数尤度
    ll_model = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # 4) ヌルモデル（切片のみ、常に y の平均を予測）の対数尤度
    p_null = np.clip(np.mean(y), eps, 1 - eps)
    ll_null = np.sum(y * np.log(p_null) + (1 - y) * np.log(1 - p_null))

    # 5) Cox–Snell の R²
    n = len(y)
    r2_cs = 1 - np.exp((ll_null - ll_model) * 2 / n)

    # 6) Nagelkerke の R²
    r2_nagelkerke = r2_cs / (1 - np.exp(2 * ll_null / n))

    return r2_nagelkerke

pseudo_r2 = nagelkerke_r2(
    X, y,
    penalty='l2',     # リッジ（L2）正則化
    C=1.0,            # デフォルト付近からスタート
    solver='lbfgs',   # 中小規模向き
    max_iter=500,     # 収束用に少し余裕を
    tol=1e-6,          # 精度重視
    class_weight='balanced' # Imbalanced Dataにも対応
)


print(f'◆　説明変数と目的変数の間の決定係数R2')
print('')
print(f'◇ Nagelkerke の擬似R² (標準設定) = {pseudo_r2:.4f}')
print(f'  擬似決係数R²の見方　0.2未満: 弱い説明力、0.2〜0.5: 中程度の説明力、0.5以上: 強い説明力')
print('')
print(f'◇ 線形回帰による決定係数R² = {r2:.4f}')
print('   決定係数R²の見方　 0.3未満: 線形性なし、0.3〜0.5: 弱い線形性、0.5〜0.7: 中程度の線形性、0.7〜0.9: 強い線形性、0.9以上: 完全に線形')
print('')
print('  (擬似決係数R² > 0.5　または  決定係数R² > 0.7 のとき、線形性があると見なせる。二値分類では擬似R²の方が妥当)')
print('')
if pseudo_r2 >= 0.5:
    print('')
    print('◇ データの線形性が見られるため、SVM の kernel と XGBoost の booster を線形にする。')
    linear = 1
elif pseudo_r2 < 0.5:
    print('')
    print('◇ データの線形性はそれほど見られないため、線形モデルの優先は行わない。')
    linear = 0
print('')
print(f'linear = {linear:.1f}')
print('')
print('')




# 残差プロット
print('◆　線形モデルによる目的変数の予測値の残差(誤差の)プロット')
y_pred = model.predict(X)
residuals = y - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('予測値', fontproperties=font_prop)
plt.ylabel('残差', fontproperties=font_prop)
plt.title('予測値と残差のプロット', fontproperties=font_prop)
plt.show()
print('')
print('')



# 連続変数の場合の線形性の判定

# # RESET（Regression Specification Error Test）
# from statsmodels.stats.diagnostic import linear_reset
# import statsmodels.api as sm

# ols = sm.OLS(y, sm.add_constant(X)).fit()
# print(linear_reset(ols, power=2))

# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression

# # 線形モデル vs 多項式モデル（2次項・3次項を加えたモデル）
# poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
# score_lin = cross_val_score(LinearRegression(), X, y, cv=5, scoring="r2").mean()
# score_poly2 = cross_val_score(poly2, X, y, cv=5, scoring="r2").mean()

# print("線形モデル R²:", score_lin)
# print("2次多項式モデル R²:", score_poly2)




# 定数列を除外する関数
def remove_constant_columns(df):
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        print(f'以下の定数列を除外します: {constant_columns}')
        df = df.drop(columns=constant_columns)
    else:
        print('定数列は存在しません。')
    return df

# 欠損値を確認・除外する関数
def check_missing_values(df_X, df_y):
    missing_X = df_X.isnull().sum()
    missing_y = df_y.isnull().sum()
    if missing_X.sum() > 0:
        print('説明変数に欠損値が含まれています。欠損値を除外します。')
        df_X = df_X.dropna()
    if missing_y > 0:
        print('目的変数に欠損値が含まれています。欠損値を除外します。')
        df_y = df_y.dropna()
    return df_X, df_y


print('◆ 説明変数と目的変数のPearsonの相関係数: 線形な相関のみ')
# 定数列の除外
X0 = remove_constant_columns(X0)

# 目的変数が定数か確認
if y0.nunique() <= 1:
    raise ValueError('目的変数 y0 が定数列です。相関係数を計算できません。')

# 欠損値の除外
X0, y0 = check_missing_values(X0, y0)


# 各説明変数と目的変数との相関係数を計算
correlations = {}
for column in X0.columns:
    correlations[column] = X0[column].corr(y0)

# 相関係数をSeriesに変換
correlation_series = pd.Series(correlations)

# 絶対値の降順にソートし、元の符号を保持
top_10_correlations = correlation_series.reindex(correlation_series.abs().sort_values(ascending=False).index).head(10)

# Top 10の相関係数を昇べき順にソート
sorted_top_10_correlations = top_10_correlations.sort_values(ascending=True)

# 色を決定
colors = ['skyblue' if val > 0 else 'lightcoral' for val in correlation_series.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(correlation_series.index, correlation_series.values, color=colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()

print('□ 青は正の相関、赤は負の相関。　1に近いほど強い正の相関、-1に近いほど強い負の相関。0は線形での相関無し')
print('□　　相関係数の絶対値Cの目安： C > 0.7 強い相関、0.4〜0.7 中程度の相関、0.2〜0.4 弱い相関　　C < 0.2 相関無し')
print('')
print('')


print('◇ 説明変数と目的変数のPearsonの相関係数: 相関係数のTop 10 のみ')

# Top 10 correlation colors
top_10_colors = ['skyblue' if val > 0 else 'lightcoral' for val in sorted_top_10_correlations.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(sorted_top_10_correlations.index, sorted_top_10_correlations.values, color=top_10_colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Top 10 Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()
print('')
print('')
print('')


# XiCorの相関係数のプロット関数
print('◆ 説明変数と目的変数のChatterjee`sの相関係数: 非線形な相関も含む (参考値)')

def xicor(x, y, ties=True):
    np.random.seed(42)
    n = len(x)
    order = np.argsort(x)
    ranked_y = np.argsort(np.argsort(y[order], kind='mergesort'))

    if ties:
        l = np.argsort(np.argsort(y[order], kind='max'))
        r = ranked_y.copy()
        return 1 - n * np.sum(np.abs(r[1:] - r[:-1])) / (2 * np.sum(l * (n - l)))
    else:
        return 1 - 3 * np.sum(np.abs(ranked_y[1:] - ranked_y[:-1])) / (n**2 - 1)

def plot_xicor(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor(df)
print('')


print('◇ 説明変数と目的変数のChatterjee`sの相関係数: 相関係数のTop 10 のみ')
def plot_xicor10(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)[-10:]  # 上位10の特徴量を選択
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(10, 6))  # グラフのサイズを調整
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('Top 10 XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor10(df)
print('')
print('\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\thttps://arxiv.org/abs/1909.10140 参照')
print('')


from datetime import datetime
start_time = datetime.now()
formatted_start_time2 = start_time.strftime('%Y%m%d%H%M')


# 出力ファイル
filename0 = os.path.basename(path_table)
filename = os.path.splitext(filename0)[0]
filepath = path_table + '/AllEvaluationResultsFor' + filename + str(formatted_start_time2) + '.xlsx'
filepath2 = path_table + '/AllAUCResultsFor' + filename + str(formatted_start_time2) + '.csv'
print('◆　計算結果の出力先')
print(filepath)
print('')
print('')


# 現在時刻（計算開始時）
from datetime import datetime
start_time = datetime.now()
formatted_start_time = start_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
formatted_start_time2 = start_time.strftime('%Y年%m月%d日 %H時%M分%')
print(f'◆ Present Time (Start): {formatted_start_time}')
print('')



# 各機械学習モデルの実行

# データがそれほど大きくないとき
if bigdata ==0:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    # print('')
    # res_logistic_S = LogisticKFoldS()
    # print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    # print('')
    # res_lasso_S = LassoKFoldS(0.01)
    # print('')
    # print('')
    # res_lasso_optuna_N = LassoKFoldOptunaN()
    # print('')
    # print('')
    # res_lasso_optuna_S = LassoKFoldOptunaS()
    # print('')
    # print('')
    # res_ridge_N = RidgeKFoldN(1.0)
    # print('')
    # print('')
    # res_ridge_S = RidgeKFoldS(1.0)
    # print('')
    # print('')
    # res_ridge_optuna_N = RidgeKFoldOptunaN()
    # print('')
    # print('')
    # res_ridge_optuna_S = RidgeKFoldOptunaS()
    # print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    # print('')
    # res_SVM_S = SVMKFoldS()
    # print('')
    # print('')
    # res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    # print('')
    # print('')
    # res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    # print('')
    # print('')
    # res_GaussianN = GaussianProcessKFold(scaler='norm')
    # print('')
    # print('')
    # res_GaussianS = GaussianProcessKFold(scaler='stand')
    # print('')
    # print('')
    # res_Gradient_Boosting = GradientBoostingKFold()
    # print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    # print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    # print('')
    # res_RandomForest_optuna = RandomForestOptunaKFold()
    # print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    # print('')
    # res_XGBoost_optuna = XGBoostOptunaKFold()
    # print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    # print('')
    # res_LightGBM_optuna = LightGBMOptunaKFold()
    # print('')
    # print('')
    # res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    # print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    # print('')
    # res_YDF = YggdrasilDecisionForestKFold()
    # print('')
    # print('')
    # res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    # print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    # print('')
    # res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    # print('')
    # print('')
    # res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    # print('')
    # res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')

    res_list = [
                res_linear,
                res_logistic_N,
                # res_logistic_S,
                res_lasso_N,
                # res_lasso_S,
                # res_lasso_optuna_N,
                # res_lasso_optuna_S,
                # res_ridge_N,
                # res_ridge_S,
                # res_ridge_optuna_N,
                # res_ridge_optuna_S,
                res_SVM_N,
                # res_SVM_S,
                # res_LDAn,
                # res_LDAs,
                # res_GaussianN,
                # res_GaussianS,
                # res_Gradient_Boosting,
                res_Adaboost,
                res_RandomForest,
                # res_RandomForest_optuna,
                res_XGBoost,
                # res_XGBoost_optuna,
                res_LightGBM,
                # res_LightGBM_optuna,
                # res_LightGBMTuner_optuna,
                # res_CatBoost,
                # res_YDF,
                # res_YDF_optuna,
                res_MLPn,
                # res_MLPs,
                # res_TabNetN,
                # res_TabNetS,
                # res_TabNetN_optuna,
                # res_TabNetS_optuna,
]

    results = pd.concat([item[0] for item in res_list], axis=0)
    res_auc = [item[1] for item in res_list]
    results.to_excel(filepath)


# データが大きいとき
else:


    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    # print('')
    # res_logistic_S = LogisticKFoldS()
    # print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    # print('')
    # res_lasso_S = LassoKFoldS(0.01)
    # print('')
    # print('')
    # res_lasso_optuna_N = LassoKFoldOptunaN()
    # print('')
    # print('')
    # res_lasso_optuna_S = LassoKFoldOptunaS()
    # print('')
    # print('')
    # res_ridge_N = RidgeKFoldN(1.0)
    # print('')
    # print('')
    # res_ridge_S = RidgeKFoldS(1.0)
    # print('')
    # print('')
    # res_ridge_optuna_N = RidgeKFoldOptunaN()
    # print('')
    # print('')
    # res_ridge_optuna_S = RidgeKFoldOptunaS()
    # print('')
    # print('')
    # res_SVM_N = SVMKFoldN()
    # print('')
    # print('')
    # res_SVM_S = SVMKFoldS()
    # print('')
    # print('')
    # res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    # print('')
    # print('')
    # res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    # print('')
    # print('')
    # res_GaussianN = GaussianProcessKFold(scaler='norm')
    # print('')
    # print('')
    # res_GaussianS = GaussianProcessKFold(scaler='stand')
    # print('')
    # print('')
    # res_Gradient_Boosting = GradientBoostingKFold()
    # print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    # print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    # print('')
    # res_RandomForest_optuna = RandomForestOptunaKFold()
    # print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    # print('')
    # res_XGBoost_optuna = XGBoostOptunaKFold()
    # print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    # print('')
    # res_LightGBM_optuna = LightGBMOptunaKFold()
    # print('')
    # print('')
    # res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    # print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    # print('')
    # res_YDF = YggdrasilDecisionForestKFold()
    # print('')
    # print('')
    # res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    # print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    # print('')
    # res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    # print('')
    # print('')
    # res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    # print('')
    # res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')

    res_list2 = [
                res_linear,
                                res_logistic_N,
                                # res_logistic_S,
                                res_lasso_N,
                                # res_lasso_S,
                                # res_lasso_optuna_N,
                                # res_lasso_optuna_S,
                                # res_ridge_N,
                                # res_ridge_S,
                                # res_ridge_optuna_N,
                                # res_ridge_optuna_S,
                                # res_SVM_N,
                                # res_SVM_S,
                                # res_LDAn,
                                # res_LDAs,
                                # res_GaussianN,
                                # res_GaussianS,
                                # res_Gradient_Boosting,
                                res_Adaboost,
                                res_RandomForest,
                                # res_RandomForest_optuna,
                                res_XGBoost,
                                # res_XGBoost_optuna,
                                res_LightGBM,
                                # res_LightGBM_optuna,
                                # res_LightGBMTuner_optuna,
                                # res_CatBoost,
                                # res_YDF,
                                # res_YDF_optuna,
                                res_MLPn,
                                # res_MLPs,
                                # res_TabNetN,
                                # res_TabNetS,
                                # res_TabNetN_optuna,
                                # res_TabNetS_optuna,
]

    results = pd.concat([item[0] for item in res_list2], axis=0)
    res_auc = [item[1] for item in res_list2]
    results.to_excel(filepath)


# プロットの準備
plt.figure(figsize=(14, 10))  # 図のサイズを少し大きく設定

# AUCが大きい順にデータをソート。res_auc の各エントリは (fpr, tpr, auc, name) の形式
data_sorted = sorted(res_auc, key=lambda x: x[2], reverse=True)

# 最もAUCが高いエントリを取得
if data_sorted:
    top_auc = data_sorted[0][2]
else:
    top_auc = None

# 各エントリをループしてプロット
for idx, entry in enumerate(data_sorted):
    fpr, tpr, auc, name = entry
    # NaNが含まれているエントリはスキップ
    if np.isnan(auc) or np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
        continue
    # 最もAUCが高いエントリは太くプロット
    if idx == 0:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=3.5)
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=1.5)

# 対角線のプロット
plt.plot([0, 1], [0, 1], 'k--', label='random prediction')

# グラフの装飾
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curves Comparison', fontsize=16)
plt.grid(True)

# アスペクト比を1に設定
plt.gca().set_aspect('equal', adjustable='box')

# 軸の範囲を0から1に設定
plt.xlim([0, 1])
plt.ylim([0, 1])

# 凡例を図の右側外側に配置
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop=font_prop)

# レイアウトの調整
plt.tight_layout()

roc_plot_path = os.path.join(path_figure, "all_ROC_curves.png")
plt.savefig(roc_plot_path, bbox_inches='tight')

print('')
print("■ Figure overlaying all the ROC curves by all the ML models")
print('')

# プロットの表示
plt.show()
plt.close()

print('')
print(f"■ All the ROC curves saved to: {roc_plot_path}")
print('')
print('')



# 計算終了時の現在時刻
end_time = datetime.now()
formatted_end_time = end_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
print(f'◆ Present Time (End): {formatted_end_time}')

# 計算に要した時間（終了時刻 - 開始時刻）
elapsed_time = end_time - start_time

# 時間、分、秒に分割して表示
hours, remainder = divmod(elapsed_time.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print('')
print(f'計算に要した時間: {hours} 時間, {minutes} 分, {seconds} 秒')
print('')


print('◆　計算結果の出力先')
print('')
print('1. 数値データ:   ' + path_table)
print('')
print('2. 画像データ:   ' + path_figure)
print('')
print('3. モデル:      ' + path_model)
print('')
print('')

## 2) 頻用モデル Set (12個の機械学習モデル)

In [ ]:
print('')
print('■ 二値分類の簡易Verson')
print('')
print(f'◆ The sample size of dataset: {n_samples}')
print('')

# 説明変数と目的変数を分離: Pandas形式
X0 = df.iloc[:, :-1]
y0 = df.iloc[:, -1]

# データの大きさ
print('◆　行と列の数')
rows, cols = df.shape
print(f'データの行数(サンプル数) {rows},  データの列数(説明変数＋目的変数の数) {cols}')
if rows >= 5000 or cols >= 5000:
    bigdata = 1
    print('')
    print('◇ データは Big Data のため、Random Forest と LightGBM への Optuna適用 と CatBoost の利用が省かれる。')
elif rows < 5000 and cols < 5000:
    bigdata = 0
    print('')
    print('◇ データは Big Data ではないため、Optuna は適用可能なモデルには適用する。')
print('')

print('')
# データの分割数 k
print(f"◆ Chosen value of k for k-fold cross-validation: {k}")
print('')
print('')

# 目的変数の分布の表示
print('◆　目的変数の分布')
sns.displot(df.iloc[:,-1].dropna())
print('　　　　　陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %')
plt.show()
print('')
print('')


import matplotlib.font_manager as fm
# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)



# データの線形性の確認

# 線形回帰モデルによる決定係数R2
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)


# (二値分類のため)Logistic回帰モデルによる擬似決定係数R2: NagelkerkeのR2 (Cox-Snellの擬似R2を0〜1にscalingしたもの)
import numpy as np
from sklearn.linear_model import LogisticRegression
def nagelkerke_r2(X, y, **logistic_kwargs):
    """
    X: array-like of shape (n_samples, n_features)
    y: array-like of shape (n_samples,), binary target (0 or 1)
    logistic_kwargs:
        追加の LogisticRegression 引数（solver, C, penalty など）

    Returns
    -------
    float
        Nagelkerke の R²
    """
    # 1) モデルのフィッティング
    model = LogisticRegression(**logistic_kwargs)
    model.fit(X, y)

    # 2) 予測確率（正クラス）の取得
    p = model.predict_proba(X)[:, 1]
    # log(0) を防ぐため eps 分だけクリップ
    eps = np.finfo(float).eps
    p = np.clip(p, eps, 1 - eps)

    # 3) フルモデルの対数尤度
    ll_model = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # 4) ヌルモデル（切片のみ、常に y の平均を予測）の対数尤度
    p_null = np.clip(np.mean(y), eps, 1 - eps)
    ll_null = np.sum(y * np.log(p_null) + (1 - y) * np.log(1 - p_null))

    # 5) Cox–Snell の R²
    n = len(y)
    r2_cs = 1 - np.exp((ll_null - ll_model) * 2 / n)

    # 6) Nagelkerke の R²
    r2_nagelkerke = r2_cs / (1 - np.exp(2 * ll_null / n))

    return r2_nagelkerke

pseudo_r2 = nagelkerke_r2(
    X, y,
    penalty='l2',     # リッジ（L2）正則化
    C=1.0,            # デフォルト付近からスタート
    solver='lbfgs',   # 中小規模向き
    max_iter=500,     # 収束用に少し余裕を
    tol=1e-6,          # 精度重視
    class_weight='balanced' # Imbalanced Dataにも対応
)


print(f'◆　説明変数と目的変数の間の決定係数R2')
print('')
print(f'◇ Nagelkerke の擬似R² (標準設定) = {pseudo_r2:.4f}')
print(f'  擬似決係数R²の見方　0.2未満: 弱い説明力、0.2〜0.5: 中程度の説明力、0.5以上: 強い説明力')
print('')
print(f'◇ 線形回帰による決定係数R² = {r2:.4f}')
print('   決定係数R²の見方　 0.3未満: 線形性なし、0.3〜0.5: 弱い線形性、0.5〜0.7: 中程度の線形性、0.7〜0.9: 強い線形性、0.9以上: 完全に線形')
print('')
print('  (擬似決係数R² > 0.5　または  決定係数R² > 0.7 のとき、線形性があると見なせる。二値分類では擬似R²の方が妥当)')
print('')
if pseudo_r2 >= 0.5:
    print('')
    print('◇ データの線形性が見られるため、SVM の kernel と XGBoost の booster を線形にする。')
    linear = 1
elif pseudo_r2 < 0.5:
    print('')
    print('◇ データの線形性はそれほど見られないため、線形モデルの優先は行わない。')
    linear = 0
print('')
print(f'linear = {linear:.1f}')
print('')
print('')



# 残差プロット
print('◆　線形モデルによる目的変数の予測値の残差(誤差の)プロット')
y_pred = model.predict(X)
residuals = y - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('予測値', fontproperties=font_prop)
plt.ylabel('残差', fontproperties=font_prop)
plt.title('予測値と残差のプロット', fontproperties=font_prop)
plt.show()
print('')
print('')



# 連続変数の場合の線形性の判定

# # RESET（Regression Specification Error Test）
# from statsmodels.stats.diagnostic import linear_reset
# import statsmodels.api as sm

# ols = sm.OLS(y, sm.add_constant(X)).fit()
# print(linear_reset(ols, power=2))

# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression

# # 線形モデル vs 多項式モデル（2次項・3次項を加えたモデル）
# poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
# score_lin = cross_val_score(LinearRegression(), X, y, cv=5, scoring="r2").mean()
# score_poly2 = cross_val_score(poly2, X, y, cv=5, scoring="r2").mean()

# print("線形モデル R²:", score_lin)
# print("2次多項式モデル R²:", score_poly2)



# 定数列を除外する関数
def remove_constant_columns(df):
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        print(f'以下の定数列を除外します: {constant_columns}')
        df = df.drop(columns=constant_columns)
    else:
        print('定数列は存在しません。')
    return df

# 欠損値を確認・除外する関数
def check_missing_values(df_X, df_y):
    missing_X = df_X.isnull().sum()
    missing_y = df_y.isnull().sum()
    if missing_X.sum() > 0:
        print('説明変数に欠損値が含まれています。欠損値を除外します。')
        df_X = df_X.dropna()
    if missing_y > 0:
        print('目的変数に欠損値が含まれています。欠損値を除外します。')
        df_y = df_y.dropna()
    return df_X, df_y


print('◆ 説明変数と目的変数のPearsonの相関係数: 線形な相関のみ')
# 定数列の除外
X0 = remove_constant_columns(X0)

# 目的変数が定数か確認
if y0.nunique() <= 1:
    raise ValueError('目的変数 y0 が定数列です。相関係数を計算できません。')

# 欠損値の除外
X0, y0 = check_missing_values(X0, y0)


# 各説明変数と目的変数との相関係数を計算
correlations = {}
for column in X0.columns:
    correlations[column] = X0[column].corr(y0)

# 相関係数をSeriesに変換
correlation_series = pd.Series(correlations)

# 絶対値の降順にソートし、元の符号を保持
top_10_correlations = correlation_series.reindex(correlation_series.abs().sort_values(ascending=False).index).head(10)

# Top 10の相関係数を昇べき順にソート
sorted_top_10_correlations = top_10_correlations.sort_values(ascending=True)

# 色を決定
colors = ['skyblue' if val > 0 else 'lightcoral' for val in correlation_series.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(correlation_series.index, correlation_series.values, color=colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()

print('□ 青は正の相関、赤は負の相関。　1に近いほど強い正の相関、-1に近いほど強い負の相関。0は線形での相関無し')
print('□　　相関係数の絶対値Cの目安： C > 0.7 強い相関、0.4〜0.7 中程度の相関、0.2〜0.4 弱い相関　　C < 0.2 相関無し')
print('')
print('')


print('◇ 説明変数と目的変数のPearsonの相関係数: 相関係数のTop 10 のみ')

# Top 10 correlation colors
top_10_colors = ['skyblue' if val > 0 else 'lightcoral' for val in sorted_top_10_correlations.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(sorted_top_10_correlations.index, sorted_top_10_correlations.values, color=top_10_colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Top 10 Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()
print('')
print('')
print('')


# XiCorの相関係数のプロット関数
print('◆ 説明変数と目的変数のChatterjee`sの相関係数: 非線形な相関も含む (参考値)')

def xicor(x, y, ties=True):
    np.random.seed(42)
    n = len(x)
    order = np.argsort(x)
    ranked_y = np.argsort(np.argsort(y[order], kind='mergesort'))

    if ties:
        l = np.argsort(np.argsort(y[order], kind='max'))
        r = ranked_y.copy()
        return 1 - n * np.sum(np.abs(r[1:] - r[:-1])) / (2 * np.sum(l * (n - l)))
    else:
        return 1 - 3 * np.sum(np.abs(ranked_y[1:] - ranked_y[:-1])) / (n**2 - 1)

def plot_xicor(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor(df)
print('')


print('◇ 説明変数と目的変数のChatterjee`sの相関係数: 相関係数のTop 10 のみ')
def plot_xicor10(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)[-10:]  # 上位10の特徴量を選択
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(10, 6))  # グラフのサイズを調整
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('Top 10 XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor10(df)
print('')
print('\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\thttps://arxiv.org/abs/1909.10140 参照')
print('')


from datetime import datetime
start_time = datetime.now()
formatted_start_time2 = start_time.strftime('%Y%m%d%H%M')


# 出力ファイル
filename0 = os.path.basename(path_table)
filename = os.path.splitext(filename0)[0]
filepath = path_table + '/AllEvaluationResultsFor' + filename + str(formatted_start_time2) + '.xlsx'
filepath2 = path_table + '/AllAUCResultsFor' + filename + str(formatted_start_time2) + '.csv'
print('◆　計算結果の出力先')
print(filepath)
print('')
print('')


# 現在時刻（計算開始時）
from datetime import datetime
start_time = datetime.now()
formatted_start_time = start_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
formatted_start_time2 = start_time.strftime('%Y年%m月%d日 %H時%M分%')
print(f'◆ Present Time (Start): {formatted_start_time}')
print('')



# 各機械学習モデルの実行

# データがそれほど大きくないとき
if bigdata ==0:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    # print('')
    # res_logistic_S = LogisticKFoldS()
    # print('')
    # print('')
    # res_lasso_N = LassoKFoldN(0.01)
    # print('')
    # print('')
    # res_lasso_S = LassoKFoldS(0.01)
    # print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    # print('')
    # res_lasso_optuna_S = LassoKFoldOptunaS()
    # print('')
    # print('')
    # res_ridge_N = RidgeKFoldN(1.0)
    # print('')
    # print('')
    # res_ridge_S = RidgeKFoldS(1.0)
    # print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    # print('')
    # res_ridge_optuna_S = RidgeKFoldOptunaS()
    # print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    # print('')
    # res_SVM_S = SVMKFoldS()
    # print('')
    # print('')
    # res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    # print('')
    # print('')
    # res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    # print('')
    # print('')
    # res_GaussianN = GaussianProcessKFold(scaler='norm')
    # print('')
    # print('')
    # res_GaussianS = GaussianProcessKFold(scaler='stand')
    # print('')
    # print('')
    # res_Gradient_Boosting = GradientBoostingKFold()
    # print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    # print('')
    # res_RandomForest = RandomForestKFold()
    # print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    # print('')
    # res_XGBoost = XGBoostKFold()
    # print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    # print('')
    # res_LightGBM = LightGBMKFold()
    # print('')
    # print('')
    # res_LightGBM_optuna = LightGBMOptunaKFold()
    # print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    # print('')
    # res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    # print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    # print('')
    # res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    # print('')
    # print('')
    # res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')

    res_list = [
                res_linear,
                res_logistic_N,
                # res_logistic_S,
                # res_lasso_N,
                # res_lasso_S,
                res_lasso_optuna_N,
                # res_lasso_optuna_S,
                # res_ridge_N,
                # res_ridge_S,
                res_ridge_optuna_N,
                # res_ridge_optuna_S,
                res_SVM_N,
                # res_SVM_S,
                # res_LDAn,
                # res_LDAs,
                # res_GaussianN,
                # res_GaussianS,
                # res_Gradient_Boosting,
                res_Adaboost,
                # res_RandomForest,
                res_RandomForest_optuna,
                # res_XGBoost,
                res_XGBoost_optuna,
                # res_LightGBM,
                # res_LightGBM_optuna,
                res_LightGBMTuner_optuna,
                # res_CatBoost,
                res_YDF,
                # res_YDF_optuna,
                res_MLPn,
                # res_MLPs,
                # res_TabNetN,
                res_TabNetS,
                # res_TabNetN_optuna,
                # res_TabNetS_optuna,
]


    results = pd.concat([item[0] for item in res_list], axis=0)
    res_auc = [item[1] for item in res_list]
    results.to_excel(filepath)


# データが大きいとき
else:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    # print('')
    # res_logistic_S = LogisticKFoldS()
    # print('')
    # print('')
    # res_lasso_N = LassoKFoldN(0.01)
    # print('')
    # print('')
    # res_lasso_S = LassoKFoldS(0.01)
    # print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    # print('')
    # res_lasso_optuna_S = LassoKFoldOptunaS()
    # print('')
    # print('')
    # res_ridge_N = RidgeKFoldN(1.0)
    # print('')
    # print('')
    # res_ridge_S = RidgeKFoldS(1.0)
    # print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    # print('')
    # res_ridge_optuna_S = RidgeKFoldOptunaS()
    # print('')
    # print('')
    # res_SVM_N = SVMKFoldN()
    # print('')
    # print('')
    # res_SVM_S = SVMKFoldS()
    # print('')
    # print('')
    # res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    # print('')
    # print('')
    # res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    # print('')
    # print('')
    # res_GaussianN = GaussianProcessKFold(scaler='norm')
    # print('')
    # print('')
    # res_GaussianS = GaussianProcessKFold(scaler='stand')
    # print('')
    # print('')
    # res_Gradient_Boosting = GradientBoostingKFold()
    # print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    # print('')
    # res_RandomForest = RandomForestKFold()
    # print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    # print('')
    # res_XGBoost = XGBoostKFold()
    # print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    # print('')
    # res_LightGBM = LightGBMKFold()
    # print('')
    # print('')
    # res_LightGBM_optuna = LightGBMOptunaKFold()
    # print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    # print('')
    # res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    # print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    # print('')
    # res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    # print('')
    # print('')
    # res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    # print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')

    res_list2 = [
                res_linear,
                res_logistic_N,
                # res_logistic_S,
                # res_lasso_N,
                # res_lasso_S,
                res_lasso_optuna_N,
                # res_lasso_optuna_S,
                # res_ridge_N,
                # res_ridge_S,
                res_ridge_optuna_N,
                # res_ridge_optuna_S,
                # res_SVM_N,
                # res_SVM_S,
                # res_LDAn,
                # res_LDAs,
                # res_GaussianN,
                # res_GaussianS,
                # res_Gradient_Boosting,
                res_Adaboost,
                # res_RandomForest,
                res_RandomForest_optuna,
                # res_XGBoost,
                res_XGBoost_optuna,
                # res_LightGBM,
                # res_LightGBM_optuna,
                res_LightGBMTuner_optuna,
                # res_CatBoost,
                res_YDF,
                # res_YDF_optuna,
                res_MLPn,
                # res_MLPs,
                # res_TabNetN,
                res_TabNetS,
                # res_TabNetN_optuna,
                # res_TabNetS_optuna
]

    results = pd.concat([item[0] for item in res_list2], axis=0)
    res_auc = [item[1] for item in res_list2]
    results.to_excel(filepath)


# プロットの準備
plt.figure(figsize=(14, 10))  # 図のサイズを少し大きく設定

# AUCが大きい順にデータをソート。res_auc の各エントリは (fpr, tpr, auc, name) の形式
data_sorted = sorted(res_auc, key=lambda x: x[2], reverse=True)

# 最もAUCが高いエントリを取得
if data_sorted:
    top_auc = data_sorted[0][2]
else:
    top_auc = None

# 各エントリをループしてプロット
for idx, entry in enumerate(data_sorted):
    fpr, tpr, auc, name = entry
    # NaNが含まれているエントリはスキップ
    if np.isnan(auc) or np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
        continue
    # 最もAUCが高いエントリは太くプロット
    if idx == 0:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=3.5)
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=1.5)

# 対角線のプロット
plt.plot([0, 1], [0, 1], 'k--', label='random prediction')

# グラフの装飾
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curves Comparison', fontsize=16)
plt.grid(True)

# アスペクト比を1に設定
plt.gca().set_aspect('equal', adjustable='box')

# 軸の範囲を0から1に設定
plt.xlim([0, 1])
plt.ylim([0, 1])

# 凡例を図の右側外側に配置
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop=font_prop)

# レイアウトの調整
plt.tight_layout()

roc_plot_path = os.path.join(path_figure, "all_ROC_curves.png")
plt.savefig(roc_plot_path, bbox_inches='tight')

print('')
print("■ Figure overlaying all the ROC curves by all the ML models")
print('')

# プロットの表示
plt.show()
plt.close()

print('')
print(f"■ All the ROC curves saved to: {roc_plot_path}")
print('')
print('')



# 計算終了時の現在時刻
end_time = datetime.now()
formatted_end_time = end_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
print(f'◆ Present Time (End): {formatted_end_time}')

# 計算に要した時間（終了時刻 - 開始時刻）
elapsed_time = end_time - start_time

# 時間、分、秒に分割して表示
hours, remainder = divmod(elapsed_time.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print('')
print(f'計算に要した時間: {hours} 時間, {minutes} 分, {seconds} 秒')
print('')


print('◆　計算結果の出力先')
print('')
print('1. 数値データ:   ' + path_table)
print('')
print('2. 画像データ:   ' + path_figure)
print('')
print('3. モデル:      ' + path_model)
print('')
print('')

## 3) 任意の機械学習モデル Set

In [ ]:
print('')
print('■ 二値分類のFull Verson')
print('')
print(f'◆ The sample size of dataset: {n_samples}')
print('')

# 説明変数と目的変数を分離: Pandas形式
X0 = df.iloc[:, :-1]
y0 = df.iloc[:, -1]

# データの大きさ
print('◆　行と列の数')
rows, cols = df.shape
print(f'データの行数(サンプル数) {rows},  データの列数(説明変数＋目的変数の数) {cols}')
if rows >= 5000 or cols >= 5000:
    bigdata = 1
    print('')
    print('◇ データは Big Data のため、Random Forest と LightGBM への Optuna適用 と CatBoost の利用が省かれる。')
elif rows < 5000 and cols < 5000:
    bigdata = 0
    print('')
    print('◇ データは Big Data ではないため、Optuna は適用可能なモデルには適用する。')
print('')

print('')
# データの分割数 k
print(f"◆ Chosen value of k for k-fold cross-validation: {k}")
print('')
print('')

# 目的変数の分布の表示
print('◆　目的変数の分布')
sns.displot(df.iloc[:,-1].dropna())
print('　　　　　陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %')
plt.show()
print('')
print('')


import matplotlib.font_manager as fm
# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)



# データの線形性の確認

# 線形回帰モデルによる決定係数R2
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# (二値分類のため)Logistic回帰モデルによる擬似決定係数R2: NagelkerkeのR2 (Cox-Snellの擬似R2を0〜1にscalingしたもの)
import numpy as np
from sklearn.linear_model import LogisticRegression
def nagelkerke_r2(X, y, **logistic_kwargs):
    """
    X: array-like of shape (n_samples, n_features)
    y: array-like of shape (n_samples,), binary target (0 or 1)
    logistic_kwargs:
        追加の LogisticRegression 引数（solver, C, penalty など）

    Returns
    -------
    float
        Nagelkerke の R²
    """
    # 1) モデルのフィッティング
    model = LogisticRegression(**logistic_kwargs)
    model.fit(X, y)

    # 2) 予測確率（正クラス）の取得
    p = model.predict_proba(X)[:, 1]
    # log(0) を防ぐため eps 分だけクリップ
    eps = np.finfo(float).eps
    p = np.clip(p, eps, 1 - eps)

    # 3) フルモデルの対数尤度
    ll_model = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # 4) ヌルモデル（切片のみ、常に y の平均を予測）の対数尤度
    p_null = np.clip(np.mean(y), eps, 1 - eps)
    ll_null = np.sum(y * np.log(p_null) + (1 - y) * np.log(1 - p_null))

    # 5) Cox–Snell の R²
    n = len(y)
    r2_cs = 1 - np.exp((ll_null - ll_model) * 2 / n)

    # 6) Nagelkerke の R²
    r2_nagelkerke = r2_cs / (1 - np.exp(2 * ll_null / n))

    return r2_nagelkerke

pseudo_r2 = nagelkerke_r2(
    X, y,
    penalty='l2',     # リッジ（L2）正則化
    C=1.0,            # デフォルト付近からスタート
    solver='lbfgs',   # 中小規模向き
    max_iter=500,     # 収束用に少し余裕を
    tol=1e-6,          # 精度重視
    class_weight='balanced' # Imbalanced Dataにも対応
)


print(f'◆　説明変数と目的変数の間の決定係数R2')
print('')
print(f'◇ Nagelkerke の擬似R² (標準設定) = {pseudo_r2:.4f}')
print(f'  擬似決係数R²の見方　0.2未満: 弱い説明力、0.2〜0.5: 中程度の説明力、0.5以上: 強い説明力')
print('')
print(f'◇ 線形回帰による決定係数R² = {r2:.4f}')
print('   決定係数R²の見方　 0.3未満: 線形性なし、0.3〜0.5: 弱い線形性、0.5〜0.7: 中程度の線形性、0.7〜0.9: 強い線形性、0.9以上: 完全に線形')
print('')
print('  (擬似決係数R² > 0.5　または  決定係数R² > 0.7 のとき、線形性があると見なせる。二値分類では擬似R²の方が妥当)')
print('')
if pseudo_r2 >= 0.5:
    print('')
    print('◇ データの線形性が見られるため、SVM の kernel と XGBoost の booster を線形にする。')
    linear = 1
elif pseudo_r2 < 0.5:
    print('')
    print('◇ データの線形性はそれほど見られないため、線形モデルの優先は行わない。')
    linear = 0
print('')
print(f'linear = {linear:.1f}')
print('')
print('')



# 残差プロット
print('◆　線形モデルによる目的変数の予測値の残差プロット')
y_pred = model.predict(X)
residuals = y - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('予測値', fontproperties=font_prop)
plt.ylabel('残差', fontproperties=font_prop)
plt.title('予測値と残差のプロット', fontproperties=font_prop)
plt.show()
print('')
print('')



# 連続変数の場合の線形性の判定

# # RESET（Regression Specification Error Test）
# from statsmodels.stats.diagnostic import linear_reset
# import statsmodels.api as sm

# ols = sm.OLS(y, sm.add_constant(X)).fit()
# print(linear_reset(ols, power=2))

# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression

# # 線形モデル vs 多項式モデル（2次項・3次項を加えたモデル）
# poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
# score_lin = cross_val_score(LinearRegression(), X, y, cv=5, scoring="r2").mean()
# score_poly2 = cross_val_score(poly2, X, y, cv=5, scoring="r2").mean()

# print("線形モデル R²:", score_lin)
# print("2次多項式モデル R²:", score_poly2)




# 定数列を除外する関数
def remove_constant_columns(df):
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        print(f'以下の定数列を除外します: {constant_columns}')
        df = df.drop(columns=constant_columns)
    else:
        print('定数列は存在しません。')
    return df

# 欠損値を確認・除外する関数
def check_missing_values(df_X, df_y):
    missing_X = df_X.isnull().sum()
    missing_y = df_y.isnull().sum()
    if missing_X.sum() > 0:
        print('説明変数に欠損値が含まれています。欠損値を除外します。')
        df_X = df_X.dropna()
    if missing_y > 0:
        print('目的変数に欠損値が含まれています。欠損値を除外します。')
        df_y = df_y.dropna()
    return df_X, df_y


print('◆ 説明変数と目的変数のPearsonの相関係数: 線形な相関のみ')
# 定数列の除外
X0 = remove_constant_columns(X0)

# 目的変数が定数か確認
if y0.nunique() <= 1:
    raise ValueError('目的変数 y0 が定数列です。相関係数を計算できません。')

# 欠損値の除外
X0, y0 = check_missing_values(X0, y0)


# 各説明変数と目的変数との相関係数を計算
correlations = {}
for column in X0.columns:
    correlations[column] = X0[column].corr(y0)

# 相関係数をSeriesに変換
correlation_series = pd.Series(correlations)

# 絶対値の降順にソートし、元の符号を保持
top_10_correlations = correlation_series.reindex(correlation_series.abs().sort_values(ascending=False).index).head(10)

# Top 10の相関係数を昇べき順にソート
sorted_top_10_correlations = top_10_correlations.sort_values(ascending=True)

# 色を決定
colors = ['skyblue' if val > 0 else 'lightcoral' for val in correlation_series.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(correlation_series.index, correlation_series.values, color=colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()

print('□ 青は正の相関、赤は負の相関。　1に近いほど強い正の相関、-1に近いほど強い負の相関。0は線形での相関無し')
print('□　　相関係数の絶対値Cの目安： C > 0.7 強い相関、0.4〜0.7 中程度の相関、0.2〜0.4 弱い相関　　C < 0.2 相関無し')
print('')
print('')


print('◇ 説明変数と目的変数のPearsonの相関係数: 相関係数のTop 10 のみ')

# Top 10 correlation colors
top_10_colors = ['skyblue' if val > 0 else 'lightcoral' for val in sorted_top_10_correlations.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(sorted_top_10_correlations.index, sorted_top_10_correlations.values, color=top_10_colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Top 10 Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()
print('')
print('')
print('')


# XiCorの相関係数のプロット関数
print('◆ 説明変数と目的変数のChatterjee`sの相関係数: 非線形な相関も含む (参考値)')

def xicor(x, y, ties=True):
    np.random.seed(42)
    n = len(x)
    order = np.argsort(x)
    ranked_y = np.argsort(np.argsort(y[order], kind='mergesort'))

    if ties:
        l = np.argsort(np.argsort(y[order], kind='max'))
        r = ranked_y.copy()
        return 1 - n * np.sum(np.abs(r[1:] - r[:-1])) / (2 * np.sum(l * (n - l)))
    else:
        return 1 - 3 * np.sum(np.abs(ranked_y[1:] - ranked_y[:-1])) / (n**2 - 1)

def plot_xicor(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor(df)
print('')


print('◇ 説明変数と目的変数のChatterjee`sの相関係数: 相関係数のTop 10 のみ')
def plot_xicor10(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)[-10:]  # 上位10の特徴量を選択
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(10, 6))  # グラフのサイズを調整
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('Top 10 XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor10(df)
print('')
print('\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\thttps://arxiv.org/abs/1909.10140 参照')
print('')


from datetime import datetime
start_time = datetime.now()
formatted_start_time2 = start_time.strftime('%Y%m%d%H%M')


# 出力ファイル
filename0 = os.path.basename(path_table)
filename = os.path.splitext(filename0)[0]
filepath = path_table + '/AllEvaluationResultsFor' + filename + str(formatted_start_time2) + '.xlsx'
filepath2 = path_table + '/AllAUCResultsFor' + filename + str(formatted_start_time2) + '.csv'
print('◆　計算結果の出力先')
print(filepath)
print('')
print('')


# 現在時刻（計算開始時）
from datetime import datetime
start_time = datetime.now()
formatted_start_time = start_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
formatted_start_time2 = start_time.strftime('%Y年%m月%d日 %H時%M分%')
print(f'◆ Present Time (Start): {formatted_start_time}')
print('')



# 各機械学習モデルの実行

# 32種類の機械学習モデルの名称と実行用関数を、番号とともに辞書で定義します
models = {
    1:  {"name": "LinearKFold",                  "func": lambda: LinearKFold()},
    2:  {"name": "LogisticKFoldN",               "func": lambda: LogisticKFoldN()},
    3:  {"name": "LogisticKFoldS",               "func": lambda: LogisticKFoldS()},
    4:  {"name": "LassoKFoldN (0.01)",           "func": lambda: LassoKFoldN(0.01)},
    5:  {"name": "LassoKFoldS (0.01)",           "func": lambda: LassoKFoldS(0.01)},
    6:  {"name": "LassoKFoldOptunaN",            "func": lambda: LassoKFoldOptunaN()},
    7:  {"name": "LassoKFoldOptunaS",            "func": lambda: LassoKFoldOptunaS()},
    8:  {"name": "RidgeKFoldN (1.0)",            "func": lambda: RidgeKFoldN(1.0)},
    9:  {"name": "RidgeKFoldS (1.0)",            "func": lambda: RidgeKFoldS(1.0)},
    10: {"name": "RidgeKFoldOptunaN",            "func": lambda: RidgeKFoldOptunaN()},
    11: {"name": "RidgeKFoldOptunaS",            "func": lambda: RidgeKFoldOptunaS()},
    12: {"name": "SVMKFoldN",                    "func": lambda: SVMKFoldN()},
    13: {"name": "SVMKFoldS",                    "func": lambda: SVMKFoldS()},
    14: {"name": "LinearDiscriminantAnalysisKFold Normalized", "func": lambda: LinearDiscriminantAnalysisKFold(scaler='norm')},
    15: {"name": "LinearDiscriminantAnalysisKFold Standard",   "func": lambda: LinearDiscriminantAnalysisKFold(scaler='stand')},
    16: {"name": "GausseanProcessKFold Normalized",            "func": lambda: GaussianProcessKFold(scaler='norm')},
    17: {"name": "GausseanProcessKFold Standardized",          "func": lambda: GaussianProcessKFold(scaler='stand')},
    18: {"name": "GradientBoostingKFold",        "func": lambda: GradientBoostingKFold()},
    19: {"name": "AdaBoostKFold",                "func": lambda: AdaBoostKFold()},
    20: {"name": "RandomForestKFold",            "func": lambda: RandomForestKFold()},
    21: {"name": "RandomForestOptunaKFold",      "func": lambda: RandomForestOptunaKFold()},
    22: {"name": "XGBoostKFold",                 "func": lambda: XGBoostKFold()},
    23: {"name": "XGBoostOptunaKFold",           "func": lambda: XGBoostOptunaKFold()},
    24: {"name": "LightGBMKFold",                "func": lambda: LightGBMKFold()},
    25: {"name": "LightGBMOptunaKFold",          "func": lambda: LightGBMOptunaKFold()},
    26: {"name": "LightGBMTunerOptunaKFold",     "func": lambda: LightGBMTunerOptunaKFold()},
    27: {"name": "CatBoostKFold",                "func": lambda: CatBoostKFold()},
    28: {"name": "YDFKFold",                     "func": lambda: YggdrasilDecisionForestKFold()},
    29: {"name": "YDFKFoldOptuna",               "func": lambda: YggdrasilDecisionForestOptunaKFold()},
    30: {"name": "MLPKFoldN",                    "func": lambda: MLPKFold(n1=128, n2=32, max_iter=500, scaler='norm')},
    31: {"name": "MLPKFoldS",                    "func": lambda: MLPKFold(n1=128, n2=32, max_iter=500, scaler='stand')},
    32: {"name": "TabNetKFoldN",                 "func": lambda: TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)},
    33: {"name": "TabNetKFoldS",                 "func": lambda: TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)},
    34: {"name": "TabNetOptunaKFoldN",           "func": lambda: TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)},
    35: {"name": "TabNetOptunaKFoldS",           "func": lambda: TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)}
}


print('')
# bigdata == 1 の場合、CatBoostKFold (キー: 20) を削除
if bigdata == 1 and 21 in models:
    sample_num = len(df)
    del models[20]
    if cols >= 5000 and rows < 5000:
        print(f'□ サンプル数({cols})が5000以上のため、CatBoostモデルは削除されました。')
    elif cols < 5000 and rows >= 5000:
        print(f'□ 説明変数の数({rows})が5000以上のため、CatBoostモデルは削除されました。')
    elif cols >= 5000 and rows >= 5000:
        print(f'□ サンプル数({cols})も説明変数の数({rows})も共に5000以上のため、CatBoostモデルは削除されました。')
else:
    print(f'□ サンプル数({cols})も説明変数の数({rows})も共に5000未満のため、CatBoostモデルも候補に入れます。')
print('')

# まず、27種類のモデル一覧を表示します
print('')
print("■ 実行可能な機械学習モデル一覧:")
print('')
for key in sorted(models.keys()):
    print(f"{key}: {models[key]['name']}")
print('')
print('')


# ユーザーに、実行したいモデルの番号をカンマ区切りで入力してもらいます
selected_input = input("\n■ 実行したいモデルの番号をカンマ区切りで入力してください。    例 1,3,7,11,17,19   　(見えない場合は左にスライドする)")

# 入力文字列をパースして、整数のリストに変換します
try:
    selected_numbers = [int(num.strip()) for num in selected_input.split(',') if num.strip()]
except ValueError:
    print("入力の形式が正しくありません。整数をカンマ区切りで入力してください。")
    raise
print('')


# 入力された番号が有効かチェックします
invalid_numbers = [num for num in selected_numbers if num not in models]
if invalid_numbers:
    print(f"無効な番号が含まれています: {invalid_numbers}")
    raise ValueError("無効な番号が含まれています。")
else:
    print("◇ 入力されたすべての番号が有効です。")
    print(f'　　入力された番号: {selected_numbers}')
    print('    選択されたモデル:')
    for num in selected_numbers:
        print(f'  {num}: {models[num]["name"]}')
print('')


# 選択されたモデルのみを実行し、その結果をリストに格納します
results_list = []
auc_list = []  # 各モデルから返される[1]要素を格納するリスト
for num in selected_numbers:
    print("\n" + "="*180)
    print(f"□ モデル {num}: {models[num]['name']} を実行中...")
    # 関数を呼び出して結果を取得
    res = models[num]['func']()
    # 戻り値は (結果DataFrame, auc値) と仮定
    results_list.append(res[0])
    auc_list.append(res[1])
    print(f"□ モデル {num}: {models[num]['name']} の実行が完了しました。")
    print("="*180)
    print('')
    print('')
    print('')

# 全ての結果DataFrameを結合し、Excelファイルに出力します
final_results = pd.concat(results_list, axis=0)
final_results.to_excel(filepath)
print("\nすべての選択されたモデルの結果がExcelファイルに出力されました。")
print('')
print('')



# すべてのROC曲線のプロットの準備
plt.figure(figsize=(14, 10))  # 図のサイズを少し大きく設定

# AUCが大きい順にデータをソート。res_auc の各エントリは (fpr, tpr, auc, name) の形式
data_sorted = sorted(auc_list, key=lambda x: x[2], reverse=True)

# 最もAUCが高いエントリを取得
if data_sorted:
    top_auc = data_sorted[0][2]
else:
    top_auc = None

# 各エントリをループしてプロット
for idx, entry in enumerate(data_sorted):
    fpr, tpr, auc, name = entry
    # NaNが含まれているエントリはスキップ
    if np.isnan(auc) or np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
        continue
    # 最もAUCが高いエントリは太くプロット
    if idx == 0:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=3.5)
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=1.5)

# 対角線のプロット
plt.plot([0, 1], [0, 1], 'k--', label='random prediction')

# グラフの装飾
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curves Comparison', fontsize=16)
plt.grid(True)

# アスペクト比を1に設定
plt.gca().set_aspect('equal', adjustable='box')

# 軸の範囲を0から1に設定
plt.xlim([0, 1])
plt.ylim([0, 1])

# 凡例を図の右側外側に配置
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop=font_prop)

# レイアウトの調整
plt.tight_layout()

roc_plot_path = os.path.join(path_figure, "all_ROC_curves.png")
plt.savefig(roc_plot_path, bbox_inches='tight')

print('')
print("■ Figure overlaying all the ROC curves by all the ML models")
print('')

# プロットの表示
plt.show()
plt.close()

print('')
print(f"■ All the ROC curves saved to: {roc_plot_path}")
print('')
print('')



# 計算終了時の現在時刻
end_time = datetime.now()
formatted_end_time = end_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
print(f'◆ Present Time (End): {formatted_end_time}')

# 計算に要した時間（終了時刻 - 開始時刻）
elapsed_time = end_time - start_time

# 時間、分、秒に分割して表示
hours, remainder = divmod(elapsed_time.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print('')
print(f'計算に要した時間: {hours} 時間, {minutes} 分, {seconds} 秒')
print('')


print('◆　計算結果の出力先')
print('')
print('1. 数値データ:   ' + path_table)
print('')
print('2. 画像データ:   ' + path_figure)
print('')
print('3. モデル:      ' + path_model)
print('')
print('')

## 4) Full Set without TabNetOptunaKFold

In [ ]:
print('')
print('■ 二値分類のFull Verson')
print('')
print(f'◆ The sample size of dataset: {n_samples}')
print('')

# 説明変数と目的変数を分離: Pandas形式
X0 = df.iloc[:, :-1]
y0 = df.iloc[:, -1]

# データの大きさ
print('◆　行と列の数')
rows, cols = df.shape
print(f'データの行数(サンプル数) {rows},  データの列数(説明変数＋目的変数の数) {cols}')
if rows >= 5000 or cols >= 5000:
    bigdata = 1
    print('')
    print('◇ データは Big Data のため、Random Forest と LightGBM への Optuna適用 と CatBoost の利用が省かれる。')
elif rows < 5000 and cols < 5000:
    bigdata = 0
    print('')
    print('◇ データは Big Data ではないため、Optuna は適用可能なモデルには適用する。')
print('')

print('')
# データの分割数 k
print(f"◆ Chosen value of k for k-fold cross-validation: {k}")
print('')
print('')

# 目的変数の分布の表示
print('◆　目的変数の分布')
sns.displot(df.iloc[:,-1].dropna())
print('　　　　　陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %')
plt.show()
print('')
print('')


import matplotlib.font_manager as fm
# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)




# データの線形性の確認

# 線形回帰モデルによる決定係数R2
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# (二値分類のため)Logistic回帰モデルによる擬似決定係数R2: NagelkerkeのR2 (Cox-Snellの擬似R2を0〜1にscalingしたもの)
import numpy as np
from sklearn.linear_model import LogisticRegression
def nagelkerke_r2(X, y, **logistic_kwargs):
    """
    X: array-like of shape (n_samples, n_features)
    y: array-like of shape (n_samples,), binary target (0 or 1)
    logistic_kwargs:
        追加の LogisticRegression 引数（solver, C, penalty など）

    Returns
    -------
    float
        Nagelkerke の R²
    """
    # 1) モデルのフィッティング
    model = LogisticRegression(**logistic_kwargs)
    model.fit(X, y)

    # 2) 予測確率（正クラス）の取得
    p = model.predict_proba(X)[:, 1]
    # log(0) を防ぐため eps 分だけクリップ
    eps = np.finfo(float).eps
    p = np.clip(p, eps, 1 - eps)

    # 3) フルモデルの対数尤度
    ll_model = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # 4) ヌルモデル（切片のみ、常に y の平均を予測）の対数尤度
    p_null = np.clip(np.mean(y), eps, 1 - eps)
    ll_null = np.sum(y * np.log(p_null) + (1 - y) * np.log(1 - p_null))

    # 5) Cox–Snell の R²
    n = len(y)
    r2_cs = 1 - np.exp((ll_null - ll_model) * 2 / n)

    # 6) Nagelkerke の R²
    r2_nagelkerke = r2_cs / (1 - np.exp(2 * ll_null / n))

    return r2_nagelkerke

pseudo_r2 = nagelkerke_r2(
    X, y,
    penalty='l2',     # リッジ（L2）正則化
    C=1.0,            # デフォルト付近からスタート
    solver='lbfgs',   # 中小規模向き
    max_iter=500,     # 収束用に少し余裕を
    tol=1e-6,          # 精度重視
    class_weight='balanced' # Imbalanced Dataにも対応
)


print(f'◆　説明変数と目的変数の間の決定係数R2')
print('')
print(f'◇ Nagelkerke の擬似R² (標準設定) = {pseudo_r2:.4f}')
print(f'  擬似決係数R²の見方　0.2未満: 弱い説明力、0.2〜0.5: 中程度の説明力、0.5以上: 強い説明力')
print('')
print(f'◇ 線形回帰による決定係数R² = {r2:.4f}')
print('   決定係数R²の見方　 0.3未満: 線形性なし、0.3〜0.5: 弱い線形性、0.5〜0.7: 中程度の線形性、0.7〜0.9: 強い線形性、0.9以上: 完全に線形')
print('')
print('  (擬似決係数R² > 0.5　または  決定係数R² > 0.7 のとき、線形性があると見なせる。二値分類では擬似R²の方が妥当)')
print('')
if pseudo_r2 >= 0.5:
    print('')
    print('◇ データの線形性が見られるため、SVM の kernel と XGBoost の booster を線形にする。')
    linear = 1
elif pseudo_r2 < 0.5:
    print('')
    print('◇ データの線形性はそれほど見られないため、線形モデルの優先は行わない。')
    linear = 0
print('')
print(f'linear = {linear:.1f}')
print('')
print('')



# 残差プロット
print('◆　線形モデルによる目的変数の予測値の残差プロット')
y_pred = model.predict(X)
residuals = y - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('予測値', fontproperties=font_prop)
plt.ylabel('残差', fontproperties=font_prop)
plt.title('予測値と残差のプロット', fontproperties=font_prop)
plt.show()
print('')
print('')



# 連続変数の場合の線形性の判定

# # RESET（Regression Specification Error Test）
# from statsmodels.stats.diagnostic import linear_reset
# import statsmodels.api as sm

# ols = sm.OLS(y, sm.add_constant(X)).fit()
# print(linear_reset(ols, power=2))

# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression

# # 線形モデル vs 多項式モデル（2次項・3次項を加えたモデル）
# poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
# score_lin = cross_val_score(LinearRegression(), X, y, cv=5, scoring="r2").mean()
# score_poly2 = cross_val_score(poly2, X, y, cv=5, scoring="r2").mean()

# print("線形モデル R²:", score_lin)
# print("2次多項式モデル R²:", score_poly2)



# 定数列を除外する関数
def remove_constant_columns(df):
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        print(f'以下の定数列を除外します: {constant_columns}')
        df = df.drop(columns=constant_columns)
    else:
        print('定数列は存在しません。')
    return df

# 欠損値を確認・除外する関数
def check_missing_values(df_X, df_y):
    missing_X = df_X.isnull().sum()
    missing_y = df_y.isnull().sum()
    if missing_X.sum() > 0:
        print('説明変数に欠損値が含まれています。欠損値を除外します。')
        df_X = df_X.dropna()
    if missing_y > 0:
        print('目的変数に欠損値が含まれています。欠損値を除外します。')
        df_y = df_y.dropna()
    return df_X, df_y


print('◆ 説明変数と目的変数のPearsonの相関係数: 線形な相関のみ')
# 定数列の除外
X0 = remove_constant_columns(X0)

# 目的変数が定数か確認
if y0.nunique() <= 1:
    raise ValueError('目的変数 y0 が定数列です。相関係数を計算できません。')

# 欠損値の除外
X0, y0 = check_missing_values(X0, y0)


# 各説明変数と目的変数との相関係数を計算
correlations = {}
for column in X0.columns:
    correlations[column] = X0[column].corr(y0)

# 相関係数をSeriesに変換
correlation_series = pd.Series(correlations)

# 絶対値の降順にソートし、元の符号を保持
top_10_correlations = correlation_series.reindex(correlation_series.abs().sort_values(ascending=False).index).head(10)

# Top 10の相関係数を昇べき順にソート
sorted_top_10_correlations = top_10_correlations.sort_values(ascending=True)

# 色を決定
colors = ['skyblue' if val > 0 else 'lightcoral' for val in correlation_series.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(correlation_series.index, correlation_series.values, color=colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()

print('□ 青は正の相関、赤は負の相関。　1に近いほど強い正の相関、-1に近いほど強い負の相関。0は線形での相関無し')
print('□　　相関係数の絶対値Cの目安： C > 0.7 強い相関、0.4〜0.7 中程度の相関、0.2〜0.4 弱い相関　　C < 0.2 相関無し')
print('')
print('')


print('◇ 説明変数と目的変数のPearsonの相関係数: 相関係数のTop 10 のみ')

# Top 10 correlation colors
top_10_colors = ['skyblue' if val > 0 else 'lightcoral' for val in sorted_top_10_correlations.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(sorted_top_10_correlations.index, sorted_top_10_correlations.values, color=top_10_colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Top 10 Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()
print('')
print('')
print('')


# XiCorの相関係数のプロット関数
print('◆ 説明変数と目的変数のChatterjee`sの相関係数: 非線形な相関も含む (参考値)')

def xicor(x, y, ties=True):
    np.random.seed(42)
    n = len(x)
    order = np.argsort(x)
    ranked_y = np.argsort(np.argsort(y[order], kind='mergesort'))

    if ties:
        l = np.argsort(np.argsort(y[order], kind='max'))
        r = ranked_y.copy()
        return 1 - n * np.sum(np.abs(r[1:] - r[:-1])) / (2 * np.sum(l * (n - l)))
    else:
        return 1 - 3 * np.sum(np.abs(ranked_y[1:] - ranked_y[:-1])) / (n**2 - 1)

def plot_xicor(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor(df)
print('')


print('◇ 説明変数と目的変数のChatterjee`sの相関係数: 相関係数のTop 10 のみ')
def plot_xicor10(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)[-10:]  # 上位10の特徴量を選択
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(10, 6))  # グラフのサイズを調整
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('Top 10 XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor10(df)
print('')
print('\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\thttps://arxiv.org/abs/1909.10140 参照')
print('')


from datetime import datetime
start_time = datetime.now()
formatted_start_time2 = start_time.strftime('%Y%m%d%H%M')


# 出力ファイル
filename0 = os.path.basename(path_table)
filename = os.path.splitext(filename0)[0]
filepath = path_table + '/AllEvaluationResultsFor' + filename + str(formatted_start_time2) + '.xlsx'
filepath2 = path_table + '/AllAUCResultsFor' + filename + str(formatted_start_time2) + '.csv'
print('◆　計算結果の出力先')
print(filepath)
print('')
print('')


# 現在時刻（計算開始時）
from datetime import datetime
start_time = datetime.now()
formatted_start_time = start_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
formatted_start_time2 = start_time.strftime('%Y年%m月%d日 %H時%M分%')
print(f'◆ Present Time (Start): {formatted_start_time}')
print('')



# 各機械学習モデルの実行

# データがそれほど大きくないとき。TabNet Optunaは外してある
if bigdata ==0:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    print('')
    res_logistic_S = LogisticKFoldS()
    print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    print('')
    res_lasso_S = LassoKFoldS(0.01)
    print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    print('')
    res_lasso_optuna_S = LassoKFoldOptunaS()
    print('')
    print('')
    res_ridge_N = RidgeKFoldN(1.0)
    print('')
    print('')
    res_ridge_S = RidgeKFoldS(1.0)
    print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    print('')
    res_ridge_optuna_S = RidgeKFoldOptunaS()
    print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    print('')
    res_SVM_S = SVMKFoldS()
    print('')
    print('')
    res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    print('')
    print('')
    res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    print('')
    print('')
    res_GaussianN = GaussianProcessKFold(scaler='norm')
    print('')
    print('')
    res_GaussianS = GaussianProcessKFold(scaler='stand')
    print('')
    print('')
    res_Gradient_Boosting = GradientBoostingKFold()
    print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    print('')
    res_LightGBM_optuna = LightGBMOptunaKFold()
    print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    print('')
    res_CatBoost = CatBoostKFold()
    print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    print('')
    res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')


    res_list = [
                res_linear,
                                res_logistic_N,
                                res_logistic_S,
                                res_lasso_N,
                                res_lasso_S,
                                res_lasso_optuna_N,
                                res_lasso_optuna_S,
                                res_ridge_N,
                                res_ridge_S,
                                res_ridge_optuna_N,
                                res_ridge_optuna_S,
                                res_SVM_N,
                                res_SVM_S,
                                res_LDAn,
                                res_LDAs,
                                res_GaussianN,
                                res_GaussianS,
                                res_Gradient_Boosting,
                                res_Adaboost,
                                res_RandomForest,
                                res_RandomForest_optuna,
                                res_XGBoost,
                                res_XGBoost_optuna,
                                res_LightGBM,
                                res_LightGBM_optuna,
                                res_LightGBMTuner_optuna,
                                res_CatBoost,
                                res_YDF,
                                res_YDF_optuna,
                                res_MLPn,
                                res_MLPs,
                                res_TabNetN,
                                res_TabNetS,
                                # res_TabNetN_optuna,
                                # res_TabNetS_optuna,
]

    # res_listの最初の要素を表計算のための出力とする
    results = pd.concat([item[0] for item in res_list], axis=0)
    res_auc = [item[1] for item in res_list]
    results.to_excel(filepath)


# データが大きいとき
else:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    print('')
    res_logistic_S = LogisticKFoldS()
    print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    print('')
    res_lasso_S= LassoKFoldS(0.01)
    print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    print('')
    res_lasso_optuna_S = LassoKFoldOptunaS()
    print('')
    print('')
    res_ridge_N = RidgeKFoldN(1.0)
    print('')
    print('')
    res_ridge_S = RidgeKFoldS(1.0)
    print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    print('')
    res_ridge_optuna_S = RidgeKFoldOptunaS()
    print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    print('')
    res_SVM_S = SVMKFoldS()
    print('')
    print('')
    res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    print('')
    print('')
    res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    print('')
    print('')
    res_GaussianN = GaussianProcessKFold(scaler='norm')
    print('')
    print('')
    res_GaussianS = GaussianProcessKFold(scaler='stand')
    print('')
    print('')
    res_Gradient_Boosting = GradientBoostingKFold()
    print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    print('')
    res_LightGBM_optuna = LightGBMOptunaKFold()
    print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    print('')
    res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    print('')
    print('')
    res_MLPn = MLPKFoldN(n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_MLPs = MLPKFoldS(n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=64)
    print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=64)
    print('')
    # print('')
    # res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    # print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')

    # CatBoostとTabNet Optunaは長時間かかるため外してある。
    res_list2 = [
                res_linear,
                res_logistic_N,
                res_logistic_S,
                res_lasso_N,
                res_lasso_S,
                res_lasso_optuna_N,
                res_lasso_optuna_S,
                res_ridge_N,
                res_ridge_S,
                res_ridge_optuna_N,
                res_ridge_optuna_S,
                res_SVM_N,
                res_SVM_S,
                res_LDAn,
                res_LDAs,
                res_GaussianN,
                res_GaussianS,
                res_Gradient_Boosting,
                res_Adaboost,
                res_RandomForest,
                res_RandomForest_optuna,
                res_XGBoost,
                res_XGBoost_optuna,
                res_LightGBM,
                res_LightGBM_optuna,
                res_LightGBMTuner_optuna,
            #  res_CatBoost,
                res_YDF,
                res_YDF_optuna,
                res_MLPn,
                res_MLPs,
                res_TabNetN,
                res_TabNetS,
            #  res_TabNetN_optuna,
            #  res_TabNetS_optuna,
]

    results = pd.concat([item[0] for item in res_list2], axis=0)
    res_auc = [item[1] for item in res_list2]
    results.to_excel(filepath)


# プロットの準備
plt.figure(figsize=(14, 10))  # 図のサイズを少し大きく設定

# AUCが大きい順にres_listの２番目のデータをソート。res_auc の各エントリは (fpr, tpr, auc, name) の形式
data_sorted = sorted(res_auc, key=lambda x: x[2], reverse=True)

# 最もAUCが高いエントリを取得
if data_sorted:
    top_auc = data_sorted[0][2]
else:
    top_auc = None

# 各エントリをループしてプロット
for idx, entry in enumerate(data_sorted):
    fpr, tpr, auc, name = entry
    # NaNが含まれているエントリはスキップ
    if np.isnan(auc) or np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
        continue
    # 最もAUCが高いエントリは太くプロット
    if idx == 0:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=3.5)
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=1.5)

# 対角線のプロット
plt.plot([0, 1], [0, 1], 'k--', label='random prediction')

# グラフの装飾
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curves Comparison', fontsize=16)
plt.grid(True)

# アスペクト比を1に設定
plt.gca().set_aspect('equal', adjustable='box')

# 軸の範囲を0から1に設定
plt.xlim([0, 1])
plt.ylim([0, 1])

# 凡例を図の右側外側に配置
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop=font_prop)

# レイアウトの調整
plt.tight_layout()

roc_plot_path = os.path.join(path_figure, "all_ROC_curves.png")
plt.savefig(roc_plot_path, bbox_inches='tight')

print('')
print("■ Figure overlaying all the ROC curves by all the ML models")
print('')

# プロットの表示
plt.show()
plt.close()

print('')
print(f"■ All the ROC curves saved to: {roc_plot_path}")
print('')
print('')



# 計算終了時の現在時刻
end_time = datetime.now()
formatted_end_time = end_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
print(f'◆ Present Time (End): {formatted_end_time}')

# 計算に要した時間（終了時刻 - 開始時刻）
elapsed_time = end_time - start_time

# 時間、分、秒に分割して表示
hours, remainder = divmod(elapsed_time.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print('')
print(f'計算に要した時間: {hours} 時間, {minutes} 分, {seconds} 秒')
print('')


print('◆　計算結果の出力先')
print('')
print('1. 数値データ:   ' + path_table)
print('')
print('2. 画像データ:   ' + path_figure)
print('')
print('3. モデル:      ' + path_model)
print('')
print('')

## 5) Full Set (機械学習の全モデル) ☜ GPUが必須

In [ ]:
print('')
print('■ 二値分類のFull Verson')
print('')
print(f'◆ The sample size of dataset: {n_samples}')
print('')

# 説明変数と目的変数を分離: Pandas形式
X0 = df.iloc[:, :-1]
y0 = df.iloc[:, -1]

# データの大きさ
print('◆　行と列の数')
rows, cols = df.shape
print(f'データの行数(サンプル数) {rows},  データの列数(説明変数＋目的変数の数) {cols}')
if rows >= 5000 or cols >= 5000:
    bigdata = 1
    print('')
    print('◇ データは Big Data のため、Random Forest と LightGBM への Optuna適用 と CatBoost の利用が省かれる。')
elif rows < 5000 and cols < 5000:
    bigdata = 0
    print('')
    print('◇ データは Big Data ではないため、Optuna は適用可能なモデルには適用する。')
print('')

print('')
# データの分割数 k
print(f"◆ Chosen value of k for k-fold cross-validation: {k}")
print('')
print('')

# 目的変数の分布の表示
print('◆　目的変数の分布')
sns.displot(df.iloc[:,-1].dropna())
print('　　　　　陽性症例の割合(事前確率): ' + str(round(100*(np.count_nonzero(y>0)/len(y)), 5)) + ' %')
plt.show()
print('')
print('')


import matplotlib.font_manager as fm
# フォントファイルのパスを確認
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# フォントプロパティを設定
font_prop = fm.FontProperties(fname=font_path)




# データの線形性の確認

# 線形回帰モデルによる決定係数R2
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

# (二値分類のため)Logistic回帰モデルによる擬似決定係数R2: NagelkerkeのR2 (Cox-Snellの擬似R2を0〜1にscalingしたもの)
import numpy as np
from sklearn.linear_model import LogisticRegression
def nagelkerke_r2(X, y, **logistic_kwargs):
    """
    X: array-like of shape (n_samples, n_features)
    y: array-like of shape (n_samples,), binary target (0 or 1)
    logistic_kwargs:
        追加の LogisticRegression 引数（solver, C, penalty など）

    Returns
    -------
    float
        Nagelkerke の R²
    """
    # 1) モデルのフィッティング
    model = LogisticRegression(**logistic_kwargs)
    model.fit(X, y)

    # 2) 予測確率（正クラス）の取得
    p = model.predict_proba(X)[:, 1]
    # log(0) を防ぐため eps 分だけクリップ
    eps = np.finfo(float).eps
    p = np.clip(p, eps, 1 - eps)

    # 3) フルモデルの対数尤度
    ll_model = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # 4) ヌルモデル（切片のみ、常に y の平均を予測）の対数尤度
    p_null = np.clip(np.mean(y), eps, 1 - eps)
    ll_null = np.sum(y * np.log(p_null) + (1 - y) * np.log(1 - p_null))

    # 5) Cox–Snell の R²
    n = len(y)
    r2_cs = 1 - np.exp((ll_null - ll_model) * 2 / n)

    # 6) Nagelkerke の R²
    r2_nagelkerke = r2_cs / (1 - np.exp(2 * ll_null / n))

    return r2_nagelkerke

pseudo_r2 = nagelkerke_r2(
    X, y,
    penalty='l2',     # リッジ（L2）正則化
    C=1.0,            # デフォルト付近からスタート
    solver='lbfgs',   # 中小規模向き
    max_iter=500,     # 収束用に少し余裕を
    tol=1e-6,          # 精度重視
    class_weight='balanced' # Imbalanced Dataにも対応
)


print(f'◆　説明変数と目的変数の間の決定係数R2')
print('')
print(f'◇ Nagelkerke の擬似R² (標準設定) = {pseudo_r2:.4f}')
print(f'  擬似決係数R²の見方　0.2未満: 弱い説明力、0.2〜0.5: 中程度の説明力、0.5以上: 強い説明力')
print('')
print(f'◇ 線形回帰による決定係数R² = {r2:.4f}')
print('   決定係数R²の見方　 0.3未満: 線形性なし、0.3〜0.5: 弱い線形性、0.5〜0.7: 中程度の線形性、0.7〜0.9: 強い線形性、0.9以上: 完全に線形')
print('')
print('  (擬似決係数R² > 0.5　または  決定係数R² > 0.7 のとき、線形性があると見なせる。二値分類では擬似R²の方が妥当)')
print('')
if pseudo_r2 >= 0.5:
    print('')
    print('◇ データの線形性が見られるため、SVM の kernel と XGBoost の booster を線形にする。')
    linear = 1
elif pseudo_r2 < 0.5:
    print('')
    print('◇ データの線形性はそれほど見られないため、線形モデルの優先は行わない。')
    linear = 0
print('')
print(f'linear = {linear:.1f}')
print('')
print('')



# 残差プロット
print('◆　線形モデルによる目的変数の予測値の残差プロット')
y_pred = model.predict(X)
residuals = y - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('予測値', fontproperties=font_prop)
plt.ylabel('残差', fontproperties=font_prop)
plt.title('予測値と残差のプロット', fontproperties=font_prop)
plt.show()
print('')
print('')



# 連続変数の場合の線形性の判定

# # RESET（Regression Specification Error Test）
# from statsmodels.stats.diagnostic import linear_reset
# import statsmodels.api as sm

# ols = sm.OLS(y, sm.add_constant(X)).fit()
# print(linear_reset(ols, power=2))

# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression

# # 線形モデル vs 多項式モデル（2次項・3次項を加えたモデル）
# poly2 = make_pipeline(PolynomialFeatures(2), LinearRegression())
# score_lin = cross_val_score(LinearRegression(), X, y, cv=5, scoring="r2").mean()
# score_poly2 = cross_val_score(poly2, X, y, cv=5, scoring="r2").mean()

# print("線形モデル R²:", score_lin)
# print("2次多項式モデル R²:", score_poly2)



# 定数列を除外する関数
def remove_constant_columns(df):
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    if constant_columns:
        print(f'以下の定数列を除外します: {constant_columns}')
        df = df.drop(columns=constant_columns)
    else:
        print('定数列は存在しません。')
    return df

# 欠損値を確認・除外する関数
def check_missing_values(df_X, df_y):
    missing_X = df_X.isnull().sum()
    missing_y = df_y.isnull().sum()
    if missing_X.sum() > 0:
        print('説明変数に欠損値が含まれています。欠損値を除外します。')
        df_X = df_X.dropna()
    if missing_y > 0:
        print('目的変数に欠損値が含まれています。欠損値を除外します。')
        df_y = df_y.dropna()
    return df_X, df_y


print('◆ 説明変数と目的変数のPearsonの相関係数: 線形な相関のみ')
# 定数列の除外
X0 = remove_constant_columns(X0)

# 目的変数が定数か確認
if y0.nunique() <= 1:
    raise ValueError('目的変数 y0 が定数列です。相関係数を計算できません。')

# 欠損値の除外
X0, y0 = check_missing_values(X0, y0)


# 各説明変数と目的変数との相関係数を計算
correlations = {}
for column in X0.columns:
    correlations[column] = X0[column].corr(y0)

# 相関係数をSeriesに変換
correlation_series = pd.Series(correlations)

# 絶対値の降順にソートし、元の符号を保持
top_10_correlations = correlation_series.reindex(correlation_series.abs().sort_values(ascending=False).index).head(10)

# Top 10の相関係数を昇べき順にソート
sorted_top_10_correlations = top_10_correlations.sort_values(ascending=True)

# 色を決定
colors = ['skyblue' if val > 0 else 'lightcoral' for val in correlation_series.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(correlation_series.index, correlation_series.values, color=colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()

print('□ 青は正の相関、赤は負の相関。　1に近いほど強い正の相関、-1に近いほど強い負の相関。0は線形での相関無し')
print('□　　相関係数の絶対値Cの目安： C > 0.7 強い相関、0.4〜0.7 中程度の相関、0.2〜0.4 弱い相関　　C < 0.2 相関無し')
print('')
print('')


print('◇ 説明変数と目的変数のPearsonの相関係数: 相関係数のTop 10 のみ')

# Top 10 correlation colors
top_10_colors = ['skyblue' if val > 0 else 'lightcoral' for val in sorted_top_10_correlations.values]

# 水平のバープロットを作成
plt.figure(figsize=(10, 8))
plt.barh(sorted_top_10_correlations.index, sorted_top_10_correlations.values, color=top_10_colors)
plt.xlabel('Pearson Correlation Coefficient', fontproperties=font_prop)
plt.ylabel('Features', fontproperties=font_prop)
plt.title('Top 10 Pearson Correlation Coefficient between Features and Target Variable', fontproperties=font_prop)
plt.grid(axis='x', linestyle='--', alpha=0.7)
# y軸のラベルフォントを変更
ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)
plt.show()
print('')
print('')
print('')


# XiCorの相関係数のプロット関数
print('◆ 説明変数と目的変数のChatterjee`sの相関係数: 非線形な相関も含む (参考値)')

def xicor(x, y, ties=True):
    np.random.seed(42)
    n = len(x)
    order = np.argsort(x)
    ranked_y = np.argsort(np.argsort(y[order], kind='mergesort'))

    if ties:
        l = np.argsort(np.argsort(y[order], kind='max'))
        r = ranked_y.copy()
        return 1 - n * np.sum(np.abs(r[1:] - r[:-1])) / (2 * np.sum(l * (n - l)))
    else:
        return 1 - 3 * np.sum(np.abs(ranked_y[1:] - ranked_y[:-1])) / (n**2 - 1)

def plot_xicor(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(size_x, size_y))
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor(df)
print('')


print('◇ 説明変数と目的変数のChatterjee`sの相関係数: 相関係数のTop 10 のみ')
def plot_xicor10(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    feature_names = df.columns[:-1]
    correlations = []
    num_features = X.shape[1]

    for i in range(num_features):
        corr = xicor(X[:, i], y)
        correlations.append(corr)

    sorted_indices = np.argsort(correlations)[-10:]  # 上位10の特徴量を選択
    sorted_correlations = np.array(correlations)[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]

    colors = ['skyblue' if corr > 0 else 'lightcoral' for corr in sorted_correlations]

    plt.figure(figsize=(10, 6))  # グラフのサイズを調整
    plt.barh(sorted_feature_names, sorted_correlations, color=colors)
    plt.xlabel('XiCor Value', fontproperties=font_prop)
    plt.ylabel('Features', fontproperties=font_prop)
    plt.title('Top 10 XiCor Values for Features', fontproperties=font_prop)
    # y軸のラベルフォントを変更
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.show()

plot_xicor10(df)
print('')
print('\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\thttps://arxiv.org/abs/1909.10140 参照')
print('')


from datetime import datetime
start_time = datetime.now()
formatted_start_time2 = start_time.strftime('%Y%m%d%H%M')


# 出力ファイル
filename0 = os.path.basename(path_table)
filename = os.path.splitext(filename0)[0]
filepath = path_table + '/AllEvaluationResultsFor' + filename + str(formatted_start_time2) + '.xlsx'
filepath2 = path_table + '/AllAUCResultsFor' + filename + str(formatted_start_time2) + '.csv'
print('◆　計算結果の出力先')
print(filepath)
print('')
print('')


# 現在時刻（計算開始時）
from datetime import datetime
start_time = datetime.now()
formatted_start_time = start_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
formatted_start_time2 = start_time.strftime('%Y年%m月%d日 %H時%M分%')
print(f'◆ Present Time (Start): {formatted_start_time}')
print('')



# 各機械学習モデルの実行

# データがそれほど大きくないとき
if bigdata ==0:

    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    print('')
    res_logistic_S = LogisticKFoldS()
    print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    print('')
    res_lasso_S = LassoKFoldS(0.01)
    print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    print('')
    res_lasso_optuna_S = LassoKFoldOptunaS()
    print('')
    print('')
    res_ridge_N = RidgeKFoldN(1.0)
    print('')
    print('')
    res_ridge_S = RidgeKFoldS(1.0)
    print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    print('')
    res_ridge_optuna_S = RidgeKFoldOptunaS()
    print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    print('')
    res_SVM_S = SVMKFoldS()
    print('')
    print('')
    res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    print('')
    print('')
    res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    print('')
    print('')
    res_GaussianN = GaussianProcessKFold(scaler='norm')
    print('')
    print('')
    res_GaussianS = GaussianProcessKFold(scaler='stand')
    print('')
    print('')
    res_Gradient_Boosting = GradientBoostingKFold()
    print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    print('')
    res_LightGBM_optuna = LightGBMOptunaKFold()
    print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    print('')
    res_CatBoost = CatBoostKFold()
    print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    print('')
    res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    print('')
    res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    print('')
    print('')
    res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    print('')




    res_list = [
                res_linear,
                                res_logistic_N,
                                res_logistic_S,
                                res_lasso_N,
                                res_lasso_S,
                                res_lasso_optuna_N,
                                res_lasso_optuna_S,
                                res_ridge_N,
                                res_ridge_S,
                                res_ridge_optuna_N,
                                res_ridge_optuna_S,
                                res_SVM_N,
                                res_SVM_S,
                                res_LDAn,
                                res_LDAs,
                                res_GaussianN,
                                res_GaussianS,
                                res_Gradient_Boosting,
                                res_Adaboost,
                                res_RandomForest,
                                res_RandomForest_optuna,
                                res_XGBoost,
                                res_XGBoost_optuna,
                                res_LightGBM,
                                res_LightGBM_optuna,
                                res_LightGBMTuner_optuna,
                                res_CatBoost,
                                res_YDF,
                                res_YDF_optuna,
                                res_MLPn,
                                res_MLPs,
                                res_TabNetN,
                                res_TabNetS,
                                res_TabNetN_optuna,
                                res_TabNetS_optuna,
]

    # res_listの最初の要素を表計算のための出力とする
    results = pd.concat([item[0] for item in res_list], axis=0)
    res_auc = [item[1] for item in res_list]
    results.to_excel(filepath)


# データが大きいとき
else:

    # CatBoostとOptunaによるTabNet normalizedは長時間かかるため外してある。
    print('')
    res_linear = LinearKFold()
    print('')
    print('')
    res_logistic_N = LogisticKFoldN()
    print('')
    print('')
    res_logistic_S = LogisticKFoldS()
    print('')
    print('')
    res_lasso_N = LassoKFoldN(0.01)
    print('')
    print('')
    res_lasso_S = LassoKFoldS(0.01)
    print('')
    print('')
    res_lasso_optuna_N = LassoKFoldOptunaN()
    print('')
    print('')
    res_lasso_optuna_S = LassoKFoldOptunaS()
    print('')
    print('')
    res_ridge_N = RidgeKFoldN(1.0)
    print('')
    print('')
    res_ridge_S = RidgeKFoldS(1.0)
    print('')
    print('')
    res_ridge_optuna_N = RidgeKFoldOptunaN()
    print('')
    print('')
    res_ridge_optuna_S = RidgeKFoldOptunaS()
    print('')
    print('')
    res_SVM_N = SVMKFoldN()
    print('')
    print('')
    res_SVM_S = SVMKFoldS()
    print('')
    print('')
    res_LDAn = LinearDiscriminantAnalysisKFold(scaler='norm')
    print('')
    print('')
    res_LDAs = LinearDiscriminantAnalysisKFold(scaler='stand')
    print('')
    print('')
    res_GaussianN = GaussianProcessKFold(scaler='norm')
    print('')
    print('')
    res_GaussianS = GaussianProcessKFold(scaler='stand')
    print('')
    print('')
    res_Gradient_Boosting = GradientBoostingKFold()
    print('')
    print('')
    res_Adaboost = AdaBoostKFold()
    print('')
    print('')
    res_RandomForest = RandomForestKFold()
    print('')
    print('')
    res_RandomForest_optuna = RandomForestOptunaKFold()
    print('')
    print('')
    res_XGBoost = XGBoostKFold()
    print('')
    print('')
    res_XGBoost_optuna = XGBoostOptunaKFold()
    print('')
    print('')
    res_LightGBM = LightGBMKFold()
    print('')
    print('')
    res_LightGBM_optuna = LightGBMOptunaKFold()
    print('')
    print('')
    res_LightGBMTuner_optuna = LightGBMTunerOptunaKFold()
    print('')
    # print('')
    # res_CatBoost = CatBoostKFold()
    # print('')
    print('')
    res_YDF = YggdrasilDecisionForestKFold()
    print('')
    print('')
    res_YDF_optuna = YggdrasilDecisionForestOptunaKFold()
    print('')
    print('')
    res_MLPn = MLPKFold(scaler='norm', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_MLPs = MLPKFold(scaler='stand', n1=128, n2=32, max_iter=500)
    print('')
    print('')
    res_TabNetN = TabNetKFoldN(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    print('')
    res_TabNetS = TabNetKFoldS(n1=32, n2=32, step_size=20, max_epochs=100, batch_size=32)
    print('')
    print('')
    res_TabNetN_optuna = TabNetOptunaKFoldN(max_epochs=100, batch_size=32, n_trials=20)
    print('')
    # print('')
    # res_TabNetS_optuna = TabNetOptunaKFoldS(max_epochs=100, batch_size=32, n_trials=20)
    # print('')



    # CatBoostとOptunaによるTabNet normalizedは長時間かかるため外してある。
    res_list2 = [
                res_linear,
                res_logistic_N,
                res_logistic_S,
                res_lasso_N,
                res_lasso_S,
                res_lasso_optuna_N,
                res_lasso_optuna_S,
                res_ridge_N,
                res_ridge_S,
                res_ridge_optuna_N,
                res_ridge_optuna_S,
                res_SVM_N,
                res_SVM_S,
                res_LDAn,
                res_LDAs,
                res_GaussianN,
                res_GaussianS,
                res_Gradient_Boosting,
                res_Adaboost,
                res_RandomForest,
                res_RandomForest_optuna,
                res_XGBoost,
                res_XGBoost_optuna,
                res_LightGBM,
                res_LightGBM_optuna,
                res_LightGBMTuner_optuna,
                # res_CatBoost,
                res_YDF,
                res_YDF_optuna,
                res_MLPn,
                res_MLPs,
                res_TabNetN,
                res_TabNetS,
                res_TabNetN_optuna,
                # res_TabNetS_optuna,
]

    results = pd.concat([item[0] for item in res_list2], axis=0)
    res_auc = [item[1] for item in res_list2]
    results.to_excel(filepath)


# プロットの準備
plt.figure(figsize=(14, 10))  # 図のサイズを少し大きく設定

# AUCが大きい順にres_listの２番目のデータをソート。res_auc の各エントリは (fpr, tpr, auc, name) の形式
data_sorted = sorted(res_auc, key=lambda x: x[2], reverse=True)

# 最もAUCが高いエントリを取得
if data_sorted:
    top_auc = data_sorted[0][2]
else:
    top_auc = None

# 各エントリをループしてプロット
for idx, entry in enumerate(data_sorted):
    fpr, tpr, auc, name = entry
    # NaNが含まれているエントリはスキップ
    if np.isnan(auc) or np.any(np.isnan(fpr)) or np.any(np.isnan(tpr)):
        continue
    # 最もAUCが高いエントリは太くプロット
    if idx == 0:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=3.5)
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=1.5)

# 対角線のプロット
plt.plot([0, 1], [0, 1], 'k--', label='random prediction')

# グラフの装飾
plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('ROC Curves Comparison', fontsize=16)
plt.grid(True)

# アスペクト比を1に設定
plt.gca().set_aspect('equal', adjustable='box')

# 軸の範囲を0から1に設定
plt.xlim([0, 1])
plt.ylim([0, 1])

# 凡例を図の右側外側に配置
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop=font_prop)

# レイアウトの調整
plt.tight_layout()

roc_plot_path = os.path.join(path_figure, "all_ROC_curves.png")
plt.savefig(roc_plot_path, bbox_inches='tight')

print('')
print("■ Figure overlaying all the ROC curves by all the ML models")
print('')

# プロットの表示
plt.show()
plt.close()

print('')
print(f"■ All the ROC curves saved to: {roc_plot_path}")
print('')
print('')



# 計算終了時の現在時刻
end_time = datetime.now()
formatted_end_time = end_time.strftime('%Y年%m月%d日 %H時%M分%S秒')
print(f'◆ Present Time (End): {formatted_end_time}')

# 計算に要した時間（終了時刻 - 開始時刻）
elapsed_time = end_time - start_time

# 時間、分、秒に分割して表示
hours, remainder = divmod(elapsed_time.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print('')
print(f'計算に要した時間: {hours} 時間, {minutes} 分, {seconds} 秒')
print('')


print('◆　計算結果の出力先')
print('')
print('1. 数値データ:   ' + path_table)
print('')
print('2. 画像データ:   ' + path_figure)
print('')
print('3. モデル:      ' + path_model)
print('')
print('')

## 6) Model by Model (１つのモデルを評価)

### 1. Linear Regression

In [ ]:
LinearKFold()

### 2. LassoKFoldN

In [ ]:
LassoKFoldN()

### 3. LassoKFoldS

In [ ]:
LassoKFoldS()

### 4. LassoKFoldOptunaN

In [ ]:
LassoKFoldOptunaN(n_trials=50, target='logloss')

### 5. LassoKFoldOptunaS

In [ ]:
LassoKFoldOptunaS()

### 6. RidgeKFoldN

In [ ]:
RidgeKFoldN()

### 7. RidgeKFoldS

In [ ]:
RidgeKFoldS()

### 8. RidgeKFoldOptunaN

In [ ]:
RidgeKFoldOptunaN()

### 9. RidgeKFoldOptunaS

In [ ]:
RidgeKFoldOptunaS()

### 10. LogisticKFoldN

In [ ]:
LogisticKFoldN()

### 11. LogisticKFoldS

In [ ]:
LogisticKFoldS()

### 12. SVMKFoldN

In [ ]:
SVMKFoldN()

### 13. SVMKFoldS

In [ ]:
SVMKFoldS()

### 14. RandomForestKFold

In [ ]:
RandomForestKFold()

### 15. RandomForestOptunaKFold

In [ ]:
RandomForestOptunaKFold()

### 16. XGBoostKFold

In [ ]:
XGBoostKFold()

### 17. XGBoostOptunaKFold

In [ ]:
XGBoostOptunaKFold()

### 18. LightGBMKFold

In [ ]:
LightGBMKFold()

### 19. LightGBMOptunaKFold

In [ ]:
LightGBMOptunaKFold(target='logloss')

### 20. LightGBMTunerOptunaKFold

In [ ]:
LightGBMTunerOptunaKFold()

### 21. CatBoostKFold

In [ ]:
CatBoostKFold()

### 22. YggdrasilDecisionForestKFold

In [ ]:
YggdrasilDecisionForestKFold()

### 23. YggdrasilDecisionForestOptunaKFold

In [ ]:
YggdrasilDecisionForestOptunaKFold()

### 24. TabNetKFoldN

In [ ]:
TabNetKFoldN()

### 25. TabNetKFoldS

In [ ]:
TabNetKFoldS()

### 26. TabNetOptunaKFoldN ☜ GPUが必須

In [ ]:
TabNetOptunaKFoldN()

### 27. TabNetOptunaKFoldS ☜ GPUが必須

In [ ]:
TabNetOptunaKFoldS()

### 28. MLPKFold with normalization

In [ ]:
MLPKFold(scaler='norm')

### 29. MLPKFold with standardization

In [ ]:
MLPKFold(scaler='stand')

### 30. GradientBoostingKFold

In [ ]:
GradientBoostingKFold()

### 31. AdaBoostKFold

In [ ]:
AdaBoostKFold()

### 32. LinearDiscriminantAnalysisKFold Normalized

In [ ]:
LinearDiscriminantAnalysisKFold(scaler='norm')

### 33. LinearDiscriminantAnalysisKFold Standardized

In [ ]:
LinearDiscriminantAnalysisKFold(scaler='stand')

### 34. GaussianProcessKFold Normalized

In [ ]:
GaussianProcessKFold(scaler='norm')

### 35. GaussianProcessKFold Standardized

In [ ]:
GaussianProcessKFold(scaler='stand')